In [55]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from collections import defaultdict
from datetime import date

import re # for regular expressions
import string
import os
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import pos_tag
from nltk.tokenize import TweetTokenizer
import advertools as adv

In [2]:
# read tweets from csv file
tweets = pd.read_csv('./data/0401_UkraineCombinedTweetsDeduped.csv')
tweets = tweets[tweets['language']=="en"]
tweets = tweets[["tweetcreatedts", "text"]]
tweets.head()

C:\Users\51588\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,tweetcreatedts,text
0,2022-04-01 00:00:00.000000,⚡The Ukrainian Air Force would like to address...
1,2022-04-01 00:00:00.000000,Chernihiv oblast. Ukrainians welcome their lib...
2,2022-04-01 00:00:00.000000,America 🇺🇸 is preparing for something worse th...
3,2022-04-01 00:00:00.000000,JUST IN: #Anonymous has hacked &amp; released ...
4,2022-04-01 00:00:00.000000,***PUBLIC MINT NOW LIVE***\n\nFor \n@billionai...


In [130]:
# for test propose, reduce records
tweets_short = tweets[0:1000]

In [125]:
from nrclex import NRCLex
from collections import Counter
from tqdm import tqdm
tqdm.pandas()
class EmotionDetector:
    # copyed from nrclex.py, remove positive and negative emotions
    lexicon = {'abacus': ['trust'], 'abandon': ['fear',  'sadness'],
           'abandoned': ['anger', 'fear',  'sadness'],
           'abandonment': ['anger', 'fear',  'sadness', 'surprise'],
           'abbot': ['trust'], 'abduction': ['fear',  'sadness', 'surprise'],
           'aberration': ['disgust'], 'abhor': ['anger', 'disgust', 'fear'],
           'abhorrent': ['anger', 'disgust', 'fear'],
           'abject': ['disgust'], 'abnormal': ['disgust'], 'abolish': ['anger'],
           'abominable': ['disgust', 'fear'],
           'abomination': ['anger', 'disgust', 'fear'],
           'abortion': ['disgust', 'fear',  'sadness'], 'abortive': [ 'sadness'],

           'abrupt': ['surprise'], 'abscess': [ 'sadness'], 'absence': ['fear',  'sadness'],
           'absent': [ 'sadness'], 'absentee': [ 'sadness'],
           'absolution': ['joy',  'trust'],

           'abundance': ['anticipation', 'disgust', 'joy',   'trust'],
           'abundant': ['joy'], 'abuse': ['anger', 'disgust', 'fear',  'sadness'],
           'abysmal': [ 'sadness'], 'abyss': ['fear',  'sadness'],
           'academic': [ 'trust'],  'accelerate': ['anticipation'],

           'accident': ['fear',  'sadness', 'surprise'], 'accidental': ['fear',  'surprise'],
           'accidentally': ['surprise'], 'accolade': ['anticipation', 'joy',  'surprise', 'trust'],
           'accompaniment': ['anticipation', 'joy',  'trust'],
           'accomplish': ['joy'], 'accomplished': ['joy'],
           'accord': [ 'trust'], 'account': ['trust'], 'accountability': [ 'trust'],
           'accountable': [ 'trust'], 'accountant': ['trust'], 'accounts': ['trust'],
           'accredited': [ 'trust'],  'accurate': [ 'trust'],
           'accursed': ['anger', 'fear',  'sadness'], 'accusation': ['anger', 'disgust'],
           'accused': ['anger', 'fear'],
           'accuser': ['anger', 'fear'], 'accusing': ['anger', 'fear'],
           'ache': [ 'sadness'], 'achieve': ['joy',  'trust'],
           'achievement': ['anticipation', 'joy',  'trust'], 'aching': [ 'sadness'],

           'acquiring': ['anticipation'], 'acrobat': ['fear', 'joy',  'trust'],
           'actionable': ['anger', 'disgust'],

           'adder': ['anger', 'disgust', 'fear',  'sadness'],
           'addresses': ['anticipation'],
           'adhering': ['trust'],  'adjudicate': ['fear'],
           'administrative': ['trust'], 'admirable': ['joy',  'trust'],
           'admiral': [ 'trust'], 'admiration': ['joy',  'trust'],
           'admire': [ 'trust'],  'admissible': [ 'trust'],
           'admonition': ['fear'], 'adorable': ['joy'],
           'adoration': ['joy',  'trust'], 'adore': ['anticipation', 'joy',  'trust'],
           'adrift': ['anticipation', 'fear',  'sadness'],
           'adultery': ['disgust',  'sadness'],
           'advance': ['anticipation', 'fear', 'joy',  'surprise'],

           'advent': ['anticipation', 'joy',  'trust'], 'adventure': ['anticipation'],
           'adversary': ['anger'],
           'adverse': ['anger', 'disgust', 'fear',  'sadness'],
           'adversity': ['anger', 'fear',  'sadness'], 'advice': ['trust'],
           'advisable': [ 'trust'], 'advise': [ 'trust'], 'advised': ['trust'],
           'adviser': [ 'trust'], 'advocacy': ['anger', 'anticipation', 'joy',  'trust'],
           'advocate': ['trust'],  'aesthetics': ['joy'],
           'affection': ['joy',  'trust'],
           'affirm': [ 'trust'],
           'affirmatively': [ 'trust'], 'afflict': ['fear',  'sadness'],
           'affliction': ['disgust', 'fear',  'sadness'],
           'affluence': ['joy'],
           'affront': ['anger', 'disgust', 'fear',  'sadness', 'surprise'],
           'afraid': ['fear'], 'aftermath': ['anger', 'disgust', 'fear',  'sadness'],
           'aga': ['fear',  'trust'], 'aggravated': ['anger'],
           'aggravating': ['anger',  'sadness'], 'aggravation': ['anger', 'disgust'],
           'aggression': ['anger', 'fear'], 'aggressive': ['anger', 'fear'],
           'aggressor': ['anger', 'fear'], 'aghast': ['disgust', 'fear',  'surprise'],
           'agitated': ['anger'],
           'agitation': ['anger'], 'agonizing': ['fear'],
           'agony': ['anger', 'fear',  'sadness'],
           'agreeable': [ 'trust'], 'agreed': [ 'trust'], 'agreeing': [ 'trust'],
           'agreement': [ 'trust'],
           'ail': [ 'sadness'],
           'ailing': ['fear',  'sadness'],  'airport': ['anticipation'],
           'airs': ['disgust'], 'akin': ['trust'],
           'alarm': ['fear',  'surprise'], 'alarming': ['fear',  'surprise'],
           'alb': ['trust'], 'alcoholism': ['anger', 'disgust', 'fear',  'sadness'],
           'alertness': ['anticipation', 'fear',  'surprise'],
           'alerts': ['anticipation', 'fear', 'surprise'], 'alien': ['disgust', 'fear'],
           'alienate': ['anger', 'disgust'], 'alienated': [ 'sadness'],
           'alienation': ['anger', 'disgust', 'fear',  'sadness'],
           'alive': ['anticipation', 'joy',  'trust'],
           'allegation': ['anger'],  'allegiance': [ 'trust'],
           'alliance': ['trust'],
           'allied': [ 'trust'],
           'allure': ['anticipation', 'joy',  'surprise'],
           'ally': [ 'trust'],  'aloha': ['anticipation', 'joy'],
           'altercation': ['anger'], 'amaze': ['surprise'],
           'amazingly': ['joy',  'surprise'], 'ambassador': [ 'trust'],
           'ambition': ['anticipation', 'joy',  'trust'],
           'ambulance': ['fear', 'trust'], 'ambush': ['anger', 'fear',  'surprise'],
           'amen': ['joy',  'trust'],

           'amicable': ['joy'], 'ammonia': ['disgust'],
           'amnesty': ['joy'], 'amortization': ['trust'],
           'amour': ['anticipation', 'joy',  'trust'], 'amphetamines': ['disgust'],
           'amuse': ['joy'], 'amused': ['joy'], 'amusement': ['joy'],
           'amusing': ['joy'], 'anaconda': ['disgust', 'fear'],
           'analyst': ['anticipation',  'trust'], 'anarchism': ['anger', 'fear'],
           'anarchist': ['anger', 'fear'], 'anarchy': ['anger', 'fear'],
           'anathema': ['anger', 'disgust', 'fear',  'sadness'], 'ancestral': ['trust'],
           'anchorage': [ 'sadness'],
           'angel': ['anticipation', 'joy',  'surprise', 'trust'],
           'angelic': ['joy',  'trust'], 'anger': ['anger'], 'angina': ['fear'],
           'angling': ['anticipation'], 'angry': ['anger', 'disgust'],
           'anguish': ['anger', 'fear',  'sadness'],
           'animated': ['joy'], 'animosity': ['anger', 'disgust', 'fear',  'sadness'],
           'animus': ['anger'], 'annihilate': ['anger', 'fear'],
           'annihilated': ['anger', 'fear',  'sadness'],
           'annihilation': ['anger', 'fear',  'sadness'], 'announcement': ['anticipation'],
           'annoy': ['anger', 'disgust'], 'annoyance': ['anger', 'disgust'],
           'annoying': ['anger'],  'annulment': [ 'sadness'],
           'anomaly': ['fear',  'surprise'],  'answerable': ['trust'],
           'antagonism': ['anger'], 'antagonist': ['anger'],
           'antagonistic': ['anger', 'disgust'], 'anthrax': ['disgust', 'fear',  'sadness'],
           'antichrist': ['anger', 'disgust', 'fear'],
           'anticipation': ['anticipation'], 'anticipatory': ['anticipation'],
           'antidote': ['anticipation',  'trust'], 'antifungal': [ 'trust'],
           'antipathy': ['anger', 'disgust'],
           'antiseptic': [ 'trust'], 'antisocial': ['anger', 'disgust', 'fear',  'sadness'],
           'antithesis': ['anger'], 'anxiety': ['anger', 'anticipation', 'fear',  'sadness'],
           'anxious': ['anticipation', 'fear'], 'apache': ['fear'],
           'apathetic': [ 'sadness'], 'apathy': [ 'sadness'],
           'aphid': ['disgust'],  'apologetic': [ 'trust'],
           'apologize': [ 'sadness', 'trust'],  'apostle': [ 'trust'],
           'apostolic': ['trust'], 'appalling': ['disgust', 'fear'], 'apparition': ['fear', 'surprise'],
           'appeal': ['anticipation'], 'appendicitis': ['fear',  'sadness'],
           'applause': ['joy',  'surprise', 'trust'], 'applicant': ['anticipation'],
           'appreciation': ['joy',  'trust'], 'apprehend': ['fear'],
           'apprehension': ['fear'], 'apprehensive': ['anticipation', 'fear'],
           'apprentice': ['trust'], 'approaching': ['anticipation'], 'approbation': [ 'trust'],
           'approve': ['joy',  'trust'],
           'arbiter': ['trust'],
           'arbitration': ['anticipation'], 'arbitrator': ['trust'], 'archaeology': ['anticipation'],
           'architecture': ['trust'], 'ardent': ['anticipation', 'joy'],
           'argue': ['anger'],
           'argument': ['anger'], 'argumentation': ['anger'],
           'arguments': ['anger'], 'arid': [ 'sadness'],
           'armament': ['anger', 'fear'], 'armaments': ['fear'],
           'armed': ['anger', 'fear'], 'armor': ['fear',  'trust'],
           'armored': ['fear'], 'armory': ['trust'],  'arouse': ['anticipation'],
           'arraignment': ['anger', 'fear',  'sadness'],
           'arrival': ['anticipation'], 'arrive': ['anticipation'],
           'arrogant': ['anger', 'disgust'],
           'arsenic': ['disgust', 'fear',  'sadness'], 'arson': ['anger', 'fear'],
           'art': ['anticipation', 'joy',  'sadness', 'surprise'],
           'artillery': ['fear'],

           'ashamed': ['disgust',  'sadness'], 'ashes': [ 'sadness'],
           'asp': ['fear'], 'aspiration': ['anticipation', 'joy',  'surprise', 'trust'],
           'aspire': ['anticipation', 'joy'], 'aspiring': ['anticipation', 'joy',  'trust'],
           'assail': ['anger', 'fear',  'surprise'],
           'assailant': ['anger', 'fear',  'sadness'],
           'assassin': ['anger', 'fear',  'sadness'], 'assassinate': ['anger', 'fear'],
           'assassination': ['anger', 'fear',  'sadness'], 'assault': ['anger', 'fear'],
           'assembly': [ 'trust'],  'asserting': [ 'trust'],
           'assessment': ['surprise', 'trust'], 'assessor': ['trust'],
           'asshole': ['anger', 'disgust'], 'assignee': ['trust'], 'assist': [ 'trust'],
           'associate': [ 'trust'], 'association': ['trust'],
           'assurance': [ 'trust'], 'assure': ['trust'],
           'assured': [ 'trust'], 'assuredly': ['trust'], 'astonishingly': [ 'surprise'],
           'astonishment': ['joy',  'surprise'], 'astray': ['fear'],
           'astrologer': ['anticipation'],
           'astronomer': ['anticipation'],  'asylum': ['fear'],
           'asymmetry': ['disgust'],  'atherosclerosis': ['fear',  'sadness'],

           'atone': ['anticipation', 'joy',  'trust'],
           'atrocious': ['anger', 'disgust'],
           'atrocity': ['anger', 'disgust', 'fear',  'sadness'],
           'atrophy': ['disgust', 'fear',  'sadness'],
           'attack': ['anger', 'fear'],
           'attacking': ['anger', 'disgust', 'fear',  'sadness', 'surprise'],
           'attainable': ['anticipation'],  'attempt': ['anticipation'],
           'attendance': ['anticipation'], 'attendant': [ 'trust'],
           'attentive': [ 'trust'],  'attenuation': [ 'sadness'],
           'attest': [ 'trust'], 'attestation': ['trust'],
           'attorney': ['anger', 'fear',  'trust'],
           'auction': ['anticipation'],
           'audience': ['anticipation'], 'auditor': ['fear', 'trust'],
           'aunt': [ 'trust'],
           'auspicious': ['anticipation', 'joy'], 'austere': ['fear',  'sadness'],
           'authentic': ['joy',  'trust'], 'authenticate': ['trust'],
           'authentication': ['trust'], 'authenticity': [ 'trust'], 'author': [ 'trust'],
           'authoritative': [ 'trust'], 'authority': [ 'trust'],
           'authorization': [ 'trust'], 'authorize': ['trust'],
           'automatic': ['trust'],
           'autopsy': ['disgust', 'fear',  'sadness'],
           'avalanche': ['fear',  'sadness', 'surprise'], 'avarice': ['anger', 'disgust'],
           'avenger': ['anger'],
           'averse': ['anger', 'disgust', 'fear'], 'aversion': ['anger', 'disgust', 'fear'],
           'avoid': ['fear'], 'avoidance': ['fear'], 'avoiding': ['fear'],
           'await': ['anticipation'], 'award': ['anticipation', 'joy',  'surprise', 'trust'],
           'awful': ['anger', 'disgust', 'fear',  'sadness'], 'awkwardness': ['disgust'],
           'axiom': ['trust'], 'axiomatic': ['trust'],
           'baboon': ['disgust'],
           'baby': ['joy'], 'babysitter': ['trust'],
           'backbone': ['anger',  'trust'], 'backer': ['trust'],
           'backwards': ['disgust'], 'backwater': [ 'sadness'],
           'bacteria': ['disgust', 'fear',  'sadness'], 'bacterium': ['disgust', 'fear'],
           'bad': ['anger', 'disgust', 'fear',  'sadness'], 'badge': ['trust'],
           'badger': ['anger'], 'badly': [ 'sadness'],
           'badness': ['anger', 'disgust', 'fear'], 'bailiff': ['fear',  'trust'],
           'bait': ['fear',  'trust'],
           'bale': ['fear'],
           'ballot': ['anticipation',  'trust'], 'balm': ['anticipation', 'joy'],

           'bane': ['anger', 'disgust', 'fear'],
           'bang': ['anger', 'disgust', 'fear',  'sadness', 'surprise'],
           'banger': ['anger', 'anticipation', 'fear',  'surprise'],
           'banish': ['anger', 'disgust', 'fear',  'sadness'],
           'banished': ['anger', 'fear',  'sadness'],
           'banishment': ['anger', 'disgust',  'sadness'], 'bank': ['trust'], 'banker': ['trust'],
           'bankrupt': ['fear',  'sadness'],
           'bankruptcy': ['anger', 'disgust', 'fear',  'sadness'],
           'banquet': ['anticipation', 'joy'],
           'banshee': ['anger', 'disgust', 'fear',  'sadness'],
           'baptismal': ['joy'], 'barb': ['anger'], 'barbarian': ['fear'],
           'barbaric': ['anger', 'disgust', 'fear'],
           'barf': ['disgust'], 'bargain': [ 'trust'], 'bark': ['anger'],
           'barren': [ 'sadness'], 'barricade': ['fear'],
           'barrier': ['anger'], 'barrow': ['disgust'], 'bartender': ['trust'], 'barter': ['trust'],
           'base': ['trust'],  'basketball': ['anticipation', 'joy'],
           'bastard': ['disgust',  'sadness'], 'bastion': ['anger'],
           'battalion': ['anger'], 'batter': ['anger', 'fear'],
           'battered': ['fear',  'sadness'], 'battery': ['anger'],
           'battle': ['anger'], 'battled': ['anger', 'fear',  'sadness'],
           'battlefield': ['fear'],  'bayonet': ['anger', 'fear'],
           'beach': ['joy'], 'beam': ['joy'], 'beaming': ['anticipation', 'joy'],
           'bear': ['anger', 'fear'],  'bearish': ['anger', 'fear'],
           'beast': ['anger', 'fear'], 'beastly': ['disgust', 'fear'],
           'beating': ['anger', 'fear',  'sadness'], 'beautification': ['joy',  'trust'],
           'beautiful': ['joy'], 'beautify': ['joy'], 'beauty': ['joy'],
           'bedrock': [ 'trust'], 'bee': ['anger', 'fear'], 'beer': ['joy'],
           'befriend': ['joy',  'trust'],
           'beg': [ 'sadness'], 'beggar': [ 'sadness'],
           'begun': ['anticipation'], 'behemoth': ['fear'],
           'believed': ['trust'], 'believer': ['trust'],
           'believing': [ 'trust'], 'belittle': ['anger', 'disgust', 'fear',  'sadness'],
           'belligerent': ['anger', 'fear'], 'bellows': ['anger'],
           'belt': ['anger', 'fear'],  'benefactor': [ 'trust'],
           'benevolence': ['joy',  'trust'],
           'benign': ['joy'], 'bequest': ['trust'], 'bereaved': [ 'sadness'],
           'bereavement': [ 'sadness'],  'berserk': ['anger'],
           'bestial': ['disgust', 'fear'],
           'betray': ['anger', 'disgust',  'sadness', 'surprise'],
           'betrayal': ['anger', 'disgust',  'sadness'],
           'betrothed': ['anticipation', 'joy',  'trust'],
           'beware': ['anticipation', 'fear'],
           'bewildered': ['fear',  'surprise'], 'bewilderment': ['fear', 'surprise'],
           'bias': ['anger'],
           'bickering': ['anger', 'disgust'], 'biennial': ['anticipation'],
           'bier': ['fear',  'sadness'], 'bigot': ['anger', 'disgust', 'fear'],
           'bigoted': ['anger', 'disgust', 'fear',  'sadness'], 'bile': ['anger', 'disgust'],
           'biopsy': ['fear'],
           'birch': ['anger', 'disgust', 'fear'],
           'birth': ['anticipation', 'fear', 'joy',  'trust'],
           'birthday': ['anticipation', 'joy',  'surprise'], 'birthplace': ['anger'],
           'bitch': ['anger', 'disgust', 'fear',  'sadness'],
           'bitterly': ['anger', 'disgust',  'sadness'],
           'bitterness': ['anger', 'disgust',  'sadness'], 'bizarre': [ 'surprise'],
           'black': [ 'sadness'],  'blackmail': ['anger', 'fear'],
           'blackness': ['fear',  'sadness'], 'blame': ['anger', 'disgust'],
           'blanket': ['trust'],
           'blasphemous': ['anger', 'disgust'], 'blasphemy': ['anger'],
           'blast': ['anger', 'fear',  'surprise'], 'blatant': ['anger', 'disgust'],
           'blaze': ['anger'], 'bleak': [ 'sadness'],
           'bleeding': ['disgust', 'fear',  'sadness'],
           'blemish': ['anger', 'disgust', 'fear',  'sadness'],
           'bless': ['anticipation', 'joy',  'trust'], 'blessed': ['joy'],
           'blessing': ['anticipation', 'joy',  'trust'],
           'blessings': ['anticipation', 'joy',  'surprise', 'trust'],
           'blight': ['disgust', 'fear',  'sadness'], 'blighted': ['disgust',  'sadness'],
           'blindfold': ['anticipation', 'fear', 'surprise'],
           'blindly': [ 'sadness'], 'blindness': [ 'sadness'], 'bliss': ['joy'],
           'blissful': ['joy'], 'blister': ['disgust'], 'blitz': ['surprise'],
           'bloated': ['disgust'], 'blob': ['disgust', 'fear'],
           'blockade': ['anger', 'fear',  'sadness'],
           'bloodshed': ['anger', 'disgust', 'fear',  'sadness', 'surprise'],
           'bloodthirsty': ['anger', 'disgust', 'fear'],
           'bloody': ['anger', 'disgust', 'fear',  'sadness'],
           'bloom': ['anticipation', 'joy',  'trust'], 'blossom': ['joy'],
           'blue': ['sadness'],
           'blues': ['fear',  'sadness'],
           'blunder': ['disgust',  'sadness'],
           'board': ['anticipation'],
           'bodyguard': [ 'trust'],
           'bogus': ['anger', 'disgust'], 'boil': ['disgust'],
           'boisterous': ['anger', 'anticipation', 'joy'],

           'bomb': ['anger', 'fear',  'sadness', 'surprise'], 'bombard': ['anger', 'fear'],
           'bombardment': ['anger', 'fear'], 'bombed': ['disgust'],
           'bomber': ['fear', 'sadness'], 'bonanza': ['joy'],
           'bondage': ['fear',  'sadness'],
           'bonus': ['anticipation', 'joy',  'surprise'],

           'boomerang': ['anticipation', 'trust'],
           'boredom': [ 'sadness'],
           'bothering': ['anger',  'sadness'],
           'bottom': [ 'sadness'], 'bottomless': ['fear'],
           'bountiful': ['anticipation', 'joy'], 'bounty': ['anticipation', 'joy',  'trust'],
           'bouquet': ['joy',  'trust'], 'bout': ['anger'],
           'bovine': ['disgust'], 'bowels': ['disgust'], 'boxing': ['anger'],
           'boy': ['disgust'],
           'bran': ['disgust'],
           'brawl': ['anger', 'disgust', 'fear'], 'brazen': ['anger'],
           'break': ['surprise'],
           'breakup': [ 'sadness'],
           'bribery': ['disgust'], 'bridal': ['anticipation', 'joy',  'trust'],
           'bride': ['anticipation', 'joy',  'trust'],
           'bridegroom': ['anticipation', 'joy',  'trust'], 'bridesmaid': ['joy',  'trust'],
           'brigade': ['fear'], 'brighten': ['joy',  'surprise', 'trust'],
           'brilliant': ['anticipation', 'joy',  'trust'],
           'brimstone': ['anger', 'fear'],
           'broadside': ['anticipation'],  'broil': ['anger'],
           'broke': ['fear',  'sadness'], 'broken': ['anger', 'fear',  'sadness'],
           'brothel': ['disgust'], 'brother': [ 'trust'],
           'brotherhood': [ 'trust'], 'brotherly': ['anticipation', 'joy',  'trust'],
           'bruise': ['anticipation'], 'brunt': ['anger'],
           'brutal': ['anger', 'fear'], 'brutality': ['anger', 'fear'],
           'brute': ['anger', 'fear',  'sadness'], 'buck': ['fear',   'surprise'],
           'buddy': ['anticipation', 'joy',  'trust'], 'budget': ['trust'],
           'buffet': ['anger'], 'bug': ['disgust', 'fear'],
           'bugaboo': ['anger', 'fear',  'sadness'], 'bugle': ['anticipation'],

           'bully': ['anger', 'fear'], 'bum': ['disgust',  'sadness'],
           'bummer': ['anger', 'disgust'], 'bunker': ['fear'],
           'burdensome': ['fear',  'sadness'], 'bureaucracy': [ 'trust'],
           'bureaucrat': ['disgust'], 'burglar': ['disgust', 'fear'],
           'burial': ['anger', 'fear',  'sadness'],
           'buried': ['fear',  'sadness'], 'burke': ['anger', 'disgust', 'fear',  'sadness'],
           'burlesque': ['surprise'], 'burnt': ['disgust'], 'bursary': ['trust'], 'bury': ['sadness'],
           'buss': ['joy'], 'busted': ['anger', 'fear'],
           'butcher': ['anger', 'disgust', 'fear'], 'butler': [ 'trust'],

           'buzz': ['anticipation', 'fear'],  'bye': ['anticipation'],
           'bylaw': ['trust'],  'cabal': ['fear'], 'cabinet': [ 'trust'],
           'cable': ['surprise'], 'cacophony': ['anger', 'disgust'],
           'cad': ['anger', 'disgust'],
           'cadaver': ['disgust', 'fear',  'sadness', 'surprise'],
           'cage': [ 'sadness'], 'calamity': ['sadness'],
           'calculation': ['anticipation'], 'calculator': [ 'trust'],
           'calf': ['joy',  'trust'], 'callous': ['anger', 'disgust'],
           'calls': ['anticipation',  'trust'],  'camouflage': ['surprise'],
           'camouflaged': ['surprise'], 'campaigning': ['anger', 'fear'],
           'cancel': [ 'sadness'], 'cancer': ['anger', 'disgust', 'fear',  'sadness'],
           'candid': ['anticipation', 'joy',  'surprise', 'trust'],
           'cane': ['anger', 'fear'], 'canker': ['anger', 'disgust'],
           'cannibal': ['disgust', 'fear'], 'cannibalism': ['disgust'],
           'cannon': ['anger', 'fear'], 'canons': ['trust'], 'cap': ['anticipation', 'trust'],

           'captivate': ['anticipation', 'joy',  'surprise', 'trust'],
           'captive': ['fear',  'sadness'], 'captivity': [ 'sadness'],
           'captor': ['fear'],
           'carcass': ['disgust', 'fear',  'sadness'], 'carcinoma': ['fear',  'sadness'],
           'cardiomyopathy': ['fear',  'sadness'], 'career': ['anticipation'],
           'carelessness': ['anger', 'disgust'],
           'caretaker': [ 'trust'],
           'caries': ['disgust'],
           'carnage': ['anger', 'disgust', 'fear',  'sadness', 'surprise'],
           'carnivorous': ['fear'], 'carol': ['joy',  'trust'],
           'cartridge': ['fear'],  'case': ['fear',  'sadness'],
           'cash': ['anger', 'anticipation', 'fear', 'joy',  'trust'], 'cashier': ['trust'],
           'casket': ['fear',  'sadness'],
           'casualty': ['anger', 'fear',  'sadness'],
           'cataract': ['anticipation', 'fear',  'sadness'],
           'catastrophe': ['anger', 'disgust', 'fear',  'sadness', 'surprise'], 'catch': ['surprise'],
           'catechism': ['disgust'],
           'cathedral': ['joy',  'trust'],
           'caution': ['anger', 'anticipation', 'fear'], 'cautionary': ['fear'],
           'cautious': ['anticipation', 'fear',  'trust'], 'cautiously': ['fear'],
           'celebrated': ['anticipation', 'joy'],
           'celebrating': ['anticipation', 'joy'],
           'celebration': ['anticipation', 'joy',  'surprise', 'trust'],
           'celebrity': ['anger', 'anticipation', 'disgust', 'joy',   'surprise', 'trust'],
           'celestial': ['anticipation', 'joy'], 'cement': ['anticipation', 'trust'],
           'cemetery': ['fear',  'sadness'], 'censor': ['anger', 'disgust', 'fear',  'trust'],
           'center': [ 'trust'],
           'ceremony': ['joy',  'surprise'],
           'certify': ['trust'], 'cess': ['disgust'],
           'chaff': ['anger', 'fear'],
           'chagrin': ['disgust',  'sadness'], 'chairman': [ 'trust'],
           'chairwoman': [ 'trust'], 'challenge': ['anger', 'fear'],
           'champion': ['anticipation', 'joy',  'trust'], 'chance': ['surprise'],
           'chancellor': ['trust'], 'change': ['fear'], 'changeable': ['anticipation', 'surprise'],
           'chant': ['anger', 'anticipation', 'joy',  'surprise'],
           'chaos': ['anger', 'fear',  'sadness'], 'chaotic': ['anger'],
           'chaplain': ['trust'],  'chargeable': ['fear',  'sadness'],
           'charitable': ['anticipation', 'joy',  'trust'],
           'charity': ['joy'],  'charmed': ['joy'],
           'chart': ['trust'],  'chasm': ['fear'],
           'chastity': ['anticipation',  'trust'],

           'cheat': ['anger', 'disgust'], 'checklist': [ 'trust'],
           'cheer': ['anticipation', 'joy',  'surprise', 'trust'],
           'cheerful': ['joy',  'surprise'],
           'cheerfulness': ['anticipation', 'joy',  'trust'], 'cheering': ['joy'],
           'cheery': ['anticipation', 'joy'],
           'chemist': [ 'trust'], 'cherish': ['anticipation', 'joy',  'surprise', 'trust'],
           'chicane': ['anticipation',  'surprise', 'trust'],
           'chicken': ['fear'],  'child': ['anticipation', 'joy'],
           'childhood': ['joy'],
           'chimera': ['fear', 'surprise'], 'chirp': ['joy'],

           'chocolate': ['anticipation', 'joy',  'trust'],
           'choir': ['joy',  'trust'], 'choke': ['anger',  'sadness'],
           'cholera': ['disgust', 'fear',  'sadness'],
           'choral': ['joy'],
           'chronic': [ 'sadness'], 'chronicle': [ 'trust'],
           'chuckle': ['anticipation', 'joy',  'surprise', 'trust'],
           'church': ['anticipation', 'joy',  'trust'],

           'civilization': [ 'trust'],
           'civilized': ['joy',  'trust'], 'claimant': ['anger', 'disgust'],
           'clamor': ['anger', 'anticipation', 'disgust',  'surprise'], 'clan': ['trust'],
           'clap': ['anticipation', 'joy',  'trust'],
           'clash': ['anger'], 'clashing': ['anger', 'fear'],
           'classics': ['joy'],
           'claw': ['anger', 'fear'], 'clean': ['joy',  'trust'],

           'clearance': [ 'trust'],  'cleave': ['fear'],
           'clerical': [ 'trust'],   'cliff': ['fear'],
           'climax': ['anticipation', 'joy',  'surprise', 'trust'], 'clock': ['anticipation'],
           'closeness': ['joy',  'trust'],
           'closure': ['anticipation', 'joy',  'sadness'],
           'clouded': [ 'sadness'], 'cloudiness': ['fear'], 'cloudy': ['sadness'],
           'clown': ['anticipation', 'joy',  'surprise'], 'clue': ['anticipation'],
           'clumsy': ['disgust'], 'coach': ['trust'], 'coalesce': ['trust'],
           'coax': ['trust'], 'cobra': ['fear'],
           'cocaine': [ 'sadness'], 'coerce': ['anger', 'disgust', 'fear'],
           'coercion': ['anger', 'disgust', 'fear',  'sadness'], 'coexist': [ 'trust'],
           'coexisting': ['trust'], 'coffin': ['fear',  'sadness'], 'cogent': [ 'trust'],
           'cohesion': ['trust'],
           'cohesive': [ 'trust'], 'coincidence': ['surprise'],
           'coldness': ['anger', 'disgust', 'fear',  'sadness'],
           'collaborator': ['trust'], 'collapse': ['disgust', 'fear',  'sadness'],
           'collateral': ['trust'], 'collectively': [ 'trust'], 'collision': ['anger'],
           'collusion': ['anger', 'disgust', 'fear',  'sadness'], 'colonel': [ 'trust'],
           'coma': ['fear',  'sadness'],
           'comatose': ['fear',  'sadness'], 'combat': ['anger', 'fear'],
           'combatant': ['anger', 'fear'], 'combative': ['anger', 'fear'],
           'comfort': ['anticipation', 'joy',  'trust'], 'coming': ['anticipation'],
           'commandant': [ 'trust'], 'commanding': [ 'trust'],
           'commemorate': ['anticipation', 'joy',  'sadness'],
           'commemoration': ['anticipation', 'joy'], 'commemorative': ['anticipation'],
           'commendable': ['joy',  'trust'],
           'commerce': ['trust'], 'commission': ['trust'], 'committal': [ 'sadness'],
           'committed': [ 'trust'], 'committee': ['trust'], 'commodore': [ 'trust'],
           'commonplace': ['anticipation', 'trust'], 'commonwealth': [ 'trust'],
           'commotion': ['anger'], 'communicate': [ 'trust'], 'communication': ['trust'],
           'communion': ['joy',  'trust'],
           'communism': ['anger', 'fear',  'sadness'],
           'compact': ['trust'],
           'companion': ['joy',  'trust'], 'compass': ['trust'], 'compassion': ['fear'],

           'compensate': ['anticipation', 'joy',  'surprise', 'trust'],
           'competence': [ 'trust'], 'competency': [ 'trust'],
           'competent': [ 'trust'], 'competition': ['anticipation'],
           'complain': ['anger',  'sadness'],
           'complaint': ['anger'],
           'complement': ['anticipation', 'joy',  'surprise', 'trust'],

           'completing': ['anticipation', 'joy'], 'completion': ['anticipation', 'joy'],
           'compliance': [ 'trust'],
           'complicate': ['anger'],

           'compliment': ['anticipation', 'joy',  'surprise', 'trust'],
           'compost': ['disgust'],
           'compress': ['anger'],
           'comptroller': ['trust'], 'compulsion': ['anger'],
           'comrade': [ 'trust'], 'conceal': [ 'sadness'],
           'concealed': ['anticipation', 'fear',  'surprise'],
           'concealment': ['anger', 'anticipation', 'fear'],
           'concerned': ['fear', 'sadness'],
           'conciliation': ['joy',  'trust'],
           'concord': [ 'trust'], 'concordance': [ 'trust'],
           'concussion': ['anger',  'sadness'], 'condemn': ['anger'],
           'condemnation': ['anger', 'anticipation', 'disgust', 'fear',  'sadness'],
           'condescension': ['anger', 'disgust',  'sadness'],
           'condolence': [ 'sadness'],
           'confederate': [ 'trust'],
           'confess': [  'trust'],
           'confession': ['anticipation', 'fear',  'sadness', 'surprise'],
           'confessional': ['fear', 'trust'], 'confide': ['trust'],
           'confidence': ['fear', 'joy',  'trust'], 'confident': ['joy',  'trust'],
           'confidential': ['trust'], 'confidentially': ['trust'],
           'confine': ['anger', 'fear',  'sadness'],
           'confined': ['anger', 'disgust', 'fear',  'sadness'],
           'confinement': ['anger', 'fear',  'sadness'], 'confirmation': ['trust'],
           'confirmed': [ 'trust'], 'confiscate': ['anger',  'sadness'],
           'conflagration': ['anger', 'fear'],
           'conflict': ['anger', 'fear',  'sadness'],
           'conformity': ['trust'],
           'confront': ['anger'],
           'confusion': ['anger', 'fear'],
           'conglomerate': ['trust'], 'congratulatory': ['joy'], 'congregation': [ 'trust'],
           'congress': ['disgust', 'trust'], 'congressman': ['trust'], 'congruence': [ 'trust'],
           'conjecture': ['anticipation'], 'conjure': ['anticipation', 'surprise'],
           'connective': ['trust'], 'connoisseur': ['joy',  'trust'],
           'conquest': ['anger', 'fear'], 'conscience': [ 'trust'],
           'conscientious': [ 'trust'],
           'consecration': ['anticipation', 'joy',  'sadness', 'trust'], 'consequent': ['anticipation'],
           'conservation': ['anticipation',  'trust'],
           'considerate': [ 'trust'], 'consistency': [ 'trust'],
           'console': [ 'sadness'],  'consort': ['trust'],
           'conspiracy': ['fear'], 'conspirator': ['anger', 'anticipation', 'disgust', 'fear'],
           'conspire': ['fear'], 'constable': ['trust'], 'constancy': [ 'trust'],
           'constant': [ 'trust'], 'constantly': ['trust'],
           'consternation': ['anger', 'fear'], 'constipation': ['disgust'],
           'constitute': ['trust'], 'constitutional': [ 'trust'], 'constrain': ['fear'],
           'constraint': ['anger', 'fear',  'sadness'],
           'consul': ['trust'], 'consult': ['trust'],
           'contagion': ['anticipation', 'disgust', 'fear'],
           'contagious': ['disgust', 'fear'], 'contaminate': ['disgust'],
           'contaminated': ['disgust', 'fear',  'sadness'], 'contamination': ['disgust'],
           'contempt': ['anger', 'disgust', 'fear'],
           'contemptible': ['anger', 'disgust'], 'contemptuous': ['anger'],
           'content': ['joy',  'trust'], 'contentious': ['anger', 'disgust', 'fear'],
           'contingent': ['anticipation'], 'continuation': ['anticipation'],
           'continue': ['anticipation',  'trust'],
           'contraband': ['anger', 'disgust', 'fear'],
           'contradict': ['anger'],

           'contributor': [ 'trust'],
           'controversial': ['anger'],
           'convent': [ 'trust'],
           'convergence': ['anticipation'],
           'conveyancing': ['trust'],
           'convict': ['anger', 'disgust', 'fear',  'sadness'],
           'convince': ['anticipation',  'trust'], 'convinced': ['trust'], 'convincing': ['trust'],
           'coop': ['anger', 'disgust'],
           'cooperating': [ 'trust'], 'cooperation': [ 'trust'],
           'cooperative': [ 'trust'], 'cop': ['fear', 'trust'],
           'copycat': ['anger', 'disgust'],
           'coronation': ['joy',  'trust'],
           'corporation': [ 'trust'],
           'corpse': ['disgust',  'sadness'],
           'correctness': ['trust'], 'correspondence': ['anticipation'],
           'corroborate': [ 'trust'], 'corroboration': ['trust'],
           'corrosive': ['fear'],
           'corrupting': ['anger', 'disgust', 'fear',  'sadness'], 'corruption': ['disgust'],
           'corse': ['sadness'], 'cosmopolitan': [ 'trust'],  'couch': ['sadness'],
           'cough': ['disgust'], 'council': ['anticipation',  'trust'],
           'counsel': [ 'trust'], 'counsellor': ['anger', 'fear',  'trust'],
           'counselor': [ 'trust'], 'count': [ 'trust'], 'countdown': ['anticipation'],
           'countryman': ['trust'], 'county': ['trust'], 'coup': ['anger', 'surprise'],
           'courageous': ['fear'], 'courier': ['trust'],
           'court': ['anger', 'anticipation', 'fear'],
           'courtship': ['anticipation', 'joy',  'trust'],
           'cove': ['anticipation', 'disgust', 'fear', 'joy'], 'covenant': [ 'trust'],
           'cover': ['trust'],  'coward': ['disgust', 'fear',  'sadness'],
           'cowardice': ['fear'], 'cowardly': ['fear'], 'coy': ['fear'], 'coyote': ['fear'],
           'crabby': ['anger'],  'cracked': ['anger', 'fear'],
           'cradle': ['anticipation', 'joy',  'trust'],
           'cramp': ['anticipation'],
           'cranky': ['anger'], 'crap': ['disgust'],
           'craps': ['anticipation'], 'crash': ['fear',  'sadness', 'surprise'],
           'crave': ['anticipation'], 'craving': ['anticipation'], 'crawl': ['disgust'],
           'crazed': ['anger', 'fear'], 'crazy': ['anger', 'fear',  'sadness'],
           'cream': ['anticipation', 'joy',  'surprise'],
           'create': ['joy'],  'creature': ['disgust', 'fear'],
           'credence': [ 'trust'], 'credential': [ 'trust'],
           'credibility': [ 'trust'], 'credible': [ 'trust'], 'credit': [ 'trust'],
           'creditable': [ 'trust'],
           'creeping': ['anticipation'], 'cremation': ['sadness'],
           'crescendo': ['anticipation', 'joy',  'surprise', 'trust'], 'crew': ['trust'],
           'crime': ['anger'], 'criminal': ['anger', 'disgust', 'fear'],
           'criminality': ['anger', 'disgust', 'fear'],
           'cringe': ['disgust', 'fear',  'sadness'], 'cripple': ['fear',  'sadness'],
           'crippled': [ 'sadness'],  'crisp': [ 'trust'],
           'criticism': ['anger',  'sadness'],
           'criticize': ['anger', 'disgust', 'fear',  'sadness'],
           'critter': ['disgust'], 'crocodile': ['fear'],
           'cross': ['anger', 'fear',  'sadness'], 'crouch': ['fear'], 'crouching': ['fear'],
           'crowning': ['anticipation', 'joy',  'surprise', 'trust'], 'crucial': [ 'trust'],
           'crucifixion': ['anger', 'disgust', 'fear',  'sadness'],
           'crude': ['disgust'], 'cruel': ['anger', 'disgust', 'fear',  'sadness'],
           'cruelly': ['anger', 'fear'], 'cruelty': ['anger', 'disgust', 'fear',  'sadness'],
           'crumbling': [ 'sadness'], 'crunch': ['anger'],
           'crusade': ['anger', 'fear'], 'crushed': ['anger', 'disgust', 'fear',  'sadness'],
           'crushing': ['anger', 'disgust', 'fear'], 'crusty': ['disgust'],
           'cry': [ 'sadness'], 'crying': [ 'sadness'],
           'crypt': ['fear',  'sadness'],  'cube': ['trust'],
           'cuckold': ['disgust'],  'cuddle': ['joy',  'trust'],
           'cue': ['anticipation'], 'culinary': [ 'trust'],

           'cult': ['fear'],
           'cultivate': ['anticipation',  'trust'],
           'cumbersome': [ 'sadness'],
           'cupping': ['disgust', 'fear',  'sadness'],
           'cur': ['anger', 'disgust', 'fear'], 'curable': [ 'trust'],
           'curiosity': ['anticipation',  'surprise'],
           'curse': ['anger', 'disgust', 'fear',  'sadness'],
           'cursed': ['anger', 'fear',  'sadness'], 'cursing': ['anger', 'disgust'],
           'cussed': ['anger'], 'custodian': ['trust'],
           'custody': ['trust'],   'cutter': ['fear'],
           'cutthroat': ['anger', 'fear'],
           'cutting': ['anger', 'disgust', 'fear',  'sadness'], 'cyanide': ['fear'],
           'cyclone': ['fear',  'surprise'], 'cyst': ['fear',  'sadness'],
           'cystic': ['disgust'], 'cytomegalovirus': [ 'sadness'],
           'dabbling': ['anger', 'disgust'],
           'daemon': ['anger', 'disgust', 'fear',  'sadness', 'surprise'],
           'daft': ['disgust'], 'dagger': ['fear'], 'daily': ['anticipation'],
           'damage': ['anger', 'disgust',  'sadness'], 'damages': [ 'sadness'],
           'dame': ['anger', 'disgust',  'trust'], 'damn': ['anger', 'disgust'],
           'damnation': ['anger', 'fear',  'sadness'],
           'dance': ['joy',  'trust'],  'dandy': ['disgust'],
           'danger': ['fear',  'sadness'], 'dangerous': ['fear'], 'dank': ['disgust'],
           'dare': ['anticipation', 'trust'],  'dark': ['sadness'],
           'darken': ['fear',  'sadness'], 'darkened': ['fear',  'sadness'],
           'darkness': ['anger', 'fear',  'sadness'], 'darling': ['joy',  'trust'],
           'dart': ['fear'], 'dashed': ['anger', 'fear',  'sadness'],
           'dastardly': ['anger', 'disgust', 'fear'], 'daughter': ['joy'],
           'dawn': ['anticipation', 'joy',  'surprise', 'trust'],
           'deacon': ['trust'],
           'deadly': ['anger', 'disgust', 'fear',  'sadness'],
           'deal': ['anticipation', 'joy',  'surprise', 'trust'], 'dealings': ['trust'],

           'death': ['anger', 'anticipation', 'disgust', 'fear',  'sadness', 'surprise'],
           'debacle': ['fear',  'sadness'],
           'debauchery': ['disgust', 'fear'], 'debenture': ['anticipation'],
           'debris': ['disgust'], 'debt': [ 'sadness'],
           'decay': ['fear',  'sadness'], 'decayed': ['disgust',  'sadness'],
           'deceased': [ 'sadness'],
           'deceit': ['anger', 'disgust', 'fear',  'sadness', 'surprise'],
           'deceitful': ['disgust',  'sadness'], 'deceive': ['anger', 'disgust',  'sadness'],
           'deceived': ['anger'], 'deceiving': [ 'trust'],


           'decompose': ['disgust'], 'decomposed': ['sadness'],
           'decomposition': ['disgust', 'fear',  'sadness'], 'decoy': ['surprise'],

           'decry': ['anger'],   'deed': ['trust'],
           'defamation': ['disgust', 'fear'], 'defamatory': ['anger'],
           'default': ['disgust', 'fear',  'sadness'],
           'defeated': [ 'sadness'], 'defect': ['anger'], 'defection': ['fear'],
           'defective': ['disgust'], 'defend': ['fear'],
           'defendant': ['anger', 'fear', 'sadness'], 'defended': [ 'trust'],
           'defender': [ 'trust'],
           'defense': ['anger', 'anticipation', 'fear'], 'defenseless': ['fear',  'sadness'],
           'deference': [ 'trust'],
           'defiance': ['anger', 'disgust', 'fear'], 'defiant': ['anger'],
           'definitive': [ 'trust'],
           'deflate': ['anger',  'sadness'], 'deflation': ['fear'],
           'deform': ['disgust'], 'deformed': ['disgust',  'sadness'],
           'deformity': ['disgust', 'fear',  'sadness'], 'defraud': ['anger', 'disgust'],
           'defunct': [ 'sadness'], 'defy': ['anger', 'fear',  'sadness', 'surprise'],
           'degeneracy': ['anger', 'disgust',  'sadness'],
           'degrade': ['disgust'],
           'degrading': ['disgust', 'fear',  'sadness'],
           'delay': ['anger', 'disgust', 'fear',  'sadness'],
           'delegate': [ 'trust'],
           'deleterious': ['anger', 'disgust', 'fear'],
           'delicious': ['joy'],
           'delight': ['anticipation', 'joy'],
           'delighted': ['anticipation', 'joy',  'surprise'],
           'delightful': ['anticipation', 'joy',  'trust'],
           'delinquent': ['anger', 'disgust'], 'delirious': [ 'sadness'],
           'delirium': ['disgust',  'sadness'],
           'deliverance': ['anticipation', 'joy',  'trust'], 'delivery': ['anticipation'],
           'deluge': ['fear',  'sadness', 'surprise'],
           'delusion': ['anger', 'fear',  'sadness'], 'delusional': ['anger', 'fear'],
           'demand': ['anger'],  'demented': ['fear'],
           'dementia': ['fear',  'sadness'], 'demise': ['fear',  'sadness'],
           'demolish': ['anger',  'sadness'],
           'demon': ['anger', 'disgust', 'fear',  'sadness'],
           'demonic': ['anger', 'disgust', 'fear',  'sadness'],
           'demonstrative': ['joy',  'sadness'], 'demoralized': ['fear',  'sadness'],
           'denied': [ 'sadness'], 'denounce': ['anger', 'disgust'],
           'dentistry': ['fear'], 'denunciation': ['anger', 'disgust', 'fear'],
           'deny': ['anger'], 'denying': ['anticipation'],
           'depart': ['anticipation', 'sadness'], 'departed': [ 'sadness'],
           'departure': [ 'sadness'], 'depend': ['anticipation', 'trust'],
           'dependence': ['fear',  'sadness'],
           'dependent': [  'trust'],
           'deplorable': ['anger', 'disgust', 'fear',  'sadness'],
           'deplore': ['anger', 'disgust',  'sadness'], 'deport': ['fear',  'sadness'],
           'deportation': ['anger', 'fear',  'sadness'], 'depository': ['trust'],
           'depraved': ['anger', 'anticipation', 'disgust', 'fear',  'sadness'],
           'depravity': ['anger', 'disgust'], 'depreciate': ['anger', 'disgust'],
           'depreciated': ['anger', 'disgust', 'fear',  'sadness'], 'depreciation': ['fear'],
           'depress': ['fear',  'sadness'], 'depressed': ['anger', 'fear',  'sadness'],
           'depressing': ['disgust',  'sadness'], 'depression': [ 'sadness'],
           'depressive': [ 'sadness'],
           'deprivation': ['anger', 'disgust', 'fear',  'sadness'], 'depth': [ 'trust'],
           'deputy': ['trust'], 'deranged': ['anger', 'disgust', 'fear'],
           'derision': ['anger', 'disgust'], 'dermatologist': ['trust'],
           'derogation': ['anger', 'disgust', 'fear',  'sadness'],
           'derogatory': ['anger', 'disgust', 'fear',  'sadness'], 'descent': ['fear', 'sadness'],
           'desecration': ['anger', 'disgust', 'fear',  'sadness'],
           'desert': ['anger', 'disgust', 'fear',  'sadness'],
           'deserted': ['anger', 'disgust', 'fear',  'sadness'],
           'deserve': ['anger', 'anticipation',  'trust'],
           'designation': ['trust'],
           'desist': ['anger', 'disgust'],
           'desolation': ['fear',  'sadness'],
           'despair': ['anger', 'disgust', 'fear',  'sadness'],
           'despairing': ['fear',  'sadness'],
           'despicable': ['anger', 'disgust'], 'despise': ['anger', 'disgust'],
           'despotic': ['fear'], 'despotism': ['anger', 'disgust', 'fear',  'sadness'],
           'destination': ['anticipation', 'fear', 'joy',  'sadness', 'surprise'],
           'destined': ['anticipation'], 'destitute': ['fear',  'sadness'],
           'destroyed': ['anger', 'fear',  'sadness'], 'destroyer': ['anger', 'fear'],
           'destroying': ['anger', 'fear',  'sadness'], 'destruction': ['anger'],
           'destructive': ['anger', 'disgust', 'fear'],
           'detainee': ['anger', 'anticipation', 'fear',  'sadness'],
           'detention': [ 'sadness'],
           'deteriorate': ['fear',  'sadness'], 'deteriorated': ['disgust',  'sadness'],
           'deterioration': ['anger', 'disgust', 'fear',  'sadness'],
           'determinate': ['anticipation', 'trust'], 'determination': [ 'trust'],
           'detest': ['anger', 'disgust'],
           'detonate': ['fear',  'surprise'], 'detonation': ['anger'], 'detract': ['anger'],

           'devastate': ['anger', 'fear',  'sadness'],
           'devastating': ['anger', 'disgust', 'fear',  'sadness', 'trust'],
           'devastation': ['anger', 'fear',  'sadness', 'surprise'],
           'develop': ['anticipation'], 'deviation': ['sadness'],
           'devil': ['anger', 'anticipation', 'disgust', 'fear',  'sadness'],
           'devilish': ['disgust', 'fear'],
           'devotional': [ 'trust'],
           'devout': ['anticipation', 'joy',  'trust'],
           'diabolical': ['anger', 'disgust', 'fear'],
           'diagnosis': ['anticipation', 'fear',  'trust'], 'diamond': ['joy'],
           'diaper': ['disgust'], 'diarrhoea': ['disgust'], 'diary': ['joy',  'trust'],
           'diatribe': ['anger', 'disgust'], 'dictator': ['fear'],
           'dictatorial': ['anger'],
           'dictatorship': ['anger', 'anticipation', 'disgust', 'fear',  'sadness'],
           'dictionary': [ 'trust'], 'dictum': ['trust'],
           'die': ['fear',  'sadness'], 'dietary': ['anticipation'],
           'differential': ['trust'], 'differently': ['surprise'], 'difficult': ['fear'],
           'difficulties': [ 'sadness'], 'difficulty': ['anger', 'fear',  'sadness'],
           'digit': ['trust'],  'dignity': [ 'trust'],
           'digress': ['anticipation'], 'dike': ['fear'],
           'dilapidated': ['disgust',  'sadness'], 'diligence': [ 'trust'],
           'diminish': [ 'sadness'],
           'dinosaur': ['fear'],
           'diplomacy': ['anticipation',  'trust'], 'diplomatic': [ 'trust'],
           'dire': ['disgust', 'fear',  'sadness', 'surprise'], 'director': [ 'trust'],
           'dirt': ['disgust'], 'dirty': ['disgust'], 'disability': [ 'sadness'],
           'disable': ['fear',  'sadness'], 'disabled': ['fear',  'sadness'],
           'disagree': ['anger'],
           'disagreeing': ['anger',  'sadness'], 'disagreement': ['anger',  'sadness'],
           'disallowed': ['anger', 'disgust', 'fear',  'sadness'], 'disappear': ['fear'],
           'disappoint': ['anger', 'disgust',  'sadness'],
           'disappointed': ['anger', 'disgust',  'sadness'], 'disappointing': [ 'sadness'],
           'disappointment': ['disgust',  'sadness'], 'disapproval': [ 'sadness'],
           'disapprove': ['anger', 'disgust', 'fear',  'sadness'],
           'disapproved': ['anger',  'sadness'],
           'disapproving': ['anger', 'disgust',  'sadness'],
           'disaster': ['anger', 'disgust', 'fear',  'sadness', 'surprise'],
           'disastrous': ['anger', 'fear',  'sadness'],
           'disciple': ['trust'],
           'discipline': ['fear'], 'disclaim': ['anger', 'disgust',  'trust'],
           'disclosed': ['trust'], 'discoloration': ['disgust'],
           'discolored': ['disgust',  'sadness'], 'discomfort': [ 'sadness'],
           'disconnect': [ 'sadness'], 'disconnected': [ 'sadness'],
           'discontent': ['anger', 'disgust', 'fear',  'sadness'],
           'discontinuity': ['disgust', 'fear',  'sadness'],
           'discord': ['anger', 'disgust'], 'discourage': ['fear',  'sadness'],

           'discreet': ['anticipation'], 'discretion': ['anticipation',  'trust'],
           'discriminate': ['anger',  'sadness'],
           'discriminating': ['disgust'],
           'discrimination': ['anger', 'disgust', 'fear',  'sadness'],
           'disdain': ['anger', 'disgust'],
           'disease': ['anger', 'disgust', 'fear',  'sadness'],
           'diseased': ['disgust', 'fear',  'sadness'], 'disembodied': ['fear',  'sadness'],
           'disfigured': ['anger', 'disgust', 'fear',  'sadness'],
           'disgrace': ['anger', 'disgust',  'sadness'],
           'disgraced': ['anger', 'disgust',  'sadness'],
           'disgraceful': ['anger', 'disgust'],
           'disgruntled': ['anger', 'disgust',  'sadness'],
           'disgust': ['anger', 'disgust', 'fear',  'sadness'],
           'disgusting': ['anger', 'disgust', 'fear'], 'disheartened': [ 'sadness'],
           'disheartening': [ 'sadness'], 'dishonest': ['anger', 'disgust',  'sadness'],
           'dishonesty': ['disgust'], 'dishonor': ['anger', 'disgust', 'fear',  'sadness'],
           'disillusionment': ['anger', 'disgust',  'sadness'],
           'disinformation': ['anger', 'fear'], 'disingenuous': ['disgust'],
           'disintegrate': ['disgust', 'fear'],
           'dislike': ['anger', 'disgust'],
           'disliked': ['anger',  'sadness'],
           'dislocated': ['anger', 'disgust', 'fear',  'sadness'],
           'dismal': ['disgust', 'fear',  'sadness'],
           'dismay': ['anger', 'anticipation', 'fear',  'sadness', 'surprise'],
           'dismemberment': ['disgust', 'fear',  'sadness'],
           'dismissal': ['anger', 'disgust', 'fear',  'sadness', 'surprise'],
           'disobedience': ['anger', 'disgust'], 'disobedient': ['anger'],
           'disobey': ['anger', 'disgust'], 'disorder': ['fear'],

           'disparage': ['anger', 'disgust',  'sadness'],
           'disparaging': ['anger', 'disgust',  'sadness'],
           'disparity': ['anger', 'disgust',  'sadness'], 'dispassionate': [ 'sadness'],
           'dispel': [ 'sadness'],
           'displaced': ['anger', 'fear', 'sadness'],
           'displeased': ['anger', 'disgust', 'fear',  'sadness'],
           'displeasure': ['disgust'],  'dispose': ['disgust'],
           'disposed': ['anticipation',  'trust'],
           'dispossessed': ['anger', 'fear',  'sadness'], 'dispute': ['anger'],
           'disqualified': ['anger', 'disgust',  'sadness'],
           'disqualify': [ 'sadness'],  'disregarded': ['disgust'],
           'disreputable': ['anger', 'disgust', 'fear'], 'disrespect': ['anger'],
           'disrespectful': ['anger', 'disgust', 'fear',  'sadness'],
           'disruption': ['anger', 'fear',  'surprise'],
           'dissection': ['disgust'],  'dissension': ['anger'],
           'disservice': ['anger', 'disgust',  'sadness'],
           'dissident': ['anger', 'fear'],
           'dissolution': ['anger', 'fear',  'sadness', 'surprise'], 'dissonance': ['anger'],
           'distaste': ['disgust'], 'distasteful': ['disgust'],
           'distorted': ['disgust'],
           'distracted': ['anger'],
           'distracting': ['anger', 'anticipation'],
           'distraught': [ 'sadness'],
           'distress': ['anger', 'disgust', 'fear',  'sadness', 'surprise'],
           'distressed': ['fear'], 'distressing': ['anger', 'fear'],
           'distrust': ['anger', 'disgust', 'fear'],
           'disturbance': ['anger', 'fear',  'sadness', 'surprise'],
           'disturbed': ['anger',  'sadness'],  'disused': ['anger'],
           'ditty': ['joy'], 'divan': ['trust'], 'divergent': [ 'surprise'],
           'diversion': [ 'surprise'],
           'divination': ['anticipation'],

           'divorce': ['anger', 'disgust', 'fear',  'sadness', 'surprise', 'trust'],

           'doctor': [ 'trust'], 'doctrine': ['trust'],
           'dogma': ['trust'],  'doldrums': [ 'sadness'],
           'dole': [ 'sadness'], 'doll': ['joy'], 'dolor': [ 'sadness'],
           'dolphin': ['joy',  'surprise', 'trust'], 'dominant': ['fear'],
           'dominate': ['anger', 'fear'],
           'domination': ['anger', 'fear',  'sadness'], 'dominion': ['fear', 'trust'],
           'don': [ 'trust'],  'donkey': ['disgust'],
           'doom': ['fear'], 'doomed': ['fear',  'sadness'],
           'doomsday': ['anger', 'anticipation', 'disgust', 'fear',  'sadness'],
           'doubt': ['fear',  'sadness', 'trust'],
           'doubtless': [ 'trust'],
           'dove': ['anticipation', 'joy',  'trust'], 'downfall': ['fear',  'sadness'],
           'downright': ['trust'],  'drab': [ 'sadness'],
           'draft': ['anticipation'], 'dragon': ['fear'],
           'dread': ['anticipation', 'fear'],
           'dreadful': ['anger', 'anticipation', 'disgust', 'fear',  'sadness'],
           'dreadfully': ['disgust', 'fear',  'sadness', 'surprise'], 'dreary': [ 'sadness'],
           'drivel': ['disgust'],  'drool': ['disgust'],
           'drown': ['fear',  'sadness'],
           'drugged': ['sadness'],
           'drunken': ['disgust'],
           'dubious': ['fear',  'trust'], 'duel': ['anger', 'anticipation', 'fear'],
           'dull': [ 'sadness'],
           'dumps': ['anger',  'sadness'],
           'dung': ['disgust'], 'dungeon': ['fear'], 'dupe': ['anger'],
           'duplicity': ['anger'], 'durability': [ 'trust'],
           'durable': [ 'trust'], 'duress': ['anger', 'disgust', 'fear',  'sadness'],
           'dutiful': ['anticipation',  'trust'],
           'dwarfed': ['fear',  'sadness'], 'dying': ['anger', 'disgust', 'fear',  'sadness'],
           'dynamic': ['surprise'], 'dysentery': ['disgust',  'sadness'],
           'eager': ['anticipation', 'joy',  'surprise', 'trust'],
           'eagerness': ['anticipation', 'joy',  'trust'], 'eagle': ['trust'],

           'earthquake': ['anger', 'fear',  'sadness', 'surprise'],

           'economy': ['trust'], 'ecstasy': ['anticipation', 'joy'],
           'ecstatic': ['anticipation', 'joy',  'surprise'], 'edict': ['fear'],
           'edification': ['anticipation', 'joy',  'trust'], 'edition': ['anticipation'],
           'educational': [ 'trust'], 'eel': ['fear'],
           'effective': [ 'trust'],
           'efficient': ['anticipation',  'trust'], 'effigy': ['anger'],
           'egotistical': ['disgust'],
           'egregious': ['anger', 'disgust'],
           'ejaculation': ['anticipation', 'joy',  'surprise', 'trust'],
           'elated': ['joy'], 'elbow': ['anger'],
           'elder': [ 'trust'], 'elders': [ 'trust'], 'elect': [ 'trust'],
           'electorate': ['trust'], 'electric': ['joy',  'surprise'],
           'elegance': ['anticipation', 'joy',  'trust'], 'elegant': ['joy'],
           'elevation': ['anticipation', 'fear', 'joy',  'trust'], 'elf': ['anger', 'disgust', 'fear'],
           'elimination': ['anger', 'disgust', 'fear',  'sadness'],
           'elite': ['anticipation', 'joy',  'trust'],
           'elucidate': [ 'trust'], 'elusive': [ 'surprise'],
           'emaciated': ['fear',  'sadness'], 'emancipation': ['anticipation', 'joy'],
           'embarrass': [ 'sadness'],
           'embarrassment': ['fear',  'sadness', 'surprise'],
           'embolism': ['fear',  'sadness'],
           'embrace': ['anticipation', 'joy',  'surprise', 'trust'],
           'emergency': ['fear',  'sadness', 'surprise'],
           'eminence': [ 'trust'],
           'emphasize': ['trust'], 'employ': ['trust'],
           'emptiness': ['sadness'],
           'enable': [ 'trust'], 'enablement': [ 'trust'],
           'enchant': ['anticipation', 'joy',  'surprise'], 'enchanted': ['joy',  'trust'],
           'enchanting': ['anticipation', 'joy'],
           'encourage': ['joy',  'trust'],
           'encroachment': ['fear'], 'encumbrance': ['anger', 'fear',  'sadness'],
           'encyclopedia': [ 'trust'], 'endanger': ['anticipation', 'fear'],
           'endangered': ['fear'], 'endeavor': ['anticipation'],
           'endemic': ['disgust', 'fear',  'sadness'],
           'endless': ['anger', 'fear', 'joy',   'sadness', 'trust'],
           'endocarditis': ['fear', 'sadness'], 'endow': [ 'trust'],
           'endowment': [ 'trust'],
           'enema': ['disgust'], 'enemy': ['anger', 'disgust', 'fear'],
           'enforce': ['anger', 'fear'],
           'engaged': ['anticipation', 'joy',  'trust'], 'engaging': ['joy',  'trust'],
           'engulf': ['anticipation'],  'enigmatic': ['fear'],
           'enjoy': ['anticipation', 'joy',  'trust'],
           'enjoying': ['anticipation', 'joy',  'trust'], 'enlighten': ['joy',  'trust'],
           'enlightenment': ['joy',  'trust'], 'enliven': ['joy',  'surprise', 'trust'],
           'enmity': ['anger', 'fear',  'sadness'],
           'enroll': ['anticipation', 'trust'], 'ensemble': [ 'trust'],
           'enslaved': ['anger', 'disgust', 'fear',  'sadness'],
           'entangled': ['anger', 'disgust', 'fear',  'sadness'],
           'entertain': ['joy'],
           'entertained': ['joy'], 'entertaining': ['anticipation', 'joy'],
           'entertainment': ['anticipation', 'joy',  'surprise', 'trust'],
           'enthusiasm': ['anticipation', 'joy',  'surprise'],
           'enthusiast': ['anticipation', 'joy',  'surprise'], 'entrails': ['disgust'],
           'entrust': ['trust'],

           'epidemic': ['anger', 'anticipation', 'disgust', 'fear',  'sadness', 'surprise'],
           'episcopal': ['trust'], 'epitaph': ['sadness'],
           'equality': ['joy',  'trust'],
           'eradicate': ['anger'],
           'eradication': ['anger', 'disgust', 'fear'], 'erase': ['fear'],
           'erotic': ['anticipation', 'joy',   'surprise', 'trust'],
           'errand': ['anticipation',  'trust'],
           'erratic': [ 'surprise'],
           'error': [ 'sadness'],  'erupt': ['anger',  'surprise'],
           'eruption': ['anger', 'fear',  'surprise'], 'escalate': ['anger'],
           'escape': ['anticipation', 'fear'], 'escaped': ['fear'],
           'eschew': ['anger',  'sadness'], 'escort': ['trust'],
           'establish': ['trust'],
           'established': ['joy'], 'esteem': ['joy',  'sadness', 'trust'],
           'ethereal': ['fear'],
           'euthanasia': ['fear',  'sadness'], 'evacuate': ['fear'],
           'evade': ['anger', 'disgust', 'fear'],
           'evanescence': ['sadness', 'surprise'], 'evasion': ['fear',  'sadness'],
           'eventual': ['anticipation'], 'eventuality': ['anticipation', 'fear'],
           'evergreen': ['joy',  'trust'],  'evict': [ 'sadness'],
           'eviction': ['anger', 'disgust', 'fear',  'sadness'], 'evident': [ 'trust'],
           'evil': ['anger', 'disgust', 'fear',  'sadness'],
           'exacerbation': ['anger', 'fear'],
           'exaggerate': ['anger'],
           'exalt': ['anticipation', 'joy',  'trust'], 'exaltation': ['joy',  'trust'],
           'exalted': ['joy',  'trust'], 'examination': ['fear',  'surprise'],
           'exasperation': ['anger', 'disgust'], 'excavation': ['anticipation',  'surprise'],
           'exceed': ['anticipation', 'joy'],
           'excel': ['anticipation', 'joy',  'surprise', 'trust'],
           'excellence': ['disgust', 'joy',  'trust'], 'excellent': ['joy',  'trust'],
           'exchange': [ 'trust'],

           'excitation': ['anger', 'anticipation', 'fear', 'joy',  'surprise'],
           'excite': ['anger', 'anticipation', 'fear', 'joy',  'surprise'],
           'excited': ['anticipation', 'joy',  'surprise', 'trust'],
           'excitement': ['anticipation', 'joy',  'surprise'],
           'exciting': ['anticipation', 'joy',  'surprise'], 'exclaim': ['surprise'],
           'excluded': ['disgust',  'sadness'], 'excluding': [ 'sadness'],
           'exclusion': ['disgust', 'fear',  'sadness'], 'excrement': ['disgust'],
           'excretion': ['disgust'], 'excruciating': ['fear'],
           'execution': ['anger', 'fear',  'sadness', 'trust'],
           'executioner': ['anger', 'fear',  'sadness'], 'executor': ['trust'],

           'exhausted': [ 'sadness'], 'exhaustion': ['anticipation',  'sadness'],
           'exhaustive': ['trust'], 'exhilaration': ['joy',  'surprise'],
           'exigent': ['anticipation', 'disgust', 'fear',  'surprise'],
           'exile': ['anger', 'fear',  'sadness'],
           'exorcism': ['fear',  'sadness'],
           'expect': ['anticipation',  'surprise', 'trust'], 'expectancy': ['anticipation'],
           'expectant': ['anticipation'], 'expectation': ['anticipation'], 'expected': ['anticipation'],
           'expecting': ['anticipation'], 'expedient': ['joy',  'trust'],
           'expedition': ['anticipation'], 'expel': ['anger', 'disgust', 'fear',  'sadness'],
           'experienced': [ 'trust'],
           'experiment': ['anticipation', 'surprise'], 'expert': [ 'trust'],
           'expertise': [ 'trust'], 'expire': [ 'sadness'],
           'explain': [ 'trust'], 'expletive': ['anger'],
           'explode': ['anger', 'fear',  'sadness', 'surprise'], 'explore': ['anticipation'],
           'explosion': ['fear',  'surprise'],
           'explosive': ['anger', 'anticipation', 'fear',  'surprise'],
           'expose': ['anticipation', 'fear'],
           'expulsion': ['anger', 'disgust', 'fear',  'sadness'], 'exquisite': ['joy'],

           'exterminate': ['fear',  'sadness'], 'extermination': ['anger', 'fear'],
           'extinct': [ 'sadness'], 'extinguish': ['anger'],
           'extrajudicial': ['fear'],
           'extricate': ['anticipation'], 'exuberance': ['joy'], 'eyewitness': ['trust'],
           'fabrication': [ 'trust'],
           'fact': ['trust'], 'facts': [ 'trust'], 'faculty': [ 'trust'],
           'faeces': ['disgust'],
           'failing': ['anger', 'anticipation', 'fear',  'sadness'],
           'failure': ['disgust', 'fear',  'sadness'],
           'fain': ['anticipation', 'joy',  'trust'],
           'fainting': ['fear',  'sadness', 'surprise'],
           'fairly': [ 'trust'], 'faith': ['anticipation', 'joy',  'trust'],
           'faithful': [ 'trust'], 'faithless': [ 'sadness'],
           'fall': [ 'sadness'], 'fallacious': ['anger'],
           'falling': [ 'sadness'],
           'falsehood': ['anger',  'trust'],
           'falsification': ['anger'], 'falsify': ['disgust'],
           'falsity': ['disgust'], 'falter': ['fear'],
           'familiar': [ 'trust'], 'familiarity': ['anticipation', 'joy',  'trust'],
           'famine': [ 'sadness'],
           'fanaticism': ['fear'], 'fancy': ['anticipation', 'joy'],
           'fanfare': ['anticipation', 'joy',  'surprise'], 'fang': ['fear'], 'fangs': ['fear'],
           'farcical': ['disgust'], 'farm': ['anticipation'],

           'fasting': [ 'sadness'], 'fat': ['disgust',  'sadness'],
           'fatal': ['anger', 'fear',  'sadness'], 'fatality': ['fear',  'sadness'],
           'fate': ['anticipation'], 'father': ['trust'],
           'fatigued': [ 'sadness'], 'fatty': ['disgust',  'sadness'],
           'fault': [ 'sadness'], 'faultless': [ 'trust'],
           'favorable': ['anticipation', 'joy',  'surprise', 'trust'],
           'favorite': ['joy',  'trust'],
           'fear': ['anger', 'fear'], 'fearful': ['fear',  'sadness'],
           'fearfully': ['fear',  'sadness', 'surprise'], 'fearing': ['fear'],
           'fearless': ['fear'], 'feat': ['anticipation', 'joy',  'surprise'],
           'fecal': ['disgust'],
           'feces': ['disgust'], 'fee': ['anger'], 'feeble': [ 'sadness'],
           'feeling': ['anger', 'anticipation', 'disgust', 'fear', 'joy',   'sadness',
                       'surprise', 'trust'],  'felicity': ['joy'],
           'fell': [ 'sadness'], 'fellow': [ 'trust'],
           'felon': ['fear'],   'fenced': ['anger'],
           'fender': ['trust'], 'ferment': ['anticipation'], 'fermentation': ['anticipation'],
           'ferocious': ['anger', 'disgust', 'fear'], 'ferocity': ['anger'],
           'fervor': ['anger', 'joy'],
           'festival': ['anticipation', 'joy',  'surprise'], 'festive': ['joy'],
           'fete': ['anticipation', 'joy',  'surprise'], 'feud': ['anger'],
           'feudalism': ['anger',  'sadness'], 'fever': ['fear'],
           'fib': ['anger'],
           'fidelity': ['joy',  'trust'],
           'fiend': ['anger', 'disgust', 'fear'], 'fierce': ['anger', 'disgust', 'fear'],
           'fiesta': ['anticipation', 'joy',  'surprise', 'trust'],
           'fight': ['anger', 'fear'], 'fighting': ['anger'],
           'fill': ['trust'], 'filth': ['disgust'], 'filthy': ['disgust'],
           'finally': ['anticipation', 'disgust', 'joy',  'surprise', 'trust'],
           'fire': ['fear'], 'firearms': ['anger', 'fear'],
           'fireman': ['trust'],
           'firmness': [ 'trust'], 'firstborn': ['anticipation', 'joy',  'trust'],
           'fits': ['anger'], 'fitting': ['anticipation', 'joy',  'trust'],
           'fixed': ['trust'],  'flabby': ['disgust'],
           'flaccid': [ 'sadness'],
           'flagrant': ['anger', 'disgust'], 'flagship': ['trust'],
           'flange': ['trust'],  'flattering': ['joy'],
           'flatulence': ['disgust'],  'flaw': [ 'sadness'],
           'flea': ['disgust'], 'fled': ['fear'], 'flee': ['fear'],
           'fleece': ['anger', 'disgust',  'sadness'],  'flesh': ['disgust'],

           'flinch': ['anticipation', 'disgust', 'fear',  'sadness', 'surprise'],
           'flirt': ['anticipation', 'joy',   'surprise', 'trust'],
           'flog': ['anger', 'disgust', 'fear',  'sadness'], 'flood': ['fear'],
           'flop': ['disgust',  'sadness'],
           'flounder': ['fear',  'sadness'],
           'flu': ['fear'], 'fluctuation': ['anger', 'fear'], 'fluke': ['surprise'],
           'flying': ['fear'],
           'foe': ['anger', 'fear'],  'follower': ['trust'],
           'fondness': ['joy'], 'food': ['joy',  'trust'],
           'fool': ['disgust'],
           'football': ['anticipation', 'joy'], 'footing': ['trust'],
           'foray': ['anger'],  'forbid': [ 'sadness'],
           'forbidding': ['anger', 'fear'], 'force': ['anger', 'fear'],
           'forced': ['fear'], 'forcibly': ['anger', 'fear'],
           'forearm': ['anger', 'anticipation'], 'foreboding': ['anticipation', 'fear'],
           'forecast': ['anticipation', 'trust'], 'foreclose': ['fear',  'sadness'],
           'forefathers': ['joy',  'trust'],
           'foreigner': ['fear'],
           'foresee': ['anticipation',  'surprise', 'trust'],
           'foreseen': ['anticipation'], 'foresight': ['anticipation',  'trust'],
           'forewarned': ['anticipation', 'fear'], 'forfeit': ['anger',  'sadness'],
           'forfeiture': ['fear',  'sadness'],

           'forgiving': [ 'trust'], 'forgotten': ['fear',  'sadness'],
           'forlorn': [ 'sadness'], 'formality': ['trust'], 'formative': [ 'trust'],
           'formidable': ['fear'], 'forming': ['anticipation'],
           'formula': [ 'trust'],  'forsake': [ 'sadness'],
           'forsaken': ['anger',  'sadness'], 'fort': ['trust'],
           'fortitude': ['joy',  'trust'],
           'fortress': ['fear',  'sadness', 'trust'],
           'fortune': ['anticipation', 'joy',  'surprise', 'trust'],
           'foul': ['anger', 'disgust', 'fear'], 'found': ['joy',  'trust'],
           'fragile': ['fear',  'sadness'],
           'frailty': [ 'sadness'], 'framework': ['trust'],
           'frank': [ 'trust'], 'frankness': [ 'trust'],
           'frantic': ['anticipation', 'disgust', 'fear',  'surprise'],
           'fraternal': ['joy',  'trust'], 'fraud': ['anger'],
           'fraudulent': ['anger', 'disgust'], 'fraught': ['fear',  'sadness'],
           'frayed': [ 'sadness'],
           'freakish': ['disgust', 'fear',  'surprise'], 'freedom': ['joy',  'trust'],
           'freely': ['joy',  'trust'],
           'frenetic': ['anger', 'fear',  'surprise'], 'frenzied': ['anger', 'fear'],
           'fret': ['fear'], 'friction': ['anger'],
           'friend': ['joy',  'trust'], 'friendliness': ['joy',  'trust'],
           'friendly': ['anticipation', 'joy',  'trust'], 'friendship': ['joy',  'trust'],
           'frigate': ['fear'], 'fright': ['fear',  'surprise'],
           'frighten': ['fear',  'sadness', 'surprise'], 'frightened': ['fear',  'surprise'],
           'frightful': ['anger', 'fear',  'sadness'],
           'frisky': ['anticipation', 'joy',  'surprise'],
           'frolic': ['joy'],
           'frown': [ 'sadness'], 'frowning': ['anger', 'disgust',  'sadness'],
           'fruitless': [ 'sadness'],
           'frustrate': ['anger', 'disgust',  'sadness'], 'frustrated': ['anger'],
           'frustration': ['anger'],
           'fugitive': ['anger', 'disgust', 'fear',  'sadness', 'trust'], 'fulfill': ['joy'],
           'fulfillment': ['anticipation', 'joy',  'trust'],
           'fully': [ 'trust'],
           'fuming': ['anger'], 'fun': ['anticipation', 'joy'],
           'fundamental': [ 'trust'], 'funeral': ['sadness'], 'fungus': ['disgust'],
           'funk': [ 'sadness'], 'furious': ['anger', 'disgust'], 'furiously': ['anger'],
           'furnace': ['anger'], 'furor': ['anger'], 'furrow': ['sadness'],
           'fury': ['anger', 'fear',  'sadness'], 'fuse': [ 'trust'],
           'fuss': ['anger',  'sadness'], 'fussy': ['disgust'],
           'futile': [ 'sadness'],  'gaby': ['disgust'],
           'gag': ['disgust'], 'gage': ['trust'], 'gain': ['anticipation', 'joy'],
           'gall': ['anger', 'disgust'],
           'gallows': ['anger', 'fear',  'sadness'],
           'gambler': ['anticipation',  'surprise'],
           'gambling': ['anticipation',  'surprise'], 'gang': ['anger', 'fear'],
           'gape': ['surprise'], 'garbage': ['disgust'],
           'garden': ['joy'], 'garish': ['disgust',  'surprise'],
           'garrison': [ 'trust'], 'gash': ['fear'],
           'gasp': ['surprise'], 'gasping': ['fear'], 'gate': ['trust'], 'gateway': ['trust'],
           'gauging': ['trust'],  'gawk': ['surprise'],
           'gazette': [ 'trust'],  'gelatin': ['disgust'],
           'gem': ['joy'], 'general': [ 'trust'],
           'generosity': ['anticipation', 'joy',  'surprise', 'trust'],
           'generous': ['joy',  'trust'], 'genial': ['joy'],
           'gent': ['anger', 'disgust', 'fear'],
           'gentleman': [ 'trust'],  'gentry': [ 'trust'],
           'genuine': [ 'trust'],  'geriatric': [ 'sadness'],
           'germ': ['anticipation', 'disgust'], 'germination': ['anticipation'],
           'ghastly': ['disgust', 'fear'], 'ghetto': ['disgust', 'fear',  'sadness'],
           'ghost': ['fear'], 'ghostly': ['fear'], 'giant': ['fear'],
           'gibberish': ['anger'], 'gift': ['anticipation', 'joy',  'surprise'],
           'giggle': ['joy'],
           'girder': [ 'trust'],
           'glad': ['anticipation', 'joy'], 'gladiator': ['fear'], 'gladness': ['joy'],
           'glare': ['anger', 'fear'], 'glaring': ['anger'], 'glee': ['joy'],
           'glide': ['anticipation', 'joy'],
           'glimmer': ['anticipation', 'joy',  'surprise'], 'glitter': ['disgust', 'joy'],
           'gloom': [ 'sadness'], 'gloomy': [ 'sadness'],
           'glorification': ['joy'],
           'glorify': ['anticipation', 'joy',  'surprise', 'trust'],
           'glory': ['anticipation', 'joy',  'trust'],
           'glow': ['anticipation', 'joy',  'trust'],
           'glum': [ 'sadness'], 'glut': ['disgust'], 'gluttony': ['disgust'],
           'gnome': ['anger', 'disgust', 'fear'], 'gob': ['disgust'],
           'goblin': ['disgust', 'fear'], 'god': ['anticipation', 'fear', 'joy',  'trust'],
           'godless': ['anger'], 'godly': ['joy',  'trust'],
           'godsend': ['joy',  'surprise'],
           'gonorrhea': ['anger', 'disgust', 'fear',  'sadness'], 'goo': ['disgust'],
           'good': ['anticipation', 'joy',  'surprise', 'trust'],
           'goodness': ['anticipation', 'joy',  'surprise', 'trust'],
           'gore': ['anger', 'disgust', 'fear',  'sadness'],
           'gorge': ['disgust'], 'gorgeous': ['joy'],
           'gory': ['anger', 'disgust', 'fear',  'sadness'], 'gospel': [ 'trust'],
           'govern': [ 'trust'],
           'governess': ['trust'], 'government': ['fear'], 'governor': ['trust'],
           'grab': ['anger'],
           'gradual': ['anticipation'],
           'graduation': ['anticipation', 'fear', 'joy',  'surprise', 'trust'], 'grammar': ['trust'],
           'grandchildren': ['anticipation', 'joy',  'trust'],
           'grandfather': ['trust'],
           'grant': ['anticipation', 'joy',  'trust'],
           'grasping': ['anticipation'],  'grated': ['anger'],
           'gratify': ['joy',  'surprise'],
           'grating': ['anger', 'disgust'], 'gratitude': ['joy'], 'gratuitous': ['disgust'],
           'grave': ['fear',  'sadness'], 'gravitate': ['anticipation'], 'gray': ['disgust', 'sadness'],
           'greasy': ['disgust'],  'greatness': ['joy',  'surprise', 'trust'],
           'greed': ['anger', 'disgust'], 'greedy': ['disgust'],
           'green': ['joy',  'trust'], 'greeting': [ 'surprise'],
           'grenade': ['fear'], 'grief': [ 'sadness'],
           'grievance': ['anger', 'disgust',  'sadness'], 'grieve': ['fear',  'sadness'],
           'grievous': ['anger', 'fear',  'sadness'],
           'grim': ['anger', 'anticipation', 'disgust', 'fear',  'sadness'],
           'grime': ['disgust'], 'grimy': ['disgust'],
           'grin': ['anticipation', 'joy',  'surprise', 'trust'],
           'grisly': ['disgust', 'fear'],  'grit': [ 'trust'],
           'grizzly': ['fear'], 'groan': ['disgust',  'sadness'],
           'grope': ['anger', 'disgust', 'fear'], 'gross': ['disgust'],
           'grotesque': ['disgust'], 'ground': ['trust'], 'grounded': ['fear',  'sadness'],

           'grow': ['anticipation', 'joy',  'trust'], 'growl': ['anger', 'fear'],
           'growling': ['anger', 'disgust', 'fear'],
           'grudge': ['anger'],  'gruesome': ['disgust'],
           'gruff': ['anger', 'disgust'], 'grumble': ['anger', 'disgust'],
           'grumpy': ['anger', 'disgust',  'sadness'], 'guarantee': [ 'trust'],
           'guard': ['fear',  'trust'], 'guarded': ['trust'], 'guardian': [ 'trust'],
           'guardianship': [ 'trust'], 'gubernatorial': [ 'trust'],
           'guerilla': ['fear'], 'guess': ['surprise'], 'guidance': [ 'trust'],
           'guide': [ 'trust'], 'guidebook': [ 'trust'],
           'guillotine': ['anger', 'anticipation', 'disgust', 'fear',  'sadness'],
           'guilt': ['disgust',  'sadness'], 'guilty': ['anger',  'sadness'],
           'gullible': [ 'sadness', 'trust'],
           'gulp': ['fear', 'surprise'], 'gun': ['anger', 'fear'], 'gunpowder': ['fear'],
           'guru': [ 'trust'], 'gush': ['disgust', 'joy'], 'gusset': ['trust'],
           'gut': ['disgust'],  'gutter': ['disgust'],
           'habitual': ['anticipation'],
           'hag': ['disgust', 'fear'], 'haggard': [ 'sadness'],
           'hail': [  'trust'], 'hairy': ['disgust'],
           'hallucination': ['fear'],
           'halter': ['anger', 'fear',  'sadness'], 'halting': ['fear',  'sadness'],
           'hamstring': ['anger',  'sadness'], 'handbook': ['trust'],
           'handicap': [ 'sadness'],
           'hanging': ['anger', 'disgust', 'fear',  'sadness'], 'hangman': ['fear'],
           'hankering': ['anticipation'], 'hap': ['anticipation', 'surprise'], 'happen': ['anticipation'],
           'happily': ['joy'], 'happiness': ['anticipation', 'joy'],
           'happy': ['anticipation', 'joy',  'trust'], 'harass': ['anger', 'disgust'],
           'harassing': ['anger'], 'harbinger': ['anger', 'anticipation', 'fear'], 'harbor': ['trust'],
           'hardened': ['anger', 'disgust', 'fear'],
           'hardship': [ 'sadness'], 'hardy': ['joy',  'trust'],
           'harlot': ['disgust'], 'harm': ['fear'],
           'harmful': ['anger', 'disgust', 'fear',  'sadness'],
           'harmoniously': ['joy',  'trust'], 'harmony': ['joy',  'trust'],
           'harrowing': ['fear'], 'harry': ['anger',  'sadness'],
           'harshness': ['anger', 'fear'], 'harvest': ['anticipation', 'joy'],
           'haste': ['anticipation'],
           'hate': ['anger', 'disgust', 'fear',  'sadness'],
           'hateful': ['anger', 'disgust', 'fear',  'sadness'], 'hating': ['anger'],
           'hatred': ['anger', 'disgust', 'fear',  'sadness'], 'haughty': ['anger'],
           'haunt': ['fear'], 'haunted': ['fear',  'sadness'],
           'haven': [ 'trust'], 'havoc': ['anger', 'fear'], 'hawk': ['fear'],
           'hazard': ['fear'], 'hazardous': ['fear'], 'haze': ['fear'],
           'headlight': ['anticipation'],
           'heal': ['joy',  'trust'], 'healing': ['anticipation', 'joy',  'trust'],
           'healthful': ['joy'],  'hearing': ['fear'],
           'hearse': ['fear',  'sadness'], 'heartache': [ 'sadness'],
           'heartfelt': ['joy',  'sadness', 'trust'],
           'heartily': ['joy'], 'heartless': [ 'sadness'], 'heartworm': ['disgust'],
           'heathen': ['fear'], 'heavenly': ['anticipation', 'joy',  'trust'],
           'heavens': ['joy',  'trust'],
           'hedonism': ['joy'],
           'heft': ['anticipation', 'fear'], 'heighten': ['fear'],
           'hell': ['anger', 'disgust', 'fear',  'sadness'],
           'hellish': ['anger', 'disgust', 'fear',  'sadness'], 'helmet': ['fear'],
           'helper': [ 'trust'], 'helpful': ['joy',  'trust'],
           'helpless': ['fear',  'sadness'], 'helplessness': ['fear',  'sadness'],
           'hemorrhage': ['disgust', 'fear',  'sadness'],
           'heretic': ['disgust'],
           'heritage': ['trust'], 'hermaphrodite': [ 'surprise'], 'hermit': ['sadness', 'trust'],
           'hero': ['anticipation', 'joy',  'surprise', 'trust'],
           'heroic': ['joy',  'surprise', 'trust'],
           'heroine': [ 'trust'], 'heroism': ['anticipation', 'joy',  'surprise', 'trust'],
           'herpes': ['disgust'], 'herpesvirus': ['disgust'],
           'hesitation': ['fear'], 'heyday': ['anticipation', 'joy',  'trust'],
           'hide': ['fear'], 'hideous': ['disgust', 'fear',  'sadness'],
           'hiding': ['fear'], 'highest': ['anticipation', 'fear', 'joy',   'surprise'],
           'hilarious': ['joy',  'surprise'], 'hilarity': ['joy'],
           'hindering': [ 'sadness'],
           'hire': ['anticipation', 'joy',  'trust'], 'hiss': ['anger', 'fear'],
           'hit': ['anger'],
           'hoary': [ 'sadness'],
           'hoax': ['anger', 'disgust',  'sadness', 'surprise'], 'hobby': ['joy'],
           'hobo': [ 'sadness'], 'hog': ['disgust'],
           'holiday': ['anticipation', 'joy'],
           'holiness': ['anticipation', 'fear', 'joy',  'surprise', 'trust'],
           'hollow': [ 'sadness'], 'holocaust': ['anger', 'disgust', 'fear',  'sadness'],

           'homeless': ['anger', 'anticipation', 'disgust', 'fear',  'sadness'],
           'homesick': [ 'sadness'], 'homework': ['fear'], 'homicidal': ['anger', 'fear'],
           'homicide': ['anger', 'disgust', 'fear',  'sadness'],

           'honest': ['anger', 'disgust', 'fear', 'joy',  'sadness', 'trust'],
           'honesty': [ 'trust'],
           'honeymoon': ['anticipation', 'joy',  'surprise', 'trust'], 'honor': [ 'trust'],
           'honorable': [ 'trust'], 'hood': ['anger', 'disgust', 'fear'], 'hooded': ['fear'],
           'hoot': ['anger', 'disgust'],
           'hope': ['anticipation', 'joy',  'surprise', 'trust'],
           'hopeful': ['anticipation', 'joy',  'surprise', 'trust'],
           'hopeless': ['fear',  'sadness'],
           'hopelessness': ['anger', 'disgust', 'fear',  'sadness'], 'horde': [ 'surprise'],
           'horizon': ['anticipation'], 'horoscope': ['anticipation'],
           'horrible': ['anger', 'disgust', 'fear'],
           'horrid': ['anger', 'disgust', 'fear',  'sadness'],
           'horrific': ['anger', 'disgust', 'fear',  'sadness'], 'horrified': ['fear'],
           'horrifying': ['disgust', 'fear',  'sadness'],
           'horror': ['anger', 'disgust', 'fear',  'sadness', 'surprise'],
           'horrors': ['fear',  'sadness'], 'horse': ['trust'], 'hospice': ['sadness'],
           'hospital': ['fear', 'sadness', 'trust'],
           'hostage': ['anger', 'fear'], 'hostile': ['anger', 'disgust', 'fear'],
           'hostilities': ['anger', 'fear'], 'hostility': ['anger', 'disgust'],
           'hot': ['anger'],
           'howl': ['anger', 'disgust', 'fear',  'sadness', 'surprise'],
           'huff': ['anger', 'disgust'], 'hug': ['joy',  'trust'], 'hulk': ['disgust'],
           'humanitarian': ['anticipation', 'joy',  'surprise', 'trust'],
           'humanity': ['joy',  'trust'], 'humble': ['disgust',   'sadness'],
           'humbled': [ 'sadness'],
           'humbug': ['anger', 'disgust',  'sadness'], 'humiliate': ['anger',  'sadness'],
           'humiliating': ['disgust'], 'humiliation': ['disgust',  'sadness'],
           'humility': [ 'trust'],  'humorous': ['joy'],
           'hungry': ['anticipation'],
           'hunter': ['anticipation', 'fear',  'sadness'],
           'hunting': ['anger', 'anticipation', 'fear'], 'hurrah': ['joy'],
           'hurricane': ['fear'], 'hurried': ['anticipation'], 'hurry': ['anticipation'],
           'hurt': ['anger', 'fear',  'sadness'],
           'hurtful': ['anger', 'disgust', 'fear',  'sadness'],
           'hurting': ['anger', 'fear',  'sadness'], 'husbandry': [ 'trust'],
           'hut': [ 'sadness'],
           'hydra': ['fear'], 'hydrocephalus': ['disgust', 'fear',  'sadness'],
           'hymn': ['anticipation', 'joy',  'sadness', 'trust'],
           'hype': ['anticipation'],
           'hypertrophy': ['disgust', 'fear', 'surprise'],
           'hypocrite': ['disgust'], 'hypocritical': ['disgust'],
           'hypothesis': ['anticipation', 'surprise'], 'hysteria': ['fear'],
           'hysterical': ['anger', 'fear'],
           'idiocy': ['anger', 'disgust',  'sadness'], 'idiot': ['disgust'],
           'idiotic': ['anger', 'disgust'],
           'idolatry': ['disgust', 'fear'],
           'ignorant': ['disgust'],
           'ill': ['anger', 'disgust', 'fear',  'sadness'],
           'illegal': ['anger', 'disgust', 'fear',  'sadness'],
           'illegality': ['anger', 'disgust', 'fear'],
           'illegitimate': ['anger', 'disgust', 'fear',  'sadness', 'surprise'],
           'illicit': ['anger', 'disgust', 'fear'], 'illiterate': ['disgust'],
           'illness': ['fear',  'sadness'],
           'illuminate': ['anticipation', 'joy',  'surprise'],
           'illumination': ['joy',  'surprise', 'trust'], 'illusion': [ 'surprise'],

           'immaculate': ['joy',  'trust'],
           'immature': ['anticipation'], 'immaturity': ['anger', 'anticipation'],
           'immediacy': ['surprise'], 'immediately': ['anticipation'],
           'immerse': ['anticipation', 'fear', 'joy',  'surprise', 'trust'],
           'immigrant': ['fear'], 'imminent': ['anticipation', 'fear'],
           'immoral': ['anger', 'disgust', 'fear',  'sadness'],
           'immorality': ['anger', 'disgust'],
           'immortality': ['anticipation'], 'immovable': [  'trust'],
           'immunization': ['trust'],
           'impart': [ 'trust'], 'impartial': [ 'trust'],
           'impartiality': [ 'trust'],
           'impatient': ['anticipation'], 'impeach': ['disgust', 'fear'],
           'impeccable': [ 'trust'],
           'impending': ['anticipation', 'fear'], 'impenetrable': ['trust'],
           'impermeable': ['anger', 'fear'],

           'implicate': ['anger'], 'impolite': ['disgust'],
           'importance': ['anticipation'], 'important': [ 'trust'],
           'impossible': [ 'sadness'],
           'impotence': ['anger', 'fear',  'sadness'],

           'impressionable': ['trust'],
           'imprisoned': ['anger', 'disgust', 'fear',  'sadness'],
           'imprisonment': ['anger', 'disgust', 'fear',  'sadness'],
           'improve': ['anticipation', 'joy',  'trust'],
           'improvement': ['joy',  'trust'],  'improvisation': ['surprise'],
           'improvise': ['anticipation',  'surprise'], 'imprudent': [ 'sadness'],
           'impure': ['disgust'], 'impurity': ['disgust'],
           'inability': [ 'sadness'],

           'inadequate': [ 'sadness'], 'inadmissible': ['anger', 'disgust'],

           'inappropriate': ['anger', 'disgust',  'sadness'], 'inattention': ['anger'],
           'inaugural': ['anticipation'],
           'inauguration': ['anticipation', 'joy',  'trust'],
           'incarceration': ['anger', 'disgust', 'fear',  'sadness'],
           'incase': ['anger', 'disgust', 'fear',  'sadness'],
           'incendiary': ['anger', 'fear',  'surprise'], 'incense': ['anger'],
           'incest': ['anger', 'disgust', 'fear',  'sadness'],
           'incestuous': ['disgust'], 'incident': ['surprise'],
           'incite': ['anger', 'anticipation', 'fear'],
           'incline': ['trust'],
           'inclusion': ['trust'],
           'income': ['anticipation', 'joy',   'sadness', 'trust'],
           'incompatible': ['anger', 'disgust',  'sadness'],
           'incompetent': ['anger',  'sadness'],
           'incongruous': ['anger'],
           'inconsequential': [ 'sadness'], 'inconsiderate': ['anger', 'disgust',  'sadness'],
           'incontinence': ['surprise'],
           'inconvenient': ['anger', 'disgust',  'sadness'],
           'incredulous': ['anger', 'disgust'],
           'incrimination': ['fear',  'sadness'], 'incubus': ['disgust', 'fear'],
           'incurable': ['anger', 'disgust', 'fear',  'sadness'],
           'incursion': ['fear'], 'indecency': ['anger', 'disgust'],
           'indecent': ['disgust'],
           'indefensible': ['fear'], 'indelible': [ 'trust'],
           'indemnity': [ 'trust'], 'indent': ['trust'], 'indenture': ['anger'],
           'independence': ['anticipation', 'joy',  'surprise', 'trust'],
           'indestructible': [ 'trust'],
           'indict': ['anger', 'fear'], 'indictment': ['fear'],
           'indifference': ['anger', 'disgust', 'fear',  'sadness'], 'indigent': [ 'sadness'],
           'indignant': ['anger'], 'indignation': ['anger', 'disgust'],
           'indivisible': ['trust'],
           'indoctrination': ['anger', 'fear'],
           'indomitable': ['fear'],
           'ineffectual': ['disgust'], 'inefficiency': ['disgust',  'sadness'],
           'inefficient': [ 'sadness'], 'inept': ['anger', 'disgust'],
           'ineptitude': ['disgust', 'fear',  'sadness'],
           'inequality': ['anger', 'fear',  'sadness'],
           'inexcusable': ['anger', 'disgust',  'sadness'],

           'inexplicable': [ 'surprise'], 'infallibility': ['trust'],
           'infamous': ['anger', 'disgust', 'fear'], 'infamy': [ 'sadness'],
           'infant': ['anticipation', 'fear', 'joy',  'surprise'],
           'infanticide': ['anger', 'anticipation', 'disgust', 'fear',  'sadness'],
           'infantile': ['anger', 'disgust'], 'infarct': ['fear',  'surprise'],
           'infect': ['disgust'], 'infection': ['fear'],
           'infectious': ['disgust', 'fear',  'sadness'], 'inferior': [ 'sadness'],
           'inferno': ['anger', 'fear'],
           'infertility': [ 'sadness'], 'infestation': ['disgust', 'fear'],
           'infidel': ['anger', 'disgust', 'fear'],
           'infidelity': ['anger', 'disgust', 'fear',  'sadness'],

           'infinity': ['anticipation', 'joy',  'trust'],
           'infirmity': ['fear'],
           'inflation': ['fear'], 'inflict': ['anger', 'fear',  'sadness'],
           'infliction': ['fear',  'sadness'],
           'influential': [ 'trust'],  'inform': ['trust'],
           'infraction': ['anger'],
           'infrequent': ['surprise'],
           'inheritance': ['anticipation', 'joy',  'surprise', 'trust'],
           'inhibit': ['anger', 'disgust',  'sadness'], 'inhospitable': [ 'sadness'],
           'inhuman': ['anger', 'disgust', 'fear',  'sadness'], 'inhumanity': [ 'sadness'],
           'inimical': ['anger',  'sadness'], 'inimitable': [ 'trust'],
           'iniquity': ['disgust'], 'injection': ['fear'],
           'injure': ['anger', 'fear',  'sadness'], 'injured': ['fear',  'sadness'],
           'injurious': ['anger', 'fear',  'sadness'],
           'injury': ['anger', 'fear',  'sadness'], 'injustice': ['anger'],
           'inmate': ['disgust', 'fear'],  'innocent': [ 'trust'],

           'inoculation': ['anticipation', 'trust'],
           'inoperative': ['anger'],  'inquiry': ['anticipation'],
           'insane': ['anger', 'fear'],
           'insanity': ['anger', 'disgust', 'fear',  'sadness'],
           'insecure': ['anger', 'fear',  'sadness'], 'insecurity': ['fear'],
           'inseparable': ['joy',  'trust'], 'insidious': ['anger', 'disgust', 'fear'],

           'insignificant': ['anger',  'sadness'],
           'insolvency': ['fear',  'sadness', 'surprise'],
           'insolvent': ['fear',  'sadness', 'trust'],
           'inspiration': ['anticipation', 'joy'],
           'inspire': ['anticipation', 'joy',  'trust'],
           'inspired': ['joy',  'surprise', 'trust'], 'instability': ['disgust', 'fear'],
           'install': ['anticipation'],
           'instinctive': ['anger', 'disgust', 'fear'], 'institute': ['trust'],
           'instruct': [ 'trust'], 'instruction': [ 'trust'],
           'instructions': ['anticipation', 'trust'], 'instructor': ['anticipation',  'trust'],
           'insufficiency': ['anger'],
           'insulation': ['trust'],
           'insult': ['anger', 'disgust',  'sadness', 'surprise'],
           'insulting': ['anger', 'disgust', 'fear',  'sadness'], 'insure': [ 'trust'],
           'insurmountable': ['fear',  'sadness'],
           'insurrection': ['anger'], 'intact': [ 'trust'],
           'integrity': [ 'trust'],
           'intelligence': ['fear', 'joy',  'trust'], 'intelligent': [ 'trust'],
           'intend': ['trust'], 'intended': ['anticipation'],
           'intense': ['anger', 'disgust', 'fear', 'joy',   'surprise', 'trust'],
           'inter': [ 'sadness'], 'intercede': ['fear', 'sadness'], 'intercession': ['trust'],

           'interested': ['disgust',  'sadness'],
           'interim': ['anticipation'], 'interior': ['disgust',  'trust'],
           'interment': [ 'sadness'],
           'interminable': ['anger', 'anticipation'], 'intermission': ['anticipation'],
           'interrogate': ['fear'], 'interrogation': ['fear'], 'interrupt': ['anger',  'surprise'],
           'interrupted': [ 'sadness'], 'intervention': [  'sadness'],
           'interviewer': ['fear'],  'intestinal': ['disgust'],
           'intimate': ['anticipation', 'joy',  'trust'], 'intimately': ['anticipation', 'fear', 'joy'],
           'intimidate': ['fear'], 'intimidation': ['anger', 'fear'],
           'intolerable': ['anger'], 'intolerance': ['anger', 'disgust', 'fear'],
           'intolerant': ['anger', 'disgust', 'fear',  'sadness'],
           'intoxicated': ['disgust'], 'intractable': ['anger'],
           'intrigue': ['anticipation', 'fear',  'surprise'],
           'intruder': ['anger', 'fear',  'surprise'], 'intrusion': ['fear'],
           'intrusive': ['anger', 'disgust', 'fear',  'surprise'], 'intuition': [ 'trust'],
           'intuitively': ['anticipation'],
           'invade': ['anger', 'fear',  'sadness', 'surprise'],
           'invader': ['anger', 'fear',  'sadness'], 'invalid': ['sadness'],

           'invasion': ['anger'],
           'investigation': ['anticipation'],
           'invitation': ['anticipation'],
           'invite': ['anticipation', 'joy',  'surprise', 'trust'],
           'inviting': ['anticipation', 'joy',  'surprise', 'trust'],
           'invocation': ['anticipation', 'trust'], 'invoke': ['anticipation'],
           'involution': ['anger'], 'involvement': ['anger'], 'irate': ['anger'],
           'ire': ['anger'], 'iris': ['fear'], 'iron': [ 'trust'],
           'irrational': ['disgust', 'fear'],
           'irreconcilable': ['anger', 'fear',  'sadness'],
           'irrefutable': [ 'trust'],
           'irreparable': ['fear',  'sadness'],

           'irritability': ['anger'], 'irritable': ['anger'],
           'irritating': ['anger', 'disgust'],
           'irritation': ['anger', 'disgust',  'sadness'], 'isolate': ['sadness'],
           'isolated': ['fear',  'sadness'], 'isolation': [ 'sadness'], 'jab': ['anger'],
           'jackpot': ['anticipation', 'joy',  'surprise'],
           'jail': ['fear',  'sadness'],  'janitor': ['disgust'],
           'jarring': ['fear',  'sadness'], 'jaundice': ['fear'],
           'jaws': ['fear'], 'jealous': ['anger', 'disgust'],
           'jealousy': ['anger', 'disgust', 'fear',  'sadness'],
           'jeopardize': ['anger', 'fear'], 'jeopardy': ['anticipation', 'fear'],
           'jerk': ['anger', 'surprise'], 'jest': ['joy',  'surprise'],
           'john': ['disgust'],
           'joker': ['joy',  'surprise'],  'jolt': ['surprise'],
           'journalism': ['trust'],
           'journey': ['anticipation', 'fear', 'joy'], 'journeyman': ['trust'],
           'jovial': ['joy'], 'joy': ['joy'], 'joyful': ['joy',  'trust'],
           'joyous': ['joy'], 'jubilant': ['joy',  'surprise', 'trust'],
           'jubilee': ['joy',  'surprise'], 'judgment': ['surprise'],
           'judicial': ['anticipation',  'trust'], 'judiciary': ['anticipation', 'trust'],
           'judicious': [ 'trust'],  'jump': ['joy'],
           'jungle': ['fear'],   'jurisprudence': ['sadness'],
           'jurist': ['trust'], 'jury': ['trust'], 'justice': [ 'trust'],
           'justifiable': [ 'trust'],
           'kennel': ['sadness'],
           'kerosene': ['fear'],   'khan': ['fear', 'trust'],
           'kick': ['anger'], 'kicking': ['anger'],
           'kidnap': ['anger', 'fear',  'sadness', 'surprise'], 'kill': ['fear',  'sadness'],
           'killing': ['anger', 'fear',  'sadness'], 'kind': ['joy',  'trust'],
           'kindred': ['anticipation', 'joy',  'trust'],
           'kiss': ['anticipation', 'joy',  'surprise'], 'kite': ['disgust'],
           'kitten': ['joy',  'trust'],  'knell': ['fear',  'sadness'],
           'knickers': ['trust'],
           'kudos': ['joy'], 'label': ['trust'],
           'labor': ['anticipation', 'joy',  'surprise', 'trust'], 'labored': [ 'sadness'],
           'labyrinth': ['anticipation'],
           'lace': ['anger', 'fear',   'sadness', 'trust'],

           'lagging': ['anger', 'anticipation', 'disgust',  'sadness'],
           'lamb': ['joy',  'trust'], 'lament': ['disgust', 'fear',  'sadness'],
           'lamenting': ['sadness'],   'landmark': ['trust'],
           'landslide': ['fear',  'sadness'],
           'languishing': ['fear',  'sadness'],
           'larceny': ['disgust'], 'larger': ['disgust', 'surprise', 'trust'],
           'laser': [ 'trust'], 'lash': ['anger', 'fear'], 'late': [ 'sadness'],
           'latent': ['anger', 'anticipation', 'disgust',  'surprise'],
           'latrines': ['disgust'],  'laugh': ['joy',  'surprise'],
           'laughable': ['disgust'], 'laughing': ['joy'],
           'laughter': ['anticipation', 'joy',  'surprise'], 'launch': ['anticipation'],
           'laureate': [ 'trust'],  'laurels': ['joy'],
           'lava': ['anger', 'fear'], 'lavatory': ['disgust'],  'law': ['trust'],
           'lawful': [ 'trust'], 'lawlessness': ['anger', 'fear'],
           'lawsuit': ['anger', 'disgust', 'fear',  'sadness', 'surprise'],
           'lawyer': ['anger', 'disgust', 'fear'], 'lax': [ 'sadness'],
           'laxative': ['disgust', 'fear'],
           'leader': [ 'trust'], 'leading': ['trust'],
           'leaning': ['trust'],
           'leave': [ 'sadness', 'surprise'],
           'leeches': ['disgust', 'fear'], 'leer': ['disgust'],
           'leery': ['surprise'],  'legal': [ 'trust'],
           'legalized': ['anger', 'fear', 'joy',  'trust'],
           'legislator': ['trust'], 'legislature': ['trust'],
           'legitimacy': ['trust'], 'leisure': ['anticipation', 'joy',  'surprise', 'trust'],
           'lemon': ['disgust'], 'lender': ['trust'],
           'leprosy': ['disgust', 'fear',  'sadness'],
           'lesbian': ['disgust',  'sadness'], 'lessen': ['anticipation'],
           'lesser': ['disgust'], 'lesson': ['anticipation',  'trust'],
           'lethal': ['disgust', 'fear',  'sadness'], 'lethargy': [ 'sadness'],
           'letter': ['anticipation'], 'lettered': ['anticipation',  'trust'],
           'leukemia': ['anger', 'fear',  'sadness'], 'levee': ['trust'], 'level': [ 'trust'],
           'lewd': ['disgust'],
           'liar': ['disgust'], 'libel': ['anger', 'fear',  'trust'],

           'liberate': ['anger', 'anticipation', 'joy',  'surprise', 'trust'],
           'liberation': ['anticipation', 'joy',  'surprise'],
           'liberty': ['anticipation', 'joy',  'surprise', 'trust'],
           'lick': ['disgust'], 'lie': ['anger', 'disgust',  'sadness'],
           'lieutenant': ['trust'],  'lifeless': ['fear',  'sadness'],
           'lightning': ['anger', 'fear', 'surprise'],
           'liking': ['joy',  'trust'], 'limited': ['anger',  'sadness'],
           'lines': ['fear'], 'linger': ['anticipation'], 'linguist': [ 'trust'],
           'lion': ['fear'], 'liquor': ['anger', 'joy',  'sadness'],
           'listless': [ 'sadness'],
           'litigate': ['anger', 'anticipation', 'disgust', 'fear',  'sadness'],
           'litigious': ['anger', 'disgust'],
           'livid': ['anger', 'disgust'],
           'loath': ['anger'], 'loathe': ['anger', 'disgust'],
           'loathing': ['disgust'], 'loathsome': ['anger', 'disgust'],
           'localize': ['anticipation'], 'lockup': ['fear',  'sadness'],
           'locust': ['fear'], 'lodging': ['trust'],
           'lone': ['sadness'], 'loneliness': ['fear',  'sadness'],
           'lonely': ['anger', 'disgust', 'fear',  'sadness'], 'lonesome': [ 'sadness'],
           'long': ['anticipation'],  'longing': ['anticipation', 'sadness'],
           'loo': ['disgust'], 'loom': ['anticipation', 'fear'],
           'loon': ['disgust'],
           'lord': ['disgust',   'trust'],
           'lose': ['anger', 'disgust', 'fear',  'sadness', 'surprise'],
           'losing': ['anger',  'sadness'], 'loss': ['anger', 'fear',  'sadness'],
           'lost': [ 'sadness'], 'lottery': ['anticipation'], 'loudness': ['anger'],
           'louse': ['disgust'], 'lovable': ['joy',  'trust'],
           'love': ['joy'],
           'lovely': ['anticipation', 'joy',  'sadness', 'surprise', 'trust'],
           'lovemaking': ['joy',  'trust'], 'lover': ['anticipation', 'joy',  'trust'],
           'loving': ['joy',  'trust'], 'lower': [ 'sadness'],
           'lowest': [ 'sadness'],  'lowly': [ 'sadness'],
           'loyal': ['fear', 'joy',  'surprise', 'trust'], 'loyalty': [ 'trust'],
           'luck': ['anticipation', 'joy',  'surprise'], 'lucky': ['joy',  'surprise'],
           'lull': ['anticipation'],
           'lumpy': ['disgust'], 'lunacy': ['anger', 'disgust', 'fear',  'sadness'],
           'lunatic': ['anger', 'disgust', 'fear'], 'lunge': ['surprise'],
           'lurid': ['disgust'],
           'luscious': ['anticipation', 'joy'], 'lush': ['disgust',  'sadness'],
           'lust': ['anticipation'], 'luster': ['joy'],
           'lusty': ['disgust'],
           'luxurious': ['joy'], 'luxury': ['joy'],
           'lying': ['anger', 'disgust'], 'lynch': ['anger', 'disgust', 'fear',  'sadness'],
           'lyre': ['joy'], 'lyrical': ['joy'], 'mace': ['fear'],
           'machine': ['trust'], 'mad': ['anger', 'disgust', 'fear',  'sadness'],
           'madden': ['anger', 'fear'], 'madman': ['anger', 'fear'],
           'madness': ['anger', 'fear'], 'mafia': ['fear'], 'mage': ['fear'],
           'maggot': ['disgust'], 'magical': ['anticipation', 'joy',  'surprise'],
           'magician': ['surprise'], 'magnet': [ 'trust'],
           'magnificence': ['anticipation', 'joy',  'trust'],
           'magnificent': ['anticipation', 'joy',  'surprise', 'trust'],
           'mail': ['anticipation'],  'mainstay': [ 'trust'],
           'maintenance': ['trust'], 'majestic': ['anticipation', 'joy',  'surprise', 'trust'],
           'majesty': [ 'trust'],  'majority': ['joy',  'trust'],
           'malaise': [ 'sadness'],
           'malaria': ['disgust', 'fear',  'sadness'],
           'malevolent': ['anger', 'disgust', 'fear',  'sadness'],
           'malfeasance': ['disgust'],
           'malice': ['anger', 'fear'],
           'malicious': ['anger', 'disgust', 'fear',  'sadness'],
           'malign': ['anger', 'disgust'], 'malignancy': ['fear',  'sadness'],
           'malignant': ['anger', 'fear'], 'malpractice': ['anger'], 'mamma': ['trust'],
           'manage': [ 'trust'], 'management': [ 'trust'], 'mandamus': ['fear'],
           'mandarin': [ 'trust'], 'mange': ['disgust', 'fear'],
           'mangle': ['anger', 'disgust', 'fear',  'sadness'],
           'maniac': ['anger', 'fear'],
           'manifestation': ['fear'],
           'manipulation': ['anger', 'fear'],

           'manslaughter': ['anger', 'disgust', 'fear',  'sadness', 'surprise'], 'manual': ['trust'],
           'manure': ['disgust'],
           'margin': [ 'sadness'], 'marine': ['trust'],

           'marriage': ['anticipation', 'joy',  'trust'], 'marrow': ['joy',  'trust'],
           'marry': ['anticipation', 'fear', 'joy',  'surprise', 'trust'],
           'marshal': [ 'trust'], 'martial': ['anger'],
           'martyr': ['fear',  'sadness'], 'martyrdom': ['fear',  'sadness'],
           'marvel': [ 'surprise'], 'marvelous': ['joy'], 'marvelously': ['joy'],
           'masochism': ['anger', 'disgust', 'fear'],
           'massacre': ['anger', 'disgust', 'fear',  'sadness'], 'massage': ['joy'],
           'masterpiece': ['joy'],
           'mastery': ['anger', 'joy',  'trust'], 'matchmaker': ['anticipation'],
           'mate': [ 'trust'],  'materialist': ['disgust'],
           'maternal': ['anticipation'], 'mathematical': ['trust'],
           'matrimony': ['anticipation', 'joy',  'trust'], 'matron': [ 'trust'],
           'mausoleum': ['sadness'], 'maxim': ['trust'],
           'meaningless': [ 'sadness'],
           'measles': ['disgust', 'fear',  'sadness'], 'measure': ['trust'],
           'measured': [ 'trust'], 'medal': ['anticipation', 'joy',  'surprise', 'trust'],
           'meddle': ['anger'], 'mediate': ['anticipation',  'trust'],
           'mediator': ['anticipation',  'trust'],
           'medical': ['anticipation', 'fear',  'trust'],
           'meditate': ['anticipation', 'joy',  'trust'],
           'meek': ['sadness'],
           'melancholic': [ 'sadness'], 'melancholy': [ 'sadness'],
           'melee': ['fear'], 'melodrama': ['anger',  'sadness'],
           'meltdown': [ 'sadness'],
           'memorable': ['joy',  'surprise', 'trust'], 'memorials': ['sadness'],
           'menace': ['anger', 'fear'], 'menacing': ['anger', 'fear'],
           'mentor': [ 'trust'],
           'mercenary': ['fear'], 'merchant': ['trust'],
           'merciless': ['fear'],  'merge': ['anticipation'],
           'merit': [ 'trust'], 'meritorious': ['joy',  'trust'],
           'merriment': ['joy',  'surprise'], 'merry': ['joy'],
           'mess': ['disgust'], 'messenger': ['trust'], 'messy': ['disgust'],

           'microscope': ['trust'],
           'midwife': ['anticipation', 'joy',   'trust'],
           'mighty': ['anger', 'fear', 'joy',  'trust'], 'mildew': ['disgust'],
           'military': ['fear'], 'militia': ['anger', 'fear',  'sadness'], 'mill': ['anticipation'],
           'mimicry': [ 'surprise'],

           'ministry': ['joy',  'trust'],
           'miracle': ['anticipation', 'joy',  'surprise', 'trust'],
           'miraculous': ['joy',  'surprise'], 'mire': ['disgust'],
           'mirth': ['joy'], 'misbehavior': ['anger', 'disgust',  'surprise'],
           'miscarriage': ['fear',  'sadness'],
           'misconception': ['anger', 'disgust', 'fear'], 'misconduct': ['disgust'],
           'miserable': ['anger', 'disgust',  'sadness'], 'miserably': [ 'sadness'],
           'misery': ['anger', 'disgust', 'fear',  'sadness'],
           'misfortune': ['fear',  'sadness'], 'misguided': ['disgust'],
           'mishap': ['disgust', 'fear',  'sadness', 'surprise'],
           'mislead': ['anger', 'fear',  'trust'], 'misleading': ['anger', 'disgust'],

           'misplace': ['anger', 'disgust'],
           'misrepresentation': [ 'sadness'], 'misrepresented': ['anger'],
           'missile': ['fear'], 'missing': ['fear',  'sadness'],
           'misstatement': ['anger', 'disgust'], 'mistake': [ 'sadness'],
           'mistaken': ['fear'], 'mistress': ['anger', 'disgust'],
           'mistrust': ['disgust', 'fear'],
           'misunderstanding': ['anger',  'sadness'],
           'mite': ['disgust'], 'moan': ['fear', 'sadness'], 'moat': ['trust'],
           'mob': ['anger', 'fear'], 'mobile': ['anticipation'], 'mockery': ['disgust'],
           'mocking': ['anger', 'disgust',  'sadness'],
           'moderator': [ 'trust'], 'modest': [ 'trust'],
           'modify': ['surprise'], 'molestation': ['anger', 'disgust', 'fear',  'sadness'],
           'momentum': ['anticipation'], 'monetary': ['anticipation'],
           'money': ['anger', 'anticipation', 'joy',  'surprise', 'trust'],
           'monk': [ 'trust'], 'monochrome': ['disgust'], 'monogamy': ['trust'],
           'monsoon': [ 'sadness'], 'monster': ['fear'],
           'monstrosity': ['anger', 'disgust', 'fear',  'surprise'],
           'moody': ['anger',  'sadness'], 'moorings': ['trust'],
           'moral': ['anger',  'trust'], 'morality': [ 'trust'],
           'morals': ['anger', 'anticipation', 'disgust', 'joy',  'surprise', 'trust'],
           'morbid': [ 'sadness'], 'morbidity': ['anger', 'disgust', 'fear',  'sadness'],
           'morgue': ['disgust', 'fear',  'sadness'], 'moribund': [ 'sadness'],
           'morn': ['anticipation'],   'morrow': ['anticipation'],
           'mortality': ['anger', 'fear',  'sadness'],
           'mortgage': ['fear'], 'mortgagee': ['trust'], 'mortgagor': ['fear'],
           'mortification': ['anticipation', 'disgust', 'fear',  'sadness'],
           'mortuary': ['fear',  'sadness'], 'mosque': ['anger'],
           'mosquito': ['anger', 'disgust'],
           'mother': ['anticipation', 'joy',   'sadness', 'trust'],
           'motherhood': ['joy',  'trust'], 'motion': ['anticipation'],
           'mountain': ['anticipation'], 'mourn': [ 'sadness'],
           'mournful': ['anger', 'fear',  'sadness'], 'mourning': [ 'sadness'],
           'mouth': ['surprise'], 'mouthful': ['disgust'],
           'muck': ['disgust'], 'mucous': ['disgust'], 'mucus': ['disgust'],
           'muddy': ['disgust'],
           'muff': ['anger', 'disgust'], 'mug': ['anger', 'fear',   'sadness'],
           'mule': ['anger',  'trust'], 'mum': ['fear'],
           'murder': ['anger', 'disgust', 'fear',  'sadness', 'surprise'],
           'murderer': ['anger', 'disgust', 'fear',  'sadness'],
           'murderous': ['anger', 'disgust', 'fear',  'sadness', 'surprise'],
           'murky': ['disgust',  'sadness'],
           'music': ['joy',  'sadness'],
           'musical': ['anger', 'anticipation', 'joy',  'sadness', 'surprise', 'trust'],
           'musket': ['fear'],  'musty': ['disgust'],
           'mutable': ['anticipation'],  'mutilated': ['disgust'],
           'mutilation': ['anger', 'disgust', 'fear',  'sadness'],
           'mutiny': ['anger', 'disgust', 'fear',  'surprise'], 'mutter': ['anger'],
           'muzzle': ['fear'], 'myopia': ['anger',  'sadness'],
           'mysterious': ['anticipation', 'fear', 'surprise'], 'mystery': ['anticipation', 'surprise'],
           'mystic': ['surprise'], 'nab': [ 'surprise'],
           'nag': ['anger'],  'nameless': ['disgust'],
           'nap': ['joy'], 'napkin': ['sadness'], 'nappy': ['disgust'],
           'nascent': ['anticipation'],
           'nasty': ['anger', 'disgust', 'fear',  'sadness'], 'nation': ['trust'],
           'nausea': ['disgust'],
           'nauseous': ['disgust',  'sadness'], 'navigable': ['anticipation'],
           'navigator': ['anticipation', 'trust'],
           'necessity': ['sadness'],
           'nefarious': ['disgust', 'fear',  'sadness', 'surprise'],
           'negation': ['anger'], 'negative':[ 'sadness'],'neglected': ['anger', 'disgust',  'sadness'],
            'negotiate': [ 'trust'],  'negro': [ 'sadness'],
            'neighbor': ['anticipation',  'trust'], 'neighborhood': ['anticipation'],
            'nepotism': ['anger', 'disgust',  'sadness'],
            'nervous': ['anticipation', 'fear'], 'nervousness': ['fear'], 'nest': ['trust'],
            'nestle': [ 'trust'], 'nether': ['anger', 'fear',  'sadness'],
            'nettle': ['anger', 'disgust'], 'network': ['anticipation'],
            'neuralgia': ['fear',  'sadness'], 'neurosis': ['fear',  'sadness'],
            'neurotic': ['disgust', 'fear'], 'neutral': ['anticipation', 'trust'],
            'neutrality': ['trust'], 'newcomer': ['fear', 'surprise'], 'nicotine': ['disgust'],
            'nightmare': ['fear'],
            'nobility': ['anticipation',  'trust'], 'noble': [ 'trust'],
            'nobleman': [ 'trust'],  'noisy': ['anger'],
            'noncompliance': ['anger', 'anticipation', 'fear',  'sadness'],
            'nonsensical': [ 'sadness'], 'noose': [ 'sadness'],
            'nose': ['disgust'], 'notable': ['joy',  'trust'],
            'notary': ['trust'],
            'nothingness': [ 'sadness'], 'notification': ['anticipation'],
            'notoriety': ['anger', 'disgust', 'fear'],
            'noxious': ['disgust', 'fear'], 'nuisance': ['anger'],
            'nullify': [ 'surprise'],
            'numbness': [ 'sadness'], 'nun': [ 'trust'], 'nurse': [ 'trust'],
            'nursery': ['joy',  'trust'],
            'nurture': ['anger', 'anticipation', 'disgust', 'fear', 'joy',  'trust'],
            'nutritious': [ 'sadness'],
            'oasis': ['anticipation', 'joy',  'trust'], 'oath': [ 'trust'],
            'obedience': [ 'trust'], 'obese': ['disgust'],
            'obesity': ['disgust',  'sadness'], 'obey': ['fear', 'trust'],
            'obi': ['disgust', 'fear'], 'obit': [ 'sadness', 'surprise'],
            'obituary': [ 'sadness'], 'objection': ['anger'],
            'objective': ['anticipation',  'trust'], 'oblige': [ 'trust'],
            'obliging': ['anger', 'anticipation', 'disgust', 'fear', 'joy',  'surprise', 'trust'],
            'obligor': ['fear'], 'obliterate': ['anger', 'fear',  'sadness'],
            'obliterated': ['anger', 'fear'], 'obliteration': ['fear',  'sadness'],
            'oblivion': ['anger', 'fear'], 'obnoxious': ['anger', 'disgust',  'sadness'],
            'obscene': ['disgust'], 'obscenity': ['anger', 'disgust'],

            'obstacle': ['anger', 'fear',  'sadness'], 'obstetrician': ['trust'],
            'obstruct': ['anger', 'fear'],
            'obstructive': ['anger'], 'obtainable': ['joy'],
            'obvious': [ 'trust'], 'occasional': ['surprise'], 'occult': ['disgust', 'fear'],
            'occupant': [ 'trust'],
            'octopus': ['disgust'], 'oddity': ['disgust',  'sadness', 'surprise'],
            'odious': ['anger', 'disgust', 'fear'],
            'offend': ['anger', 'disgust'], 'offended': ['anger',  'sadness'],
            'offender': ['anger', 'disgust', 'fear',  'sadness'],
            'offense': ['anger', 'disgust', 'fear',  'sadness'],
            'offensive': ['anger', 'disgust'],  'offering': ['trust'],
            'officer': [ 'trust'], 'official': ['trust'],
            'offset': ['anticipation'],
            'ogre': ['disgust', 'fear'], 'older': ['sadness'], 'olfactory': ['anticipation'],
            'omen': ['anticipation', 'fear'],
            'ominous': ['anticipation', 'fear'],
            'omnipotence': ['fear'], 'omniscient': [ 'trust'],
            'onerous': ['anger',  'sadness'], 'ongoing': ['anticipation'], 'onset': ['anticipation'],
            'ooze': ['disgust'],
            'opera': ['anger', 'anticipation', 'fear', 'joy',  'sadness', 'surprise', 'trust'],
            'operation': ['fear', 'trust'],
            'opinionated': ['anger'], 'opium': ['anger', 'disgust', 'fear',  'sadness'],
            'opponent': ['anger', 'anticipation', 'disgust', 'fear'], 'opportune': ['joy'],
            'opportunity': ['anticipation'],
            'opposed': ['anger', 'fear'], 'opposition': ['anger'],
            'oppress': ['anger', 'disgust', 'fear',  'sadness'],
            'oppression': ['anger', 'disgust', 'fear',  'sadness'],
            'oppressive': ['anger', 'disgust', 'fear',  'sadness'],
            'oppressor': ['anger', 'fear',  'sadness'],
            'optimism': ['anticipation', 'joy',  'surprise', 'trust'], 'optimist': [ 'trust'],
            'oracle': ['anticipation',  'trust'],
            'orc': ['anger', 'disgust', 'fear'],
            'orchestra': ['anger', 'joy',  'sadness', 'trust'], 'ordained': [ 'trust'],
            'ordeal': ['anger',  'surprise'],  'ordinance': ['trust'],
            'ordination': ['anticipation', 'joy',  'trust'], 'ordnance': ['fear'],
            'organ': ['anticipation', 'joy'],
            'organization': ['anticipation', 'joy',  'surprise', 'trust'],
            'orgasm': ['anticipation', 'joy'],
            'originality': [ 'surprise'],
            'orphan': ['fear',  'sadness'], 'orthodoxy': ['trust'],
            'oust': ['anger',  'sadness'],
            'outburst': ['anger', 'fear', 'joy',   'sadness', 'surprise'],
            'outcast': ['disgust', 'fear',  'sadness'],
            'outcry': ['anger', 'fear',  'surprise'], 'outdo': ['anticipation'],
            'outhouse': ['disgust'],
            'outpost': ['fear'], 'outrage': ['anger', 'disgust'], 'outrageous': ['surprise'],
            'outsider': ['fear'], 'outstanding': ['joy'],
            'ovation': [ 'sadness'], 'overbearing': ['anger'],

            'overdue': ['anticipation',  'sadness', 'surprise'], 'overestimate': ['surprise'],

            'overjoyed': ['joy'], 'overload': [ 'sadness'],
            'overpowering': ['anger', 'fear'],
            'overpriced': ['anger', 'disgust',  'sadness'],  'overt': ['fear'],
            'overthrow': ['anticipation', 'fear'], 'overture': ['anticipation'],
            'overwhelmed': [ 'sadness'],

            'owing': ['anger', 'anticipation', 'disgust', 'fear',  'sadness', 'trust'],
            'pact': ['trust'],
            'paddle': ['anticipation'],
            'pain': ['fear',  'sadness'], 'pained': ['fear',  'sadness'],
            'painful': ['anger', 'disgust', 'fear',  'sadness'], 'painfully': [ 'sadness'],
            'palpable': ['surprise'],
            'palsy': ['disgust', 'fear',  'sadness'],
            'pandemic': ['fear',  'sadness'], 'pang': [ 'surprise'],
            'panic': ['fear'],   'parachute': ['fear'],
            'parade': ['anticipation', 'fear', 'joy',   'surprise'],
            'paragon': ['anticipation', 'joy',  'trust'],
            'paralysis': ['anger', 'anticipation', 'fear',  'sadness'],
            'paralyzed': ['anger', 'fear',  'sadness', 'surprise'],
            'paranoia': ['fear'],
            'parasite': ['disgust', 'fear'],
            'pare': ['anger', 'anticipation', 'fear',  'sadness'],
            'parietal': [ 'trust'], 'parish': ['trust'], 'parliament': ['trust'],
            'parole': ['anticipation'], 'parrot': ['disgust'],
            'partake': [ 'trust'],  'parting': ['sadness'],
            'partnership': [ 'trust'],
            'passenger': ['anticipation'],
            'passion': ['anticipation', 'joy',  'trust'],
            'passionate': ['anticipation', 'joy',  'trust'],
            'pastor': ['joy',  'trust'],
            'pastry': ['joy'],
            'pathetic': ['disgust',  'sadness'], 'patience': ['anticipation',  'trust'],
            'patient': ['anticipation'], 'patriarchal': [ 'trust'], 'patrol': ['trust'],
            'patron': [ 'trust'], 'patronage': [ 'trust'],
            'patter': ['anger'], 'paucity': ['anger', 'disgust',  'sadness'],
            'pauper': [ 'sadness'], 'pavement': ['trust'], 'pawn': [ 'trust'],
            'pay': ['anticipation', 'joy',  'trust'], 'payback': ['anger'],
            'peace': ['anticipation', 'joy',  'trust'],
            'peaceful': ['anticipation', 'joy',  'surprise', 'trust'],

            'pedigree': [ 'trust'],
            'penal': ['fear',  'sadness'],
            'penalty': ['anger', 'fear',  'sadness'], 'penance': ['fear', 'sadness'],
            'penetration': ['anger', 'fear'],
            'penitentiary': ['anger'], 'pensive': ['sadness'], 'perceive': [ 'trust'],
            'perchance': ['surprise'],
            'perdition': ['anger', 'disgust', 'fear',  'sadness'], 'perennial': [ 'trust'],
            'perfect': ['anticipation', 'joy',  'trust'],
            'perfection': ['anticipation', 'joy',  'surprise', 'trust'],
            'peri': ['surprise'], 'peril': ['anticipation', 'fear',  'sadness'],
            'perilous': ['anticipation', 'fear',  'sadness'], 'periodicity': ['trust'],
            'perish': ['fear',  'sadness'],
            'perished': [ 'sadness'], 'perishing': ['fear',  'sadness'],
            'perjury': ['fear',  'surprise'],
            'pernicious': ['anger', 'fear',  'sadness'],
            'perpetrator': ['anger', 'disgust', 'fear',  'sadness'], 'perpetuate': ['anticipation'],
            'perpetuity': ['anticipation',  'trust'],
            'perplexity': [ 'sadness'],
            'persecute': ['anger', 'fear'],
            'persecution': ['anger', 'disgust', 'fear',  'sadness'],
            'personal': ['trust'],
            'perspiration': ['disgust'], 'persuade': ['trust'], 'pertinent': [ 'trust'],
            'perturbation': ['fear'],
            'perverse': ['anger', 'disgust', 'fear'],
            'perversion': ['anger', 'disgust',  'sadness'], 'pervert': ['anger', 'disgust'],
            'perverted': ['disgust'], 'pessimism': ['anger', 'fear',  'sadness'],
            'pessimist': ['fear',  'sadness'], 'pest': ['anger', 'disgust', 'fear'],
            'pestilence': ['disgust', 'fear'],
            'petroleum': ['disgust'],  'phalanx': ['fear', 'trust'],
            'phantom': ['fear'],  'philanthropic': ['trust'],
            'philanthropist': [ 'trust'],
            'philosopher': [ 'trust'], 'phlegm': ['disgust'],
            'phony': ['anger', 'disgust'], 'physician': [ 'trust'],
            'physicist': ['trust'],
            'picket': ['anger', 'anticipation', 'fear'],
            'picketing': ['anger'],
            'picnic': ['anticipation', 'joy',  'surprise', 'trust'], 'picturesque': ['joy'],
            'pig': ['disgust'],
            'piles': ['disgust'], 'pill': [ 'trust'],
            'pillage': ['anger', 'disgust', 'fear'],
            'pilot': [ 'trust'],  'pimple': ['disgust'],
            'pine': [ 'sadness'], 'pinion': ['fear'],
            'pious': ['disgust',  'sadness', 'trust'],
            'pique': ['anger'],  'pirate': ['anger'],
            'pitfall': ['anger', 'disgust', 'fear',  'sadness', 'surprise'],
            'pith': [ 'trust'], 'pity': ['sadness'], 'pivot': [ 'trust'],
            'placard': ['surprise'],  'plagiarism': ['disgust'],
            'plague': ['disgust', 'fear',  'sadness'],
            'plaintive': ['sadness'], 'plan': ['anticipation'], 'planning': ['anticipation',  'trust'],

            'playful': ['anger', 'joy',  'surprise', 'trust'],
            'playground': ['anticipation', 'joy',  'surprise', 'trust'], 'playhouse': ['joy'],
            'plea': ['anticipation', 'fear',  'sadness'],
            'pleasant': ['anticipation', 'joy',  'surprise', 'trust'], 'pleased': ['joy'],
            'pleasurable': ['joy'], 'pledge': ['joy',  'trust'],

            'plight': ['anticipation', 'disgust', 'fear',  'sadness'],
            'plummet': ['fear'], 'plump': ['anticipation'],
            'plunder': ['anger', 'disgust', 'fear',  'sadness', 'surprise'],
            'plunge': ['fear'],
            'pneumonia': ['fear'], 'poaching': ['anger', 'disgust', 'fear',  'sadness'],
            'pointless': [ 'sadness'],
            'poison': ['anger', 'disgust', 'fear',  'sadness'],
            'poisoned': ['anger', 'disgust', 'fear',  'sadness'], 'poisoning': ['disgust'],
            'poisonous': ['anger', 'disgust', 'fear',  'sadness'], 'poke': ['anticipation'],
            'polarity': ['surprise'], 'polemic': ['anger', 'disgust'],
            'police': ['fear',  'trust'], 'policeman': ['fear',  'trust'], 'policy': ['trust'],
            'polio': ['fear',  'sadness'],
            'politic': ['disgust'],
            'politics': ['anger'], 'poll': ['trust'], 'pollute': ['disgust'],
            'pollution': ['disgust'], 'polygamy': ['disgust'],
            'pompous': ['disgust'],  'pontiff': ['trust'],
            'pop': [ 'surprise'],

            'porn': ['disgust'],
            'pornography': ['disgust'],
            'posse': ['fear'], 'possess': ['anticipation', 'joy',  'trust'],
            'possessed': ['anger', 'disgust', 'fear'],
            'possession': ['anger', 'disgust', 'fear'], 'possibility': ['anticipation'],
            'posthumous': [ 'sadness'], 'postponement': [ 'surprise'],
            'pound': ['anger'],
            'poverty': ['anger', 'disgust', 'fear',  'sadness'], 'pow': ['anger'],
            'powerful': ['anger', 'anticipation', 'disgust', 'fear', 'joy',  'trust'],
            'powerfully': ['fear'], 'powerless': ['anger', 'disgust', 'fear',  'sadness'],
            'practiced': ['joy',  'surprise', 'trust'],
            'practise': ['anticipation'], 'praise': ['joy',  'trust'],
            'praised': ['joy'], 'praiseworthy': ['anticipation', 'joy',  'surprise', 'trust'],
            'prank': [ 'surprise'],
            'pray': ['anticipation', 'fear', 'joy',  'surprise', 'trust'],
            'precarious': ['anticipation', 'fear',  'sadness'],
            'precedence': [ 'trust'],
            'precious': ['anticipation', 'joy',  'surprise'], 'precipice': ['fear'],
            'preclude': ['anger'], 'precursor': ['anticipation'],
            'predicament': ['fear'], 'predict': ['anticipation'],
            'prediction': ['anticipation'], 'predilection': ['anticipation'],
            'predispose': ['anticipation'], 'predominant': [ 'trust'],
            'prefer': [ 'trust'], 'pregnancy': ['disgust'],
            'prejudice': ['anger'], 'prejudiced': ['disgust', 'fear'],
            'prejudicial': ['anger'], 'preliminary': ['anticipation'], 'premature': ['surprise'],
            'premeditated': ['fear'],
            'preparation': ['anticipation'], 'preparatory': ['anticipation'],
            'prepare': ['anticipation'], 'prepared': ['anticipation',  'trust'],
            'preparedness': ['anticipation'], 'preponderance': ['trust'], 'prerequisite': ['anticipation'],
            'prescient': ['anticipation'],
            'present': ['anticipation', 'joy',  'surprise', 'trust'],

            'preservative': ['anticipation', 'joy',  'surprise', 'trust'],
            'president': [ 'trust'],  'prestige': ['joy',  'trust'],
            'presto': ['joy',  'surprise'], 'presumption': ['trust'],
            'presumptuous': ['anger', 'disgust'],
            'pretending': ['anger'],
            'pretty': ['anticipation', 'joy',  'trust'], 'prevail': ['anticipation', 'joy'],
            'prevalent': ['trust'], 'prevent': ['fear'], 'prevention': ['anticipation'],
            'prey': ['fear'],
            'prick': ['anger', 'disgust', 'fear',  'surprise'],
            'pride': ['joy'], 'priest': [ 'trust'],
            'priesthood': ['anticipation', 'joy',  'sadness', 'trust'],
            'primer': [ 'trust'],
            'princely': ['anticipation', 'joy',  'surprise', 'trust'],
            'principal': [ 'trust'],
            'prison': ['anger', 'fear',  'sadness'],
            'prisoner': ['anger', 'disgust', 'fear',  'sadness'],
            'privileged': ['joy',  'trust'], 'privy': [ 'trust'],
            'probability': ['anticipation'], 'probation': ['anticipation', 'fear', 'sadness'],
            'probity': [ 'trust'], 'problem': ['fear',  'sadness'],
            'procedure': ['fear'], 'proceeding': ['anticipation'],
            'procession': ['joy',  'sadness', 'surprise'],
            'proctor': [ 'trust'],

            'production': ['anticipation'],
            'profane': ['anger'],
            'profanity': ['anger'],  'professional': [ 'trust'],
            'professor': [ 'trust'], 'professorship': ['trust'],
            'proficiency': ['anticipation', 'joy',  'surprise', 'trust'],
            'proficient': [ 'trust'],
            'prognosis': ['anticipation', 'fear'], 'prognostic': ['anticipation'],
            'progress': ['anticipation', 'joy'],
            'progression': ['anticipation', 'joy',  'sadness', 'trust'],
            'prohibited': ['anger', 'disgust', 'fear'],
            'projectile': ['fear'], 'projectiles': ['fear'],  'prologue': ['anticipation'],
            'prolong': ['disgust'],
            'promise': ['joy',  'trust'],
            'proof': ['trust'],
            'prophecy': ['anticipation'], 'prophet': ['anticipation',  'trust'],
            'prophetic': ['anticipation'], 'prophylactic': ['anticipation',  'trust'],
            'prosecute': ['anger', 'fear',  'sadness'],
            'prosecution': ['disgust'],  'prospectively': ['anticipation'],
            'prosper': ['anticipation', 'joy'],
            'prosperous': ['joy'], 'prostitute': ['disgust'],
            'prostitution': ['disgust',  'sadness'],  'protected': ['trust'],
            'protecting': [ 'trust'],  'protector': [ 'trust'],
            'protestant': ['fear'], 'proud': ['anticipation', 'joy',  'trust'],
            'proven': ['trust'],  'provide': [ 'trust'],
            'providing': ['anticipation', 'joy',  'trust'], 'proviso': ['trust'],
            'provocation': ['anger'], 'provoking': ['anger', 'disgust'],
            'prowl': ['fear', 'surprise'], 'proxy': ['trust'],
            'prudent': [ 'trust'], 'pry': ['anger', 'anticipation',  'trust'],
            'psychosis': ['anger', 'fear',  'sadness'],
            'public': ['anticipation'],
            'puke': ['disgust'],   'puma': ['fear', 'surprise'],
            'punch': ['anger', 'fear',  'sadness', 'surprise'],
            'punctual': ['anticipation',  'trust'],
            'pungent': ['disgust'], 'punish': ['fear'],
            'punished': ['anger', 'anticipation', 'disgust', 'fear',  'sadness'],
            'punishing': ['anger', 'fear',  'sadness'],
            'punishment': ['anger', 'disgust', 'fear'],
            'punitive': ['anger', 'fear',  'sadness'], 'punt': ['anticipation'],
            'puppy': ['anticipation',  'trust'], 'purely': [ 'trust'],
            'purgatory': ['disgust', 'fear'], 'purge': ['fear'],
            'purification': [ 'trust'], 'purify': ['joy',  'trust'],
            'purity': [ 'surprise'], 'purr': ['joy',  'trust'], 'pus': ['disgust'],
            'putative': ['trust'], 'quack': ['disgust'], 'quagmire': ['disgust'],
            'quail': ['fear'], 'quaint': ['joy',  'trust'], 'quake': ['fear'],
            'qualified': [ 'trust'],
            'quandary': ['anger', 'fear'], 'quarantine': ['fear'], 'quarrel': ['anger'],
            'quash': ['fear'],  'quest': ['anticipation'],
            'questionable': ['disgust'], 'quicken': ['anticipation'],
            'quickness': [ 'surprise'], 'quicksilver': [ 'surprise'],
            'quiet': [ 'sadness'],
            'quiver': ['fear'], 'quivering': ['fear'],
            'quote': ['anticipation',   'surprise'],
            'rabble': ['anger', 'fear'],
            'rabid': ['anger', 'anticipation', 'disgust', 'fear',  'sadness'],
            'rack': [ 'sadness'],  'radar': ['trust'],
            'radiance': ['anticipation', 'joy',  'trust'], 'radiant': ['joy'],
            'radiation': ['fear'],
            'radioactive': ['fear'], 'radon': ['fear'],
            'raffle': ['anticipation', 'surprise'], 'rage': ['anger'],
            'raging': ['anger', 'disgust', 'fear'], 'rags': ['disgust'],
            'raid': ['anger', 'fear',  'surprise'], 'rail': ['anger', 'anticipation'],
            'rainy': ['sadness'], 'ram': ['anger', 'anticipation'],
            'rampage': ['anger', 'fear'], 'rancid': ['disgust'], 'randomly': ['surprise'],
            'ranger': ['trust'], 'ransom': ['anger', 'fear'],
            'rape': ['anger', 'disgust', 'fear',  'sadness'], 'rapid': ['surprise'], 'rapping': ['anger'],
            'rapt': ['joy',  'surprise', 'trust'], 'raptors': ['fear'],
            'rapture': ['anticipation', 'joy'], 'rarity': ['surprise'],
            'rascal': ['anger', 'disgust', 'fear'], 'rash': ['disgust'],
            'rat': ['disgust', 'fear'], 'ratify': ['trust'],
            'rating': ['anger', 'fear',  'sadness'], 'rational': [ 'trust'],
            'rattlesnake': ['fear'],
            'rave': ['anger', 'disgust', 'joy',   'surprise', 'trust'],
            'ravenous': ['anger', 'fear',  'sadness'], 'ravine': ['fear'],
            'raving': ['anger', 'anticipation', 'fear', 'joy',  'surprise'], 'razor': ['fear'],
            'react': ['anger', 'fear'],
            'readiness': ['anticipation', 'joy',  'trust'],
            'ready': ['anticipation'],  'real': [ 'trust'],
            'reappear': ['surprise'],
            'reassurance': [ 'trust'], 'reassure': [ 'trust'],
            'rebel': ['anger', 'fear'], 'rebellion': ['anger', 'disgust', 'fear'],
            'recalcitrant': ['anger', 'disgust'],

            'receiving': ['anticipation', 'joy',  'surprise'], 'recesses': ['fear'],
            'recession': ['anger', 'disgust', 'fear',  'sadness'],
            'recidivism': ['anger', 'disgust',  'sadness'], 'recipient': ['anticipation'],
            'reckless': ['anger', 'fear'],
            'recklessness': ['anger', 'disgust', 'fear',  'surprise'],
            'recline': [ 'trust'], 'recognizable': ['anticipation'],
            'recombination': ['anticipation'], 'recommend': [ 'trust'],
            'reconciliation': ['anticipation', 'joy',  'trust'], 'reconsideration': [ 'trust'],
            'reconstruct': ['anticipation'], 'reconstruction': ['anticipation'],

            'recreation': ['anticipation', 'joy'], 'recreational': ['anticipation', 'joy'],
            'recruits': ['trust'],  'recurrent': ['anticipation'],
            'referee': ['trust'],
            'reflex': ['surprise'],
            'reflux': ['disgust'],
            'refugee': ['sadness'],

            'refused': [ 'sadness'], 'refutation': ['fear'], 'regal': [ 'trust'],
            'regatta': ['anticipation'], 'regent': [ 'trust'], 'regiment': ['fear'],
            'registry': ['trust'],
            'regret': [ 'sadness'],
            'regrettable': [ 'sadness'], 'regretted': [ 'sadness'],
            'regretting': [ 'sadness'], 'regularity': ['anticipation',  'trust'],
            'regulatory': ['fear'], 'regurgitation': ['disgust'],
            'rehabilitation': ['anticipation'],
            'reimbursement': [ 'trust'],  'reinforcement': [ 'trust'],
            'reinforcements': ['trust'],
            'reject': ['anger', 'fear',  'sadness'],
            'rejection': ['anger', 'disgust', 'fear',  'sadness'],
            'rejects': ['anger', 'fear',  'sadness'],
            'rejoice': ['anticipation', 'joy',  'surprise', 'trust'],
            'rejoicing': ['anticipation', 'joy',  'surprise'],

            'rekindle': ['anticipation', 'fear', 'joy',   'surprise'],
            'relapse': ['fear',  'sadness'], 'related': ['trust'], 'relative': ['trust'],
            'relevant': [ 'trust'],
            'reliability': [ 'trust'], 'reliable': [ 'trust'],
            'reliance': [ 'trust'], 'relics': ['sadness'],
            'religion': ['trust'],
            'reluctant': ['fear'], 'remains': ['disgust', 'fear',   'trust'],
            'remand': ['anger'],
            'remarkable': ['joy',  'surprise', 'trust'],
            'remedy': ['anticipation', 'joy',  'trust'],
            'remiss': ['anger', 'disgust',  'sadness'],
            'remorse': [ 'sadness'],
            'remove': ['anger', 'fear',  'sadness'],

            'renegade': ['anger'],  'renounce': ['anger'],
            'renovate': ['anticipation'], 'renovation': ['anticipation', 'joy'],

            'reparation': [ 'trust'],
            'repay': ['anger', 'anticipation', 'joy',  'trust'],
            'repellant': ['disgust', 'fear'], 'repellent': ['anger', 'disgust', 'fear'],
            'repelling': ['disgust'], 'repent': ['fear'],
            'reporter': [ 'trust'],
            'representing': ['anticipation'], 'repress': [ 'sadness'],
            'repression': ['fear'], 'reprimand': ['anger'],
            'reprisal': ['anger', 'fear',  'sadness'],
            'reproach': ['anger', 'disgust',  'sadness'], 'reproductive': ['joy'],
            'repudiation': ['anger', 'disgust'],
            'repulsion': ['disgust', 'fear'], 'reputable': [ 'trust'], 'requiem': ['sadness'],

            'rescue': ['anticipation', 'joy',  'surprise', 'trust'], 'resection': ['fear'],
            'resent': ['anger'], 'resentful': ['anger'],
            'resentment': ['anger', 'disgust',  'sadness'],
            'resign': ['anger', 'disgust', 'fear',  'sadness'],
            'resignation': [ 'sadness', 'surprise'], 'resigned': [ 'sadness'],
            'resistance': ['anger'],
            'resistant': ['fear'], 'resisting': ['anger', 'fear',  'sadness'],
            'resources': ['joy',  'trust'],
            'respect': ['anticipation', 'joy',  'trust'],
            'respectable': [ 'trust'], 'respectful': [ 'trust'],
            'respects': [ 'trust'], 'respite': ['joy',  'trust'],
            'resplendent': ['joy'], 'responsible': [ 'trust'],
            'responsive': ['anticipation',  'trust'],
            'restitution': ['anger'], 'restlessness': ['anticipation'],
            'restorative': ['anticipation', 'joy',  'trust'],
            'restrain': ['anger', 'fear'], 'restrained': ['fear'],
            'restrict': [ 'sadness'], 'restriction': ['anger', 'fear',  'sadness'],
            'result': ['anticipation'], 'resultant': ['anticipation'],
            'retain': ['trust'], 'retaliate': ['anger'],
            'retaliation': ['anger', 'fear'], 'retaliatory': ['anger'],
            'retard': ['disgust', 'fear',  'sadness'],
            'reticent': ['fear'],
            'retirement': ['anticipation', 'fear', 'joy',   'sadness', 'trust'],
            'retract': ['anger'],
            'retrenchment': ['fear'], 'retribution': ['anger', 'fear',  'sadness'],
            'reunion': ['anticipation',  'trust'],
            'revel': ['joy'], 'revels': ['joy'],
            'revenge': ['anger', 'anticipation', 'fear',  'surprise'],
            'revere': ['anticipation', 'joy',  'trust'], 'reverence': ['joy',  'trust'],
            'reverend': ['joy'], 'reverie': ['joy',  'trust'],
            'reversal': ['anger', 'disgust',  'surprise'],
            'revival': ['anticipation', 'joy',  'trust'],
            'revive': ['anticipation'],
            'revoke': ['anger', 'disgust', 'fear',  'sadness'],
            'revolt': ['anger',  'surprise'], 'revolting': ['anger', 'disgust', 'fear'],
            'revolution': ['anger', 'anticipation', 'fear',   'sadness', 'surprise'],
            'revolver': ['anger', 'fear',  'sadness'],
            'revulsion': ['anger', 'disgust', 'fear'],
            'reward': ['anticipation', 'joy',  'surprise', 'trust'],
            'rheumatism': ['anger', 'fear',  'sadness'],
            'rhythmical': ['joy',  'surprise'], 'ribbon': ['anger', 'anticipation', 'joy'],
            'riddle': ['surprise'],
            'ridicule': ['anger', 'disgust',  'sadness'],
            'ridiculous': ['anger', 'disgust'],
            'rifle': ['anger', 'fear'],

            'rigor': ['disgust', 'fear'],  'ringer': ['anger'],
            'riot': ['anger', 'fear'], 'riotous': ['anger', 'fear',  'surprise'],
            'ripen': ['anticipation'],
            'rising': ['anticipation', 'joy'], 'risk': ['anticipation', 'fear'],
            'risky': ['anticipation', 'fear'], 'rivalry': ['anger'],
            'roadster': ['joy',  'trust'],
            'rob': ['anger', 'disgust', 'fear',  'sadness'], 'robber': ['disgust', 'fear'],
            'robbery': ['anger', 'disgust', 'fear',  'sadness'],
            'rocket': ['anger'], 'rod': ['fear',  'trust'],
            'rogue': ['disgust'], 'rollicking': ['joy'],
            'romance': ['anticipation', 'fear', 'joy',  'sadness', 'surprise', 'trust'],
            'romantic': ['anticipation', 'joy',  'trust'], 'romanticism': ['joy'],
            'romp': ['joy'], 'rook': ['anger', 'disgust'], 'rooted': [ 'trust'],
            'rot': ['disgust', 'fear',  'sadness'], 'rota': [ 'trust'],
            'rotting': ['disgust'],  'roulette': ['anticipation'],
            'routine': [ 'trust'], 'row': ['anger'],
            'rubbish': ['disgust'],
            'rubble': ['fear',  'sadness'],  'rue': [ 'sadness'],
            'ruin': ['fear',  'sadness'],
            'ruined': ['anger', 'disgust', 'fear',  'sadness'],
            'ruinous': ['anger', 'disgust', 'fear',  'sadness'], 'rule': ['fear', 'trust'],
            'rumor': [ 'sadness'], 'runaway': [ 'sadness'],
            'rupture': ['fear',  'sadness', 'surprise'],
            'ruthless': ['anger', 'disgust', 'fear'],
            'saber': ['anger', 'fear'],
            'sabotage': ['anger', 'disgust', 'fear',  'sadness', 'surprise'],
            'sacrifices': ['disgust', 'fear',  'sadness'], 'sadly': [ 'sadness'],
            'sadness': [ 'sadness', 'trust'], 'safe': ['joy',  'trust'],
            'safeguard': [ 'trust'], 'safekeeping': ['trust'], 'sag': ['fear'],
            'sage': [ 'trust'], 'saint': ['anticipation', 'joy',  'surprise', 'trust'],
            'saintly': ['anticipation', 'joy',  'surprise', 'trust'],
            'salary': ['anticipation', 'joy',  'trust'],
            'saliva': ['anticipation'], 'sally': ['surprise'], 'saloon': ['anger'],
            'salutary': ['joy',  'trust'], 'salute': ['joy'],
            'salvation': ['anticipation', 'joy',  'trust'],
            'samurai': ['fear'], 'sanctification': ['joy',  'trust'],
            'sanctify': ['anticipation', 'joy',  'sadness', 'surprise', 'trust'],
            'sanctuary': ['anticipation', 'joy',  'trust'],
            'sap': [ 'sadness'], 'sappy': ['trust'],
            'sarcasm': ['anger', 'disgust',  'sadness'], 'sarcoma': ['fear',  'sadness'],
            'satanic': ['anger'],
            'satisfied': ['joy'], 'saturated': ['disgust'],
            'savage': ['anger', 'fear'], 'savagery': ['anger', 'fear'],
            'save': ['joy',  'trust'],
            'savor': ['anticipation', 'disgust', 'joy',  'sadness', 'trust'],
            'scaffold': ['fear'],
            'scalpel': ['fear'],  'scandal': ['fear'],
            'scandalous': ['anger'],  'scapegoat': ['anger', 'fear'],
            'scar': ['anger', 'disgust', 'fear',  'sadness'], 'scarce': ['fear',  'sadness'],
            'scarcely': [ 'sadness'], 'scarcity': ['anger', 'fear',  'sadness'],
            'scare': ['anger', 'anticipation', 'fear',  'surprise'],
            'scarecrow': ['fear'],  'sceptical': ['trust'],
            'schism': ['anger'],
            'schizophrenia': ['anger', 'disgust', 'fear',  'sadness'],
            'scholarship': ['joy'], 'school': ['trust'],
            'scientific': [ 'trust'], 'scientist': ['anticipation',  'trust'],
            'scoff': ['anger', 'disgust'],
            'scold': ['anger', 'disgust', 'fear',  'sadness'], 'scolding': ['anger'],
            'scorching': ['anger'], 'score': ['anticipation', 'joy',  'surprise'],
            'scorn': ['anger'], 'scorpion': ['anger', 'disgust', 'fear',  'surprise'],
            'scoundrel': ['anger', 'disgust', 'fear',  'trust'],
            'scourge': ['anger', 'fear',  'sadness'],
            'scrapie': ['anger', 'fear',  'sadness'],
            'scream': ['anger', 'disgust', 'fear',  'surprise'],
            'screaming': ['anger', 'disgust', 'fear'], 'screech': ['fear',  'surprise'],
            'screwed': ['anger'],  'scrimmage': [ 'surprise'],
            'scripture': ['trust'], 'scrub': ['disgust'],
            'scrutinize': ['anticipation'],
            'scum': ['disgust'],
            'seal': [ 'trust'], 'seals': ['trust'],
            'secluded': [ 'sadness'],
            'seclusion': ['fear'],
            'secrecy': ['surprise', 'trust'], 'secret': ['trust'],
            'secrete': ['disgust'], 'secretion': ['disgust'],
            'sectarian': ['anger', 'fear'], 'secular': ['anticipation'], 'securities': ['trust'],
            'sedition': ['anger',  'sadness'],
            'seductive': ['anticipation'], 'seek': ['anticipation'],
            'segregate': ['anger', 'disgust',  'sadness'],
            'seize': ['fear'], 'seizure': ['fear'], 'selfish': ['anger', 'disgust'],
            'senate': ['trust'], 'senile': ['fear',  'sadness'],
            'seniority': [ 'trust'], 'sensational': ['joy'],
            'senseless': ['anger', 'disgust', 'fear',  'sadness', 'surprise'],

            'sensual': ['anticipation', 'joy',   'surprise', 'trust'],
            'sensuality': ['anticipation', 'joy'], 'sensuous': ['joy'],
            'sentence': ['anger', 'anticipation', 'disgust', 'fear',  'sadness'],
            'sentinel': [ 'trust'],
            'sentry': ['trust'], 'separatist': ['anger', 'disgust'],
            'sepsis': ['fear',  'sadness'], 'septic': ['disgust'], 'sequel': ['anticipation'],
            'sequestration': [ 'sadness'], 'serene': [ 'trust'],
            'serenity': ['anticipation', 'joy',  'trust'], 'serial': ['anticipation'],
            'series': ['trust'], 'seriousness': ['fear', 'sadness'], 'sermon': [ 'trust'],
            'serpent': ['disgust', 'fear'],  'servant': [ 'trust'],
            'serve': [ 'trust'], 'servile': ['disgust', 'fear',  'sadness'],
            'setback': [ 'sadness'], 'settlor': ['fear'],
            'severance': ['sadness'], 'sewage': ['disgust'], 'sewer': ['disgust'],
            'sewerage': ['disgust'], 'sex': ['anticipation', 'joy',  'trust'],
            'shabby': ['disgust'], 'shack': ['disgust',  'sadness'],
            'shackle': ['anger', 'anticipation', 'disgust', 'fear',  'sadness'],
            'shady': ['fear'], 'shaking': ['fear'],
            'shaky': ['anger', 'anticipation', 'fear'], 'sham': ['anger', 'disgust'],
            'shame': ['disgust', 'fear',  'sadness'],
            'shameful': [ 'sadness'], 'shameless': ['disgust'],
            'shanghai': ['disgust', 'fear'], 'shank': ['fear'],
            'share': ['anticipation', 'joy',  'trust'],
            'sharpen': ['anger', 'anticipation'], 'shatter': ['anger', 'fear',  'sadness', 'surprise'],
            'shattered': [ 'sadness'],
            'shell': ['anger', 'fear',  'sadness', 'surprise'], 'shelter': [ 'trust'],
            'shepherd': [ 'trust'], 'sheriff': ['trust'],
            'shining': ['anticipation', 'joy'], 'ship': ['anticipation'],
            'shipwreck': ['fear',  'sadness'], 'shit': ['anger', 'disgust'],
            'shiver': ['anger', 'anticipation', 'fear',  'sadness'],
            'shock': ['anger', 'fear',  'surprise'], 'shockingly': ['surprise'],
            'shoddy': ['anger', 'disgust'], 'shoot': ['anger', 'fear'], 'shooter': ['fear'],
            'shooting': ['anger', 'fear'], 'shopkeeper': ['trust'],
            'shoplifting': ['anger', 'disgust'],
            'shopping': ['anticipation', 'joy',  'surprise', 'trust'],
            'shortage': ['anger', 'fear',  'sadness'],
            'shortly': ['anticipation'], 'shot': ['anger', 'fear',  'sadness', 'surprise'],
            'shoulder': [ 'trust'], 'shout': ['anger', 'surprise'], 'shove': ['anger'],
            'show': ['trust'],  'shrapnel': ['fear'],
            'shriek': ['anger', 'fear',  'sadness', 'surprise'],
            'shrill': ['anger', 'fear',  'surprise'], 'shrink': ['fear',  'sadness'],
            'shroud': ['sadness'],  'shudder': ['fear'],
            'shun': ['anger', 'disgust',  'sadness'], 'sib': ['trust'],
            'sick': ['disgust',  'sadness'],
            'sickening': ['anger', 'disgust', 'fear',  'sadness'],
            'sickly': ['disgust',  'sadness'], 'sickness': ['disgust', 'fear',  'sadness'],
            'signature': ['trust'], 'signify': ['anticipation'],  'silly': ['joy'],
            'simmer': ['anger', 'anticipation'], 'simmering': ['anticipation'],
            'simplify': ['anticipation', 'joy',  'surprise', 'trust'],
            'sin': ['anger', 'disgust', 'fear',  'sadness'], 'sincere': [ 'trust'],
            'sinful': ['anger', 'disgust', 'fear',  'sadness'],
            'sing': ['anticipation', 'joy',  'sadness', 'trust'],
            'singularly': ['surprise'], 'sinister': ['anger', 'disgust', 'fear'],
            'sinner': ['anger', 'disgust', 'fear',  'sadness'], 'sinning': ['disgust'],
            'sir': [ 'trust'], 'siren': ['fear'],
            'sisterhood': ['anger',  'sadness', 'surprise', 'trust'], 'sizzle': ['anger'],
            'skewed': ['anger', 'anticipation'],
            'skid': ['anger', 'fear',  'sadness'],
            'skillful': [ 'trust'],  'skirmish': ['anger'],

            'slam': ['anger', 'fear',  'surprise'], 'slander': ['anger', 'disgust'],
            'slap': ['anger',  'surprise'], 'slash': ['anger'],
            'slaughter': ['anger', 'disgust', 'fear',  'sadness', 'surprise'],
            'slaughterhouse': ['anger', 'disgust', 'fear',  'sadness'],
            'slaughtering': ['anger', 'disgust', 'fear',  'sadness', 'surprise'],
            'slave': ['anger', 'fear',  'sadness'],
            'slavery': ['anger', 'disgust', 'fear',  'sadness'], 'slay': ['anger'],
            'slayer': ['anger', 'disgust', 'fear',  'sadness', 'surprise'],
            'slime': ['disgust'],
            'slimy': ['disgust'],  'slip': [ 'surprise'],
            'slop': ['disgust'], 'sloppy': ['disgust'], 'sloth': ['disgust'],

            'sludge': ['disgust'], 'slug': ['disgust'], 'sluggish': [ 'sadness'],
            'slum': ['disgust'], 'slump': [ 'sadness'],
            'slur': ['anger', 'disgust',  'sadness'], 'slush': ['disgust',  'surprise'],
            'slut': ['anger', 'disgust'], 'sly': ['anger', 'disgust', 'fear'],
            'smack': ['anger'],  'smash': ['anger', 'fear'],
            'smell': ['anger', 'disgust'],
            'smelling': ['disgust'], 'smile': ['joy',  'surprise', 'trust'],
            'smiling': ['joy'],  'smite': ['anger', 'fear',  'sadness'],
            'smith': ['trust'],
            'smother': ['anger'],
            'smuggle': ['fear'], 'smuggler': ['anger', 'disgust', 'fear'],
            'smut': ['disgust', 'fear'], 'snag': [ 'surprise'],
            'snake': ['disgust', 'fear'], 'snare': ['fear'],
            'snarl': ['anger', 'disgust'], 'snarling': ['anger'],
            'sneak': ['anger', 'fear',  'surprise'],
            'sneaking': ['anticipation', 'fear',  'trust'], 'sneer': ['anger', 'disgust'],
            'sneeze': ['disgust',  'surprise'],
            'snort': ['sadness'],  'sob': [ 'sadness'],
            'sobriety': [ 'trust'],  'socialism': ['disgust', 'fear'],
            'socialist': ['anger', 'disgust', 'fear',  'sadness'], 'soil': ['disgust'],
            'soiled': ['disgust'],  'soldier': ['anger',  'sadness'],
            'solidarity': ['trust'], 'solidity': [ 'trust'],
            'somatic': [ 'surprise'],
            'somber': [ 'sadness'], 'sonar': ['anticipation'],
            'sonnet': ['joy',  'sadness'], 'sonorous': ['joy'],
            'soot': ['disgust'],  'soothing': ['joy',  'trust'],
            'sorcery': ['anticipation', 'fear',  'surprise'],
            'sordid': ['anger', 'disgust', 'fear',  'sadness'], 'sore': ['anger',  'sadness'],
            'sorely': [ 'sadness'], 'soreness': ['disgust',  'sadness'],
            'sorrow': ['fear',  'sadness'], 'sorrowful': [ 'sadness'],
            'sortie': ['fear'], 'soulless': ['disgust', 'fear',  'sadness'],
            'soulmate': ['fear'], 'soundness': ['anticipation', 'joy',  'trust'],
            'sour': ['disgust'], 'sovereign': ['trust'],
            'spa': ['anticipation', 'joy',  'surprise', 'trust'],
            'spaniel': ['joy',  'trust'], 'spank': ['anger', 'fear',  'sadness'],
            'spanking': ['anger'], 'sparkle': ['anticipation', 'joy',  'surprise'],
            'spasm': ['fear'], 'spat': ['anger'],
            'spear': ['anger', 'anticipation', 'fear'], 'special': ['joy'],
            'specialist': ['trust'], 'specialize': ['trust'],
            'speck': ['disgust'],
            'spectacular': ['anticipation', 'surprise'], 'specter': ['fear',  'sadness'],
            'speculation': ['fear',  'sadness'],
            'speculative': ['anticipation'],
            'spew': ['disgust'],
            'spider': ['disgust', 'fear'], 'spike': ['fear'], 'spine': ['anger'],
            'spinster': ['fear',  'sadness'],
            'spirits': ['anticipation', 'joy',  'surprise'], 'spit': ['disgust'],
            'spite': ['anger'], 'spiteful': ['anger'], 'splash': ['surprise'],
            'splendid': ['joy',  'surprise'], 'splendor': ['anticipation', 'joy',  'surprise'],
            'splitting': [ 'sadness'],
            'spoil': ['disgust'], 'spoiler': [ 'sadness'],
            'spokesman': ['trust'],  'sponsor': [ 'trust'],
            'spook': ['fear'], 'spotless': [ 'trust'], 'spouse': ['joy',  'trust'],
            'sprain': [ 'sadness'],  'sprite': ['fear'],
            'spur': ['fear'], 'spurious': ['disgust'],
            'squall': ['fear',  'sadness'],
            'squeamish': ['disgust', 'fear'], 'squelch': ['anger', 'disgust'],
            'squirm': ['disgust'], 'stab': ['anger', 'fear',  'sadness', 'surprise'],
            'stable': [ 'trust'],  'stagger': ['surprise'],
            'stagnant': [ 'sadness'], 'stain': ['disgust'],
            'stalemate': ['anger', 'disgust'],
            'stalk': ['fear'], 'stall': ['disgust'],
            'stamina': [ 'trust'],  'standoff': ['anger', 'fear'],
            'standstill': ['anger'], 'star': ['anticipation', 'joy',  'trust'],
            'stark': [ 'trust'],
            'starry': ['anticipation', 'joy'], 'start': ['anticipation'],
            'startle': ['fear',  'surprise'], 'startling': ['surprise'],
            'starvation': ['fear',  'sadness'],
            'statement': [ 'trust'],
            'statistical': ['trust'],
            'steadfast': [ 'trust'], 'steady': ['surprise', 'trust'],
            'steal': ['anger', 'fear',  'sadness'], 'stealing': ['disgust', 'fear'],
            'stealth': ['surprise'], 'stealthily': ['surprise'],
            'stealthy': ['anticipation', 'fear',  'surprise'],
            'sterile': [ 'sadness'],
            'sterling': ['anger', 'anticipation', 'joy',   'trust'],
            'steward': [ 'trust'], 'sticky': ['disgust'],
            'stifled': ['anger', 'fear',  'sadness'],
            'stigma': ['anger', 'disgust', 'fear',  'sadness'], 'stillborn': [ 'sadness'],
            'stillness': ['fear',  'sadness'], 'sting': ['anger', 'fear'],
            'stingy': ['anger', 'disgust', 'fear',  'sadness'],
            'stink': ['disgust'], 'stinking': ['disgust'],
            'stint': ['fear',  'sadness'],  'stolen': ['anger'],
            'stomach': ['disgust'], 'stone': ['anger'],
            'stools': ['disgust'],  'store': ['anticipation'],
            'storm': ['anger'], 'storming': ['anger'], 'stormy': ['fear'],
            'straightforward': [ 'trust'], 'strained': ['anger'],
            'straits': ['fear'],  'stranger': ['fear'],
            'strangle': ['anger', 'disgust', 'fear',  'sadness', 'surprise'],
            'strategist': ['anticipation',  'trust'],
            'strength': [ 'trust'],
            'strengthening': ['joy',  'trust'],  'stretcher': ['fear', 'sadness'],
            'stricken': ['sadness'], 'strife': ['anger'], 'strike': ['anger'],
            'strip': [ 'sadness'],
            'stripped': ['anger', 'anticipation', 'disgust', 'fear',  'sadness'],
            'strive': ['anticipation'], 'stroke': ['fear',  'sadness'],
            'structural': ['trust'], 'structure': [ 'trust'],
            'struggle': ['anger', 'fear',  'sadness'],

            'stunned': ['fear',  'surprise'],

            'sty': ['disgust'],  'subito': ['surprise'],
            'subjected': [ 'sadness'],
            'subjugation': ['anger', 'disgust', 'fear',  'sadness'], 'sublimation': ['joy'],
            'submit': ['anticipation'], 'subordinate': ['fear'],
            'subscribe': ['anticipation'], 'subsidence': [ 'sadness'],
            'subsidy': ['anger', 'disgust'],
            'substantiate': ['trust'],
            'subversion': ['anger', 'fear'], 'subversive': ['anger',  'surprise'],
            'subvert': ['disgust', 'fear',  'sadness'],
            'succeed': ['anticipation', 'joy',  'surprise', 'trust'],
            'succeeding': ['anticipation', 'joy',  'trust'],
            'success': ['anticipation', 'joy'],
            'successful': ['anticipation', 'joy',  'trust'],

            'sucker': ['anger'], 'sudden': ['surprise'], 'suddenly': ['surprise'],
            'sue': ['anger',  'sadness'],
            'sufferer': ['fear',  'sadness'], 'suffering': ['disgust', 'fear',  'sadness'],
            'suffocating': ['disgust', 'fear',  'sadness'],
            'suffocation': ['anger', 'fear'],  'suggest': ['trust'],
            'suicidal': ['anger', 'disgust', 'fear',  'sadness'],
            'suicide': ['anger', 'fear',  'sadness'],
            'sullen': ['anger',  'sadness'], 'sultan': ['fear'],
            'sump': ['disgust'],
            'sun': ['anticipation', 'joy',  'surprise', 'trust'], 'sundial': ['anticipation', 'trust'],
            'sunk': ['disgust', 'fear',  'sadness'],
            'sunny': ['anticipation', 'joy',  'surprise'], 'sunset': ['anticipation'],
            'sunshine': ['joy'],

            'superman': ['joy',  'trust'],
            'superstar': ['joy',  'trust'], 'superstition': ['fear'],
            'superstitious': ['anticipation', 'fear'], 'supplication': [ 'trust'],

            'supporter': ['joy',  'trust'], 'supporting': [ 'trust'],
            'suppress': ['anger', 'fear',  'sadness'],
            'suppression': ['anger', 'disgust', 'fear'],
            'supremacy': ['anger', 'anticipation', 'fear', 'joy',   'surprise', 'trust'],
            'surcharge': ['anger'],
            'surety': [ 'trust'], 'surge': ['surprise'], 'surgery': ['fear', 'sadness'],
            'surly': ['anger', 'disgust'],
            'surprise': ['fear', 'joy',  'surprise'], 'surprised': ['surprise'],
            'surprising': ['surprise'], 'surprisingly': ['anticipation', 'surprise'],
            'surrender': ['fear',  'sadness'], 'surrendering': [ 'sadness'],
            'surrogate': ['trust'], 'surround': ['anticipation'], 'surveillance': ['fear'],

            'suspect': ['fear'], 'suspense': ['anticipation', 'fear', 'surprise'],
            'suspension': ['fear'], 'suspicion': ['fear'],
            'suspicious': ['anger', 'anticipation'],
            'swamp': ['disgust', 'fear'], 'swampy': ['disgust', 'fear'],
            'swarm': ['disgust'], 'swastika': ['anger', 'fear'], 'swear': [ 'trust'],
            'sweat': ['fear'], 'sweet': ['anticipation', 'joy',  'surprise', 'trust'],
            'sweetheart': ['anticipation', 'joy',  'sadness', 'trust'],
            'sweets': ['anticipation', 'joy'],
            'swelling': ['fear'], 'swerve': ['fear', 'surprise'],
            'swig': ['disgust'], 'swim': ['anticipation', 'fear', 'joy'],
            'swine': ['disgust'],
            'symmetry': ['joy',  'trust'],
            'sympathetic': ['fear', 'joy',  'sadness', 'trust'], 'sympathize': ['sadness'],
            'sympathy': [ 'sadness'], 'symphony': ['anticipation', 'joy'],
            'synchronize': ['anticipation', 'joy',  'surprise', 'trust'],
            'syncope': ['fear',  'sadness', 'surprise'], 'synergistic': [ 'trust'],
            'synod': [ 'trust'], 'synonymous': ['fear',   'trust'],
            'syringe': ['fear'], 'system': ['trust'], 'taboo': ['disgust', 'fear'],
            'tabulate': ['anticipation'], 'tackle': ['anger', 'surprise'],
            'tactics': ['fear', 'trust'], 'taint': [ 'sadness'],

            'talons': ['anger', 'fear'], 'tandem': ['trust'],
            'tantalizing': ['anticipation', 'joy',   'surprise'],
            'tantamount': ['trust'],
            'tariff': ['anger', 'disgust'], 'tarnish': ['disgust',  'sadness'],

            'tasteless': ['disgust'],  'taught': ['trust'],
            'taunt': ['anger', 'fear',  'sadness'], 'tawny': ['disgust'], 'tax': [ 'sadness'],
            'teach': ['joy',  'surprise', 'trust'], 'teacher': [ 'trust'], 'team': ['trust'],
            'tearful': ['disgust', 'fear', 'sadness'], 'tease': ['anger', 'anticipation',  'sadness'],
            'teasing': ['anger', 'fear'],
            'teeming': ['disgust'],
            'temperate': ['trust'],
            'tempest': ['anger', 'anticipation', 'fear',  'sadness', 'surprise'],

            'tender': ['joy',  'trust'],
            'tenderness': ['joy'],  'tension': ['anger'],
            'terminal': ['fear',  'sadness'], 'terminate': ['sadness'],
            'termination': [ 'sadness'], 'termite': ['disgust'],
            'terrible': ['anger', 'disgust', 'fear',  'sadness'], 'terribly': ['sadness'],
            'terrific': ['sadness'], 'terror': ['fear'],
            'terrorism': ['anger', 'disgust', 'fear',  'sadness'],
            'terrorist': ['anger', 'disgust', 'fear',  'sadness', 'surprise'],
            'terrorize': ['anger', 'fear',  'sadness'], 'testament': ['anticipation', 'trust'],
            'testimony': ['trust'], 'tetanus': ['disgust'],
            'thankful': ['joy'], 'thanksgiving': ['joy'],
            'theft': ['anger', 'disgust', 'fear',  'sadness'], 'theism': ['disgust'],
            'theocratic': ['anger', 'fear',  'sadness', 'trust'], 'theological': ['trust'],
            'theology': ['anticipation'], 'theorem': ['trust'],
            'theory': ['anticipation', 'trust'], 'therapeutic': ['joy',  'trust'],
            'thermocouple': ['anticipation'], 'thermometer': ['trust'],
            'thief': ['anger', 'disgust', 'fear',  'sadness', 'surprise'],
            'thirst': ['anticipation', 'sadness', 'surprise'],  'thirteenth': ['fear'],
            'thorny': ['fear'],
            'thought': ['anticipation'], 'thoughtful': [ 'trust'],
            'thoughtless': ['anger', 'disgust'],
            'thrash': ['anger', 'disgust', 'fear',  'sadness'], 'threat': ['anger', 'fear'],
            'threaten': ['anger', 'anticipation', 'fear'],
            'threatening': ['anger', 'disgust', 'fear'],
            'thresh': ['anger', 'fear',  'sadness'], 'thrift': ['disgust',  'trust'],
            'thrill': ['anticipation', 'fear', 'joy',  'surprise'],
            'thrilling': ['anticipation', 'joy',  'surprise'],
            'thriving': ['anticipation', 'joy'], 'throb': ['fear',  'sadness'],
            'throne': [ 'trust'], 'throttle': ['anger'],
            'thug': ['anger', 'disgust', 'fear'], 'thump': ['anger'], 'thumping': ['fear'],
            'thundering': ['anger', 'fear'], 'thwart': [ 'surprise'],
            'tickle': ['anticipation', 'joy',  'surprise', 'trust'], 'tiff': ['anger'],
            'tighten': ['anger'],  'time': ['anticipation'],
            'timid': ['fear',  'sadness'], 'timidity': ['anticipation', 'fear'],
            'tinsel': ['joy'],  'tirade': ['anger', 'disgust'],

            'title': [ 'trust'], 'toad': ['disgust'], 'toast': ['joy'],
            'toilet': ['disgust'],
            'tolerate': ['anger',  'sadness'],
            'tomb': ['sadness'], 'tomorrow': ['anticipation'], 'toothache': ['fear'],
            'top': ['anticipation',  'trust'], 'topple': ['surprise'],
            'torment': ['anger', 'fear',  'sadness'],  'tornado': ['fear'],
            'torpedo': ['anger'], 'torrent': ['fear'],
            'tortious': ['anger', 'disgust'],
            'torture': ['anger', 'anticipation', 'disgust', 'fear',  'sadness'],
            'touchy': ['anger',  'sadness'], 'tough': [ 'sadness'],
            'toughness': ['anger', 'fear',  'trust'],
            'towering': ['anticipation', 'fear'], 'toxic': ['disgust'],
            'toxin': ['fear'], 'track': ['anticipation'], 'tract': ['fear'], 'trade': ['trust'],
            'tragedy': ['fear',  'sadness'],
            'trainer': ['trust'], 'traitor': ['anger', 'disgust', 'fear',  'sadness'],
            'tramp': ['disgust', 'fear',  'sadness'],
            'tranquil': ['joy'], 'tranquility': ['joy',  'trust'], 'transaction': ['trust'],
            'transcendence': ['anticipation', 'joy',  'surprise', 'trust'],
            'transcript': ['trust'],
            'transitional': ['anticipation'], 'translation': ['trust'],
            'trash': ['disgust',  'sadness'], 'trashy': ['disgust'],
            'traumatic': ['anger', 'fear',  'sadness'],
            'travesty': ['disgust', 'fear',  'sadness'],
            'treacherous': ['anger', 'disgust', 'fear'],
            'treachery': ['anger', 'fear',  'sadness', 'surprise'], 'treadmill': ['anticipation'],
            'treason': ['anger', 'disgust', 'fear',  'surprise'],
            'treasure': ['anticipation', 'joy',  'trust'], 'treasurer': ['trust'],
            'treat': ['anger', 'anticipation', 'disgust', 'fear', 'joy',   'sadness',
            'surprise', 'trust'],
            'tree': ['anger', 'anticipation', 'disgust', 'joy',  'surprise', 'trust'],
            'trembling': ['fear'],
            'tremor': ['anger', 'anticipation', 'fear',  'sadness'],
            'trepidation': ['anticipation', 'fear',  'surprise'],
            'trespass': ['anger'], 'tribe': ['trust'], 'tribulation': ['fear',  'sadness'],
            'tribunal': ['anticipation', 'disgust', 'fear',  'trust'], 'tribune': ['trust'],
            'tributary': ['anticipation'],  'trick': [ 'surprise'],
            'trickery': ['anger', 'disgust', 'fear',  'sadness', 'surprise'],
            'trip': ['surprise'], 'tripping': ['anger',  'sadness'],
            'triumph': ['anticipation', 'joy'],
            'triumphant': ['anticipation', 'joy',  'trust'], 'troll': ['anger', 'fear'],
            'trophy': ['anticipation', 'joy',  'surprise', 'trust'],
            'troublesome': ['anger', 'fear'], 'truce': ['joy',  'trust'], 'truck': ['trust'],
            'true': ['joy',  'trust'], 'trump': ['surprise'],  'truss': ['trust'],
            'trust': ['trust'], 'trustee': ['trust'],  'truth': [ 'trust'],
            'truthful': ['trust'], 'truthfulness': [ 'trust'],
            'tumor': ['fear'], 'tumour': ['fear',  'sadness'],
            'tumult': ['anger', 'fear',  'surprise'], 'tumultuous': ['anger', 'fear'],
            'turbulence': ['anger', 'fear'], 'turbulent': ['fear'],
            'turmoil': ['anger', 'fear',  'sadness'], 'tussle': ['anger'],
            'tutelage': [ 'trust'],
            'twinkle': ['anticipation', 'joy'],  'typhoon': ['fear'],
            'tyrannical': ['anger', 'disgust', 'fear'], 'tyranny': ['fear',  'sadness'],
            'tyrant': ['anger', 'disgust', 'fear',  'sadness'],
            'ugliness': ['disgust', 'fear',  'sadness'], 'ugly': ['disgust'],
            'ulcer': ['anger', 'disgust', 'fear',  'sadness'],
            'ultimate': ['anticipation', 'sadness'], 'ultimately': ['anticipation'],
            'ultimatum': ['anger', 'fear'], 'umpire': [ 'trust'],
            'unable': [ 'sadness'], 'unacceptable': [ 'sadness'],
            'unaccountable': ['anticipation', 'disgust',  'sadness', 'trust'],
            'unacknowledged': ['sadness'],
            'unanticipated': ['surprise'],
            'unattainable': ['anger',  'sadness'],
            'unattractive': ['disgust',  'sadness'],
            'unbearable': ['disgust',  'sadness'],
            'unbeaten': ['anticipation', 'joy',   'sadness', 'surprise'],


            'unbridled': ['anger', 'anticipation', 'fear',   'surprise'],
            'unbroken': [ 'trust'], 'uncanny': ['fear',  'surprise'],
            'uncaring': ['anger', 'disgust',  'sadness'],
            'uncertain': ['anger', 'disgust', 'fear',  'surprise'],
            'unclean': ['disgust'],
            'unconscionable': ['disgust'],
            'unconstrained': ['joy'],
            'uncontrollable': ['anger', 'anticipation',  'surprise'],
            'uncover': ['surprise'], 'undecided': ['anticipation', 'fear'],
            'underestimate': ['surprise'],
            'underpaid': ['anger',  'sadness'],
            'understanding': [ 'trust'], 'undertaker': ['sadness'], 'undertaking': ['anticipation'],
            'underwrite': [ 'trust'], 'undesirable': ['anger', 'disgust', 'fear',  'sadness'],
            'undesired': [ 'sadness'], 'undisclosed': ['anticipation'], 'undiscovered': ['surprise'],
            'undoubted': ['anticipation', 'disgust'],
            'undying': ['anticipation', 'joy',  'sadness', 'trust'],
            'uneasiness': ['anticipation',  'sadness'], 'uneasy': ['disgust', 'fear'],
            'uneducated': [ 'sadness'], 'unemployed': ['fear',  'sadness'],
            'unequal': ['anger', 'disgust', 'fear',  'sadness'], 'unequivocal': ['trust'],

            'unexpected': ['anticipation', 'fear', 'joy',   'surprise'],
            'unexpectedly': ['surprise'], 'unexplained': ['anticipation',  'sadness'],
            'unfair': ['anger', 'disgust',  'sadness'], 'unfairness': ['anger',  'sadness'],
            'unfaithful': ['disgust'], 'unfavorable': ['disgust',  'sadness'],
            'unfold': ['anticipation'], 'unforeseen': ['surprise'],
            'unforgiving': ['anger',  'sadness'], 'unfortunate': [ 'sadness'],
            'unfriendly': ['anger', 'disgust', 'fear',  'sadness'],
            'unfulfilled': ['anger', 'anticipation',  'sadness', 'surprise'],
            'ungodly': [ 'sadness'], 'ungrateful': ['anger', 'disgust'],
            'unguarded': ['surprise'], 'unhappiness': [ 'sadness'],
            'unhappy': ['anger', 'disgust',  'sadness'],
            'unhealthy': ['disgust', 'fear',  'sadness'], 'unholy': ['fear'],
            'unification': ['anticipation', 'joy',  'trust'],
            'unimaginable': [  'surprise'], 'unimportant': [ 'sadness'],

            'uninspired': [ 'sadness'],
            'unintended': ['surprise'], 'unintentional': ['surprise'],
            'unintentionally': [ 'surprise'], 'uninterested': [ 'sadness'],
            'uninteresting': [ 'sadness'], 'uninvited': ['sadness'], 'unique': [ 'surprise'],
            'united': [ 'trust'],
            'unity': [ 'trust'], 'university': ['anticipation'],
            'unjust': ['anger'], 'unjustifiable': ['anger', 'disgust', 'fear'],
            'unkind': ['anger', 'disgust', 'fear',  'sadness'],
            'unknown': ['anticipation', 'fear'],
            'unlawful': ['anger', 'disgust', 'fear',  'sadness'],
            'unlucky': ['anger', 'disgust', 'fear',  'sadness'],
            'unmanageable': ['disgust'], 'unnatural': ['disgust', 'fear'],
            'unpaid': ['anger',  'sadness'],
            'unpleasant': ['disgust',  'sadness'], 'unpopular': ['disgust',  'sadness'],
            'unprecedented': ['surprise'], 'unpredictable': [ 'surprise'],

            'unpublished': ['anticipation',  'sadness'],
            'unquestionable': [ 'trust'], 'unquestionably': [ 'trust'],
            'unquestioned': [ 'trust'], 'unreliable': [ 'trust'],
            'unrequited': [ 'sadness'], 'unresolved': ['anticipation'], 'unrest': ['fear', 'sadness'],
            'unruly': ['anger', 'disgust', 'fear'], 'unsafe': ['fear'],
            'unsatisfactory': ['disgust'], 'unsatisfied': ['disgust',  'sadness'],
            'unseat': ['sadness'],
            'unsettled': ['anger', 'disgust', 'fear'],
            'unsightly': ['disgust'],
            'unspeakable': ['fear'], 'unstable': ['fear',  'surprise'], 'unsteady': ['fear'],
            'unsuccessful': [ 'sadness'],
            'unsurpassed': ['anticipation', 'fear', 'joy',  'trust'],
            'unsuspecting': ['surprise'],  'unsympathetic': ['anger'],

            'unthinkable': ['anger', 'disgust', 'fear'], 'untidy': ['disgust'],
            'untie': ['joy'],  'untitled': [ 'sadness'],
            'untold': ['anticipation'], 'untoward': ['anger', 'disgust'],
            'untrustworthy': ['anger'],
            'unverified': ['anticipation'],  'unwashed': ['disgust'],
            'unwavering': [ 'trust'], 'unwelcome': [ 'sadness'],
            'unwell': [ 'sadness'],
            'unworthy': ['disgust'],
            'upheaval': ['anger', 'fear',  'sadness'],
            'uphill': ['anticipation', 'fear'],
            'uplift': ['anticipation', 'joy',  'trust'], 'upright': [ 'trust'],
            'uprising': ['anger', 'anticipation', 'fear'],
            'upset': ['anger',  'sadness'],
            'urgency': ['anticipation', 'fear', 'surprise'],
            'urgent': ['anticipation', 'fear',  'surprise'], 'urn': ['sadness'],
            'usher': [ 'trust'],
            'usual': [ 'trust'], 'usurp': ['anger'], 'usurped': ['anger', 'fear'],
            'utopian': ['anticipation', 'joy',  'trust'],
            'vacation': ['anticipation', 'joy'],
            'vacuous': ['disgust'],
            'vainly': ['disgust',  'sadness'],  'validity': ['fear'],
            'valor': [ 'trust'],
            'vampire': ['anger', 'disgust', 'fear'],  'vanish': ['surprise'],
            'vanished': ['fear',  'sadness', 'surprise'],
            'variable': ['surprise'],
            'varicella': ['disgust', 'fear',  'sadness'],
            'veal': ['sadness', 'trust'], 'veer': ['fear', 'surprise'],
            'vegetative': ['disgust',  'sadness'], 'vehement': ['anger', 'fear'],
            'vendetta': ['anger', 'fear',  'sadness'],
            'venerable': ['anticipation', 'joy',  'trust'],
            'vengeance': ['anger'], 'vengeful': ['anger', 'fear'],
            'venom': ['anger', 'disgust', 'fear'], 'venomous': ['anger', 'disgust', 'fear'],
            'vent': ['anger'], 'veracity': ['anticipation', 'joy',  'surprise', 'trust'],
            'verdict': ['fear'],
            'verge': ['anticipation', 'fear'], 'verification': [ 'trust'],
            'verified': [ 'trust'], 'verily': [ 'trust'],
            'vermin': ['anger', 'disgust', 'fear'], 'vernal': ['joy'],
            'versus': ['anger'], 'vertigo': ['fear'],
            'vesicular': ['disgust'], 'veteran': [ 'trust'], 'veto': ['anger'],
            'vicar': [ 'trust'],  'vicious': ['anger', 'disgust'],
            'victim': ['anger', 'fear',  'sadness'],
            'victimized': ['anger', 'disgust', 'fear',  'sadness', 'surprise'],
            'victor': ['joy'], 'victorious': ['joy'],
            'victory': ['anticipation', 'joy',  'trust'], 'vigil': ['anticipation'],
            'vigilance': ['anticipation',  'trust'], 'vigilant': ['fear',  'trust'],
            'vigorous': [ 'trust'], 'villager': [ 'trust'],
            'villain': ['fear'], 'villainous': ['anger', 'disgust', 'fear'],
            'vindicate': ['anger'],
            'vindication': ['anticipation', 'joy',  'trust'],
            'vindictive': ['anger', 'disgust'],
            'violation': ['anger', 'fear',  'sadness', 'surprise'],
            'violence': ['anger', 'fear',  'sadness'],
            'violent': ['anger', 'disgust', 'fear',  'surprise'],
            'violently': ['anger', 'disgust', 'fear',  'sadness'], 'viper': ['fear'],
            'virgin': [ 'trust'], 'virginity': ['anticipation'],
            'virtue': [ 'trust'], 'virtuous': ['joy',  'trust'],
            'virulence': ['anger', 'fear'],
            'vision': ['anticipation'], 'visionary': ['anticipation', 'joy',  'trust'],
            'visitor': ['anticipation', 'joy'],
            'visor': ['anticipation', 'surprise'],  'vitality': ['joy',  'trust'],
            'vivacious': ['joy'], 'vivid': ['joy'],
            'volatility': ['anger', 'anticipation', 'fear',  'surprise'],
            'volcano': ['fear',  'surprise'],
            'volunteer': ['anticipation', 'fear', 'joy',  'trust'], 'volunteers': ['trust'],
            'voluptuous': ['anticipation', 'joy'], 'vomit': ['disgust'],

            'vote': ['anger', 'anticipation', 'joy',   'sadness', 'surprise', 'trust'],
            'votive': ['trust'], 'vouch': [ 'trust'], 'voucher': ['trust'],
            'vow': ['anticipation', 'joy',  'trust'], 'voyage': ['anticipation'],
            'vulgar': ['disgust'], 'vulgarity': ['anger', 'disgust',  'sadness'],
            'vulnerability': ['fear',  'sadness'], 'vulture': ['disgust', 'fear'],
            'waffle': ['anger',  'sadness'], 'wages': ['joy'],
            'wail': ['fear',  'sadness'], 'wait': ['anticipation'],
            'wallow': ['disgust',  'sadness'], 'wan': ['fear',  'sadness'],
            'wane': [ 'sadness'], 'wanting': [ 'sadness'], 'war': ['fear'],
            'warden': ['anger', 'fear',  'trust'], 'ware': ['fear'],
            'warfare': ['anger', 'fear',  'sadness'], 'warlike': ['anger', 'fear'],
            'warlock': ['fear'], 'warn': ['anticipation', 'fear',  'surprise', 'trust'],
            'warned': ['anticipation', 'fear', 'surprise'], 'warning': ['fear'],
            'warp': ['anger',  'sadness'],  'warranty': [ 'trust'],
            'warrior': ['anger', 'fear'], 'wart': ['disgust'], 'wary': ['fear'],
            'waste': ['disgust'], 'wasted': ['anger', 'disgust'],
            'wasteful': ['anger', 'disgust',  'sadness'],
            'wasting': ['disgust', 'fear',  'sadness'], 'watch': ['anticipation', 'fear'],
            'watchdog': [ 'trust'], 'watchful': [ 'trust'], 'watchman': [ 'trust'],
            'waver': ['fear'],
            'weakly': ['fear',  'sadness'],
            'wealth': ['joy',  'trust'], 'wear': [ 'trust'],
            'wearily': [ 'sadness'], 'weariness': [ 'sadness'],
            'weary': [ 'sadness'],  'weeds': [ 'sadness'],
            'weep': [ 'sadness'], 'weeping': ['sadness'], 'weigh': ['anticipation', 'trust'],
            'weight': ['anticipation', 'disgust', 'fear', 'joy',   'sadness', 'surprise',
            'trust'], 'weighty': ['fear'], 'weird': ['disgust'],
            'weirdo': ['fear'], 'welcomed': ['joy'],
            'wench': ['anger', 'disgust'],
            'whim': ['anticipation', 'joy',  'surprise'], 'whimper': ['fear', 'sadness'],
            'whimsical': ['joy'], 'whine': ['disgust',  'sadness'], 'whip': ['anger'],
            'whirlpool': ['fear'], 'whirlwind': ['fear'],
            'white': ['anticipation', 'joy',  'trust'], 'whiteness': ['joy'],
            'wholesome': [ 'trust'], 'whore': ['disgust'], 'wicked': ['fear'],
            'wickedness': ['disgust'],
            'widow': ['sadness'], 'widower': ['sadness'], 'wild': [ 'surprise'],
            'wilderness': ['anticipation', 'fear', 'sadness'],
            'wildfire': ['fear',  'sadness', 'surprise'], 'willful': ['anger',  'sadness'],
            'wimp': ['disgust', 'fear'],
            'wimpy': ['anger', 'disgust', 'fear',  'sadness'],
            'wince': ['anger', 'disgust', 'fear',  'sadness'],
            'winner': ['anticipation', 'joy',  'surprise'],
            'winning': ['anticipation', 'disgust', 'joy',  'sadness', 'surprise', 'trust'],
            'winnings': ['anticipation', 'joy'],
            'wireless': ['anger', 'anticipation',  'surprise'],
            'wisdom': [ 'trust'],  'wishful': ['anticipation'],
            'witch': ['anger', 'disgust', 'fear'],
            'witchcraft': ['anger', 'fear',  'sadness'], 'withdraw': [ 'sadness'],
            'wither': [ 'sadness'], 'withered': ['disgust'],
            'withstand': ['anticipation', 'fear'], 'witness': ['trust'],
            'witty': ['joy'], 'wizard': ['anticipation',  'surprise'],
            'woe': ['disgust', 'fear',  'sadness'], 'woeful': [ 'sadness'],
            'woefully': ['disgust',  'sadness'],
            'wonderful': ['joy',  'surprise', 'trust'], 'wonderfully': ['joy',  'surprise'],
            'wont': ['anticipation'], 'wop': ['anger'], 'word': [ 'trust'],
            'words': ['anger'],
            'worm': ['anticipation',  'surprise'], 'worn': [ 'sadness'],
            'worried': [ 'sadness'], 'worry': ['anticipation', 'fear',  'sadness'],
            'worrying': ['anticipation', 'fear',  'sadness'], 'worse': ['fear',  'sadness'],
            'worsening': ['disgust',  'sadness'],
            'worship': ['anticipation', 'fear', 'joy',  'trust'],
            'worthless': ['anger', 'disgust',  'sadness'], 'worthy': [ 'trust'],
            'wot': [ 'trust'], 'wound': ['anger', 'fear',  'sadness'],
            'wrangling': ['anger', 'disgust', 'fear',  'sadness'], 'wrath': ['anger', 'fear'],
            'wreak': ['anger'], 'wreck': ['anger', 'disgust', 'fear',  'sadness', 'surprise'],
            'wrecked': ['anger', 'fear',  'sadness'],
            'wretch': ['anger', 'disgust',  'sadness'], 'wretched': ['disgust',  'sadness'],
            'wring': ['anger'], 'wrinkled': ['sadness'],
            'wrongdoing': ['anger', 'disgust',  'sadness'],
            'wrongful': ['anger', 'disgust',  'sadness'],
            'wrongly': ['anger', 'fear',  'sadness'],
            'xenophobia': ['fear'],
            'yearning': ['anticipation', 'joy',   'trust'],
            'yell': ['anger', 'fear',  'surprise'],
            'yelp': ['anger', 'fear',  'surprise'],
            'young': ['anticipation', 'joy',  'surprise'],
            'youth': ['anger', 'anticipation', 'fear', 'joy',  'surprise'], 'zany': ['surprise'],
            'zeal': ['anticipation', 'joy',  'surprise', 'trust'],
            'zealous': ['joy',  'trust'], 'zest': ['anticipation', 'joy',  'trust']}
    wordnet_lemmatizer = WordNetLemmatizer()
    tk = TweetTokenizer()
    stop_words = set(stopwords.words('english'))
    # initialize lexicon
    def __init__(self, emoji_factor = 1):
        # load lexico from https://github.com/abushoeb/EmoTag for emoji emotion detection
        #ncr = pd.read_csv('resource_folder/lexicon.csv', sep =';')
        emoji_lexicon = pd.read_csv('resource_folder/EmoTag1200-scores.csv', sep =',')
        self.emoji_dict = emoji_lexicon.set_index('emoji').T.to_dict()
        self.emoji_factor = emoji_factor

    def process(self, dataFrame, columnName='text'):
        print("begin extract emoji ")
        emoji_summary = adv.extract_emoji(dataFrame[columnName])
        dataFrame['emojis'] = emoji_summary['emoji']
        #dataFrame['tokens'] = dataFrame[columnName].apply(EmotionDetector.pre_process)
        dataFrame['tokens'] = dataFrame[columnName].progress_apply(EmotionDetector.pre_process)
        res = dataFrame.progress_apply(self.emotion_analysis,axis=1)
        result = pd.concat([dataFrame, res], axis=1)
        return result

    def emotion_analysis(self, row):
        #print("1111")
        #print(row)
        words = row["tokens"]
        emojis = row['emojis']
        # refer NRCLex
        affect_list = []
        affect_dict = dict()
        affect_frequencies = Counter()
        lexicon_keys = EmotionDetector.lexicon.keys()
        for word in words:
            if word in lexicon_keys:
                affect_list.extend(EmotionDetector.lexicon[word])
                affect_dict.update({word: EmotionDetector.lexicon[word]})
        for word in affect_list:
            affect_frequencies[word] += 1



        affect_percent = {'anticipation': 0.0, 'trust': 0.0, 'surprise': 0.0,
                           'sadness': 0.0, 'disgust': 0.0, 'joy': 0.0}

        emoji_lexicon_keys = self.emoji_dict.keys();
        for emoji in emojis:
            #print(emoji)
            if emoji in emoji_lexicon_keys:
                emoji_affect_dic = self.emoji_dict[emoji]
                for key in affect_percent.keys():
                    if emoji_affect_dic[key] > 0:
                        affect_frequencies[key] += emoji_affect_dic[key] * self.emoji_factor
                        affect_list.append(key)
                        #affect_dict.update({emoji: {affect:emoji_affect_dic[affect] for affect in emoji_affect_dic if emoji_affect_dic[affect] >0}})
        sum_values = sum(affect_frequencies.values())
        for key in affect_frequencies.keys():
            #print(affect_frequencies)
            affect_percent.update({key: float(affect_frequencies[key]) / float(sum_values)})
        emo_dict = affect_frequencies
        top_emotions = []
        if emo_dict.__len__()>0:
            max_value = max(emo_dict.values())
            for key in emo_dict.keys():
                if emo_dict[key] == max_value:
                    top_emotions.append((key, max_value))

        return pd.Series([affect_list,affect_dict,dict(affect_frequencies),affect_percent,top_emotions],
                         index=['affect_list', 'affect_dict', 'affect_frequencies', 'affect_percent','top_emotions'])

    @staticmethod
    def pre_process(text):
        #print(text)
        # clean text
        text = EmotionDetector.cleanText(str(text))
        #print(text)
        #emoji_summary = adv.extract_emoji([text])
        #print(emoji_summary['emoji'])
        #tokenization = nltk.word_tokenize(text)
        tokenization = EmotionDetector.tk.tokenize(text)

        filtered_words = [word for word in tokenization if word not in EmotionDetector.stop_words]
        map_object = map(EmotionDetector.wordnet_lemmatizer.lemmatize, filtered_words)
        lemma_words = list(map_object)
        #print(lemma_words)
        return lemma_words


    @staticmethod
    def cleanText(text):
        text=text.lower()
        text = re.sub(r'@\w+','', text) # remove @member
        text = re.sub(r'#','', text) # remove # symbol
        text = re.sub(r'https?:\/\/\S+','', text) # remove the hyper link
        return text

In [132]:
# test
ed = EmotionDetector()
res = ed.process(tweets_short)
res.head(100)

<ipython-input-125-b7a30d5378f4>:2837: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame['emojis'] = emoji_summary['emoji']
100%|██████████| 10000/10000 [00:04<00:00, 2326.60it/s]
<ipython-input-125-b7a30d5378f4>:2839: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame['tokens'] = dataFrame[columnName].progress_apply(EmotionDetector.pre_process)
100%|██████████| 10000/10000 [00:03<00:00, 2615.58it/s]


,tweetcreatedts,text,emojis,tokens,affect_list,affect_dict,affect_frequencies,affect_percent,top_emotions
0,2022-04-01 00:00:00.000000,⚡The Ukrainian Air Force would like to address...,"[⚡, 🇺🇦, 🧵]","[⚡, ukrainian, air, force, would, like, addres...","[anger, fear, trust, anticipation, trust, surp...","{'force': ['anger', 'fear'], 'ally': ['trust']}","{'anger': 1, 'fear': 1, 'trust': 1.14000000000...","{'anticipation': 0.07848101265822785, 'trust':...","[(trust, 1.1400000000000001)]"
1,2022-04-01 00:00:00.000000,Chernihiv oblast. Ukrainians welcome their lib...,[],"[chernihiv, oblast, ., ukrainian, welcome, lib...",[],{},{},"{'anticipation': 0.0, 'trust': 0.0, 'surprise'...",[]
2,2022-04-01 00:00:00.000000,America 🇺🇸 is preparing for something worse th...,"[🇺🇸, 🇹🇼, 🇷🇺, 🇺🇸, 🇨🇳, 👇]","[america, 🇺, 🇸, preparing, something, worse, r...","[fear, sadness, fear, trust, anticipation, trust]","{'worse': ['fear', 'sadness'], 'war': ['fear']...","{'fear': 2, 'sadness': 1, 'trust': 1.11, 'anti...","{'anticipation': 0.03294117647058824, 'trust':...","[(fear, 2)]"
3,2022-04-01 00:00:00.000000,JUST IN: #Anonymous has hacked &amp; released ...,[],"[:, anonymous, hacked, &, released, 62,000, em...",[],{},{},"{'anticipation': 0.0, 'trust': 0.0, 'surprise'...",[]
4,2022-04-01 00:00:00.000000,***PUBLIC MINT NOW LIVE***\n\nFor \n@billionai...,[],"[*, *, *, public, mint, live, *, *, *, win, $,...","[anticipation, anticipation, anticipation, joy...","{'public': ['anticipation'], 'good': ['anticip...","{'anticipation': 4, 'joy': 2, 'surprise': 2, '...","{'anticipation': 0.4444444444444444, 'trust': ...","[(anticipation, 4)]"
...,...,...,...,...,...,...,...,...,...
123,2022-04-01 00:00:30.000000,💙💛 Cover of the April issue of Polish Vogue \...,"[💙, 💛]","[💙, 💛, cover, april, issue, polish, vogue, cre...","[trust, anticipation, trust, surprise, sadness...",{'cover': ['trust']},"{'trust': 1.6400000000000001, 'anticipation': ...","{'anticipation': 0.09729729729729729, 'trust':...","[(trust, 1.6400000000000001)]"
124,2022-04-01 00:00:30.000000,#Ukrainian forces successfully conducted local...,[],"[ukrainian, force, successfully, conducted, lo...","[anger, fear, anticipation]","{'force': ['anger', 'fear'], 'coming': ['antic...","{'anger': 1, 'fear': 1, 'anticipation': 1}","{'anticipation': 0.3333333333333333, 'trust': ...","[(anger, 1), (fear, 1), (anticipation, 1)]"
125,2022-04-01 00:00:31.000000,⚡The Ukrainian Air Force would like to address...,"[⚡, 🇺🇦, 🧵]","[⚡, ukrainian, air, force, would, like, addres...","[anger, fear, trust, anticipation, trust, surp...","{'force': ['anger', 'fear'], 'ally': ['trust']}","{'anger': 1, 'fear': 1, 'trust': 1.14000000000...","{'anticipation': 0.07848101265822785, 'trust':...","[(trust, 1.1400000000000001)]"
127,2022-04-01 00:00:31.000000,❌Myth: The US and @NATO are providing #Ukraine...,"[❌, ✅]","[❌, myth, :, u, providing, ukraine, needed, we...","[anticipation, joy, trust, trust, anger, fear,...","{'providing': ['anticipation', 'joy', 'trust']...","{'anticipation': 2.45, 'joy': 1.19, 'trust': 4...","{'anticipation': 0.18846153846153849, 'trust':...","[(trust, 4.39)]"


In [134]:
res.to_csv("res.csv")

,tweetcreatedts,text,tokens,emoji
0,2022-04-01 00:00:00.000000,⚡The Ukrainian Air Force would like to address...,"[⚡, ukrainian, air, force, would, like, addres...","[⚡, 🇺🇦, 🧵]"
1,2022-04-01 00:00:00.000000,Chernihiv oblast. Ukrainians welcome their lib...,"[chernihiv, oblast, ., ukrainian, welcome, lib...",[]
2,2022-04-01 00:00:00.000000,America 🇺🇸 is preparing for something worse th...,"[america, 🇺, 🇸, preparing, something, worse, r...","[🇺🇸, 🇹🇼, 🇷🇺, 🇺🇸, 🇨🇳, 👇]"
3,2022-04-01 00:00:00.000000,JUST IN: #Anonymous has hacked &amp; released ...,"[:, anonymous, hacked, &, released, 62,000, em...",[]
4,2022-04-01 00:00:00.000000,***PUBLIC MINT NOW LIVE***\n\nFor \n@billionai...,"[*, *, *, public, mint, live, *, *, *, win, $,...",[]
5,2022-04-01 00:00:00.000000,The Amazing story of Former Sussex County Dela...,"[amazing, story, former, sussex, county, delaw...",[🇺🇸]
6,2022-04-01 00:00:00.000000,&amp;quot;How we were waiting for you!&amp;quo...,"[&, quot, ;, waiting, !, &, quot, ;:, chernihi...",[]
7,2022-04-01 00:00:00.000000,India's purchase of discounted #Russian crude ...,"[india's, purchase, discounted, russian, crude...",[]
8,2022-04-01 00:00:00.000000,The most basic tenet of what stability exists ...,"[basic, tenet, stability, exists, world, borde...",[]
9,2022-04-01 00:00:00.000000,"""The image that Russia acquired over the past ...","["", image, russia, acquired, past, two, decade...",[]
